In [ ]:
!pip install google-cloud-bigquery --quiet

In [ ]:
!pip install --quiet 'protobuf<5' chromadb --upgrade
!pip install --quiet sentence-transformers transformers accelerate torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 124.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 189.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 187.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 39.4 M

In [ ]:
from google.cloud import bigquery

# Set the project ID explicitly if the client doesn't pick it up automatically
PROJECT_ID = 'stgw-shared-ai-dev'

# The BigQuery client should automatically use the Service Account
# associated with your Colab Enterprise notebook/runtime.
client = bigquery.Client(project=PROJECT_ID)

print(f"BigQuery Client initialized for project: {PROJECT_ID}")
# You can now proceed to run your BigQuery queries (CREATE TABLE, SELECT, etc.)

In [ ]:
query = """
SELECT
  predicted_cluster,
  COUNT(*) AS num_records,
  ROUND(COUNT(*) * 100 / SUM(COUNT(*)) OVER (), 2) AS pct_of_total
FROM `stgw-shared-ai-dev.parquet_data_set.user_embeddings_50D_clusters`
GROUP BY predicted_cluster
ORDER BY predicted_cluster;
"""

cluster_distribution = client.query(query).to_dataframe()
cluster_distribution


In [ ]:
query = """
SELECT *
FROM ML.CENTROIDS(MODEL `stgw-shared-ai-dev.parquet_data_set.kmeans_segmentation_50D`);
"""

centroids = client.query(query).to_dataframe()
centroids.head()


In [ ]:
query = """
CREATE OR REPLACE TABLE `stgw-shared-ai-dev.parquet_data_set.clustered_profiles` AS
SELECT
  c.predicted_cluster,
  f.*
FROM `stgw-shared-ai-dev.parquet_data_set.user_embeddings_50D_clusters` c
JOIN `stgw-shared-ai-dev.parquet_data_set.IMP_profile_FE_SAMPLE` f
  USING (extern_tuid);
"""
client.query(query).result()

In [ ]:
query = """
SELECT *
FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles`
LIMIT 10
"""
df = client.query(query).to_dataframe()
df


In [ ]:
# Step 1: Get all columns
cols_df = client.query("""
SELECT column_name
FROM `stgw-shared-ai-dev.parquet_data_set`.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'clustered_profiles'
""").to_dataframe()

# Step 2: Keep only those ending with '_imputed'
imputed_cols = [c for c in cols_df['column_name'] if c.endswith('_imputed')]

# Step 3: Build your SQL
cols_select = ',\n  '.join(['extern_tuid', 'predicted_cluster'] + imputed_cols)

query = f"""
CREATE OR REPLACE TABLE `stgw-shared-ai-dev.parquet_data_set.clustered_profiles_imputed_only` AS
SELECT
  {cols_select}
FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles`
"""

# Step 4: Run
client.query(query).result()
print("✅ Created table: clustered_profiles_imputed_only")



In [ ]:
query = """
SELECT *
FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles_imputed_only`
LIMIT 10
"""
df = client.query(query).to_dataframe()
df

In [ ]:
for c in df.columns:
    print(c)


In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

query = """
SELECT COUNT(*) AS row_count
FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles_imputed_only`
"""

result = client.query(query).to_dataframe()
print(f"✅ Total rows: {result['row_count'][0]:,}")


In [ ]:
# Step 1: Get all column names
cols_df = client.query("""
SELECT column_name
FROM `stgw-shared-ai-dev.parquet_data_set`.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'clustered_profiles_imputed_only'
""").to_dataframe()

# Step 2: Filter for imputed columns
imputed_cols = [c for c in cols_df['column_name'] if c.endswith('_imputed')]

# Step 3: Dynamically create CASE expressions
case_clauses = [
    f"CASE WHEN TRIM({col}) = '' OR {col} IS NULL THEN '__MISSING__' ELSE {col} END AS {col}"
    for col in imputed_cols
]

# Step 4: Combine into query
cols_select = ',\n  '.join(['extern_tuid', 'predicted_cluster'] + case_clauses)

query = f"""
CREATE OR REPLACE TABLE `stgw-shared-ai-dev.parquet_data_set.clustered_profiles_imputed_only_cleaned` AS
SELECT
  {cols_select}
FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles_imputed_only`
"""

# Step 5: Execute
job = client.query(query)
job.result()
print("✅ Cleaned table created: clustered_profiles_imputed_only_cleaned")


In [ ]:
!pip install google-cloud-bigquery --quiet
from google.cloud import bigquery
import pandas as pd

# --- 1️⃣ Connect to BigQuery ---
client = bigquery.Client(project="stgw-shared-ai-dev")

# --- 2️⃣ Table reference ---
dataset = "parquet_data_set"
table = "clustered_profiles_imputed_only_cleaned"
dataset_ref = f"{client.project}.{dataset}"

# --- 3️⃣ Get column names & datatypes ---
query_columns = f"""
SELECT column_name, data_type
FROM `{dataset_ref}.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = '{table}'
ORDER BY ordinal_position
"""
cols = client.query(query_columns).to_dataframe()

print(f"✅ Found {len(cols)} columns to profile.")

# --- 4️⃣ Safe quoting function ---
def safe(col):
    return f"`{col.replace('`', '')}`"

# --- 5️⃣ Loop over columns and compute stats ---
profiles = []
for _, row in cols.iterrows():
    col, dtype = row["column_name"], row["data_type"]
    cq = safe(col)

    q = f"""
    SELECT
      '{col}' AS column_name,
      '{dtype}' AS data_type,
      COUNT(*) AS total_rows,
      COUNT(DISTINCT {cq}) AS unique_values,
      ROUND(COUNT(DISTINCT {cq}) * 100.0 / COUNT(*), 2) AS uniqueness_pct,
      (
        SELECT ARRAY_AGG(STRUCT(val, freq) ORDER BY freq DESC LIMIT 3)
        FROM (
          SELECT {cq} AS val, COUNT(*) AS freq
          FROM `{dataset_ref}.{table}`
          WHERE {cq} IS NOT NULL
          GROUP BY {cq}
          ORDER BY freq DESC
          LIMIT 3
        )
      ) AS top_values,
      (
        SELECT SAFE_DIVIDE(MAX(freq), SUM(freq))
        FROM (
          SELECT {cq} AS val, COUNT(*) AS freq
          FROM `{dataset_ref}.{table}`
          WHERE {cq} IS NOT NULL
          GROUP BY {cq}
        )
      ) AS imbalance_ratio
    FROM `{dataset_ref}.{table}`
    LIMIT 1
    """

    try:
        subdf = client.query(q).to_dataframe()
        profiles.append(subdf)
    except Exception as e:
        print(f"⚠️ Skipping '{col}' — {e}")
        continue

# --- 6️⃣ Combine all column profiles ---
if profiles:
    df_profile = pd.concat(profiles, ignore_index=True)
else:
    raise RuntimeError("No valid column profiles generated.")

# --- 7️⃣ Recommendations based on value diversity ---
def rec(row):
    if row.unique_values <= 1:
        return "⚠️ Constant — drop"
    if row.unique_values < 10:
        return "✅ Good categorical"
    if row.unique_values < 50:
        return "ℹ️ Moderate — check imbalance"
    if row.unique_values < 200:
        return "🧩 Consider binning/grouping"
    return "📉 Continuous / high-cardinality"

df_profile["recommendation"] = df_profile.apply(rec, axis=1)

# --- 8️⃣ Display the summarized profile ---
pd.set_option("display.max_colwidth", None)
df_profile = df_profile.sort_values("unique_values", ascending=True)

df_profile[[
    "column_name", "data_type", "unique_values",
    "uniqueness_pct", "imbalance_ratio", "recommendation", "top_values"
]].head(40)
df_profile.to_csv("/content/cluster_profile_stats.csv", index=False)
print("✅ Saved profiling summary to /content/cluster_profile_stats.csv")


In [ ]:
import pandas as pd

# Load the CSV you created earlier
df_profile = pd.read_csv("/content/cluster_profile_stats.csv")

# Quick check
print("📊 Columns profiled:", len(df_profile))
df_profile.head()


In [ ]:
import pandas as pd

df_profile = pd.read_csv("/content/cluster_profile_stats.csv")
print("✅ Columns in profiling summary:", len(df_profile))
df_profile.head()


In [ ]:
# Keep categorical-like columns (based on recommendation)
df_selected = df_profile[
    df_profile["recommendation"].isin([
        "✅ Good categorical",
        "ℹ️ Moderate — check imbalance",
        "🧩 Consider binning/grouping"
    ])
]

# Keep columns that are not extremely unique or skewed
df_selected = df_selected[
    (df_selected["uniqueness_pct"] < 50) &
    (df_selected["imbalance_ratio"] < 0.9)
]

print("✅ Columns selected for categorical encoding:", len(df_selected))
df_selected[["column_name", "unique_values", "imbalance_ratio", "recommendation"]].head(20)


In [ ]:
selected_columns = df_selected["column_name"].tolist()
print(f"✅ {len(selected_columns)} columns chosen for clustering.")


In [ ]:
df_selected[["column_name"]]

In [ ]:
from google.cloud import bigquery
client = bigquery.Client(project="stgw-shared-ai-dev")

dataset = "parquet_data_set"
table = "clustered_profiles_imputed_only_cleaned"
dataset_ref = f"{client.project}.{dataset}"

query = f"""
SELECT {', '.join([f'`{col}`' for col in selected_columns])}
FROM `{dataset_ref}.{table}`
"""
df = client.query(query).to_dataframe()
print("✅ Loaded dataset with selected columns:", df.shape)


In [ ]:
def cap_top_values(series, top_n=100):
    top_values = series.value_counts().nlargest(top_n).index
    return series.where(series.isin(top_values), other="Other")

df_encoded = df.copy()

for col in selected_columns:
    df_encoded[col] = cap_top_values(df_encoded[col].astype(str))


In [ ]:
non_null_pct = df_encoded.notnull().mean().sort_values(ascending=False)
df_nonnull = non_null_pct.reset_index()
df_nonnull.columns = ["column", "filled_pct"]

# Keep columns that are at least 70% filled
df_final = df_encoded[df_nonnull[df_nonnull["filled_pct"] > 0.7]["column"]]
print("✅ Columns after completeness filtering:", df_final.shape[1])


In [ ]:
df_final.to_csv("/content/cluster_ready_dataset.csv", index=False)
print("💾 Saved final clustering-ready dataset → /content/cluster_ready_dataset.csv")


In [ ]:
# Explore the CSV
import pandas as pd

# Load the file you just saved
df_ready = pd.read_csv("/content/cluster_ready_dataset.csv")

print("✅ Dataset loaded successfully!")
print("Shape:", df_ready.shape)
df_ready.head(5)


In [ ]:
#Shape
df_ready.shape

In [ ]:
# Quick summary of completeness & diversity
# Percentage of non-null values per column
filled_pct = df_ready.notnull().mean().sort_values(ascending=False)

# Number of unique values per column (after top 100 filtering)
unique_counts = df_ready.nunique().sort_values(ascending=False)

summary = pd.DataFrame({
    "filled_pct": filled_pct,
    "unique_values": unique_counts
}).reset_index().rename(columns={"index": "column"})

summary.head(20)

In [ ]:
# Preview distinct examples from the dataset
df_ready.sample(10, random_state=42)


In [ ]:
#Check the top categories for a few key columns
for c in df_ready.columns[:5]:
    print(f"\n🧠 {c}")
    print(df_ready[c].value_counts().head(10))

# **FLOW 1**

In [ ]:

import pandas as pd, numpy as np, os
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from tqdm import tqdm

# --- Parameters ---
DATA_PATH = "/content/cluster_ready_dataset.csv"
SAMPLE_FRAC = 0.02   # 2% sample of 1.3M rows ≈ 26K
N_CLUSTERS = 12      # adjust based on diversity you expect
MIN_CLUSTER_SIZE = 50
SILHOUETTE_Q = 0.25
MAX_PERSONA_EXAMPLES = 100

# --- 1️⃣ Load and sample ---
df = pd.read_csv(DATA_PATH)
df_sample = df.sample(frac=SAMPLE_FRAC, random_state=42).reset_index(drop=True)
print("✅ Sampled shape:", df_sample.shape)

# --- 2️⃣ Combine all categorical columns into a descriptive text per row ---
def build_persona_text(row):
    parts = []
    for col, val in row.items():
        if pd.notna(val) and str(val).lower() not in ("nan", "none", "__missing__"):
            parts.append(f"{col.replace('_imputed','').replace('_',' ')}: {val}")
    return " | ".join(parts)

tqdm.pandas(desc="Building persona text")
df_sample["persona_text"] = df_sample.progress_apply(build_persona_text, axis=1)

# --- 3️⃣ Encode with sentence embeddings ---
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = embed_model.encode(df_sample["persona_text"].tolist(), show_progress_bar=True)

# --- 4️⃣ Cluster using MiniBatchKMeans ---
kmeans = MiniBatchKMeans(n_clusters=N_CLUSTERS, random_state=42, batch_size=2048, n_init=10)
df_sample["cluster"] = kmeans.fit_predict(embeddings)

# --- 5️⃣ Filter small clusters ---
valid_clusters = df_sample["cluster"].value_counts()[lambda x: x >= MIN_CLUSTER_SIZE].index
df_core = df_sample[df_sample["cluster"].isin(valid_clusters)].copy()

# --- 6️⃣ Compute silhouette scores (approx) ---
try:
    sil = silhouette_samples(embeddings, df_sample["cluster"])
    df_sample["silhouette"] = sil
except:
    df_sample["silhouette"] = np.nan

# --- 7️⃣ Build cluster summaries ---
def mode_safe(s): s = s.dropna().astype(str); return s.mode().iloc[0] if not s.mode().empty else None

cluster_summary = []
for cid, grp in df_core.groupby("cluster"):
    row = {"cluster": int(cid), "size": len(grp)}
    for c in df.columns:
        row[c] = mode_safe(grp[c])
    row["persona_examples"] = " | ".join(grp["persona_text"].head(MAX_PERSONA_EXAMPLES).tolist())[:1000]
    cluster_summary.append(row)

cluster_summary = pd.DataFrame(cluster_summary)
print("✅ Generated cluster summaries:", cluster_summary.shape)

# --- 8️⃣ Summarize personas (text generation) ---
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=-1)

def summarize_persona(r):
    prompt = f"""
    Persona of individuals described by:
    Gender: {r.get('gender_imputed','Unknown')},
    Age: {r.get('age_imputed','Unknown')},
    Urbanicity: {r.get('urbanicity_imputed','Unknown')},
    Income decile: {r.get('family_income_decile_imputed','Unknown')},
    Credit tier: {r.get('credit_tier_imputed','Unknown')}.
    Key behavioral traits: {r.get('persona_examples','')[:600]}
    """
    try:
        out = summarizer(prompt, max_length=80, min_length=25, truncation=True)
        return out[0]['summary_text']
    except:
        return prompt[:200]

tqdm.pandas(desc="Summarizing clusters")
cluster_summary["persona_summary"] = cluster_summary.progress_apply(summarize_persona, axis=1)

# --- 9️⃣ Save results ---
cluster_summary.to_csv("/content/cluster_personas_summary_v2.csv", index=False)
print("📁 Saved summarized personas → /content/cluster_personas_summary_v2.csv")


In [ ]:
#Inspect the personas right now
import pandas as pd

df_personas = pd.read_csv("/content/cluster_personas_summary_v2.csv")
print("Personas:", df_personas.shape)
df_personas[["cluster", "size", "persona_summary"]].head(10)

In [ ]:
#View one full persona in detail
# View a specific persona cluster in detail
cluster_id = 3  # change as needed
df_personas[df_personas["cluster"] == cluster_id].T


In [ ]:
#View one full persona in detail
# View a specific persona cluster in detail
cluster_id = 3  # change as needed
df_personas[df_personas["cluster"] == cluster_id].T


In [ ]:


import chromadb
from sentence_transformers import SentenceTransformer

# Load summarized personas
cluster_summary = pd.read_csv("/content/cluster_personas_summary_v2.csv")
print("✅ Loaded summarized personas:", cluster_summary.shape)

# Set up persistent directory for Chroma
PERSIST_DIR = "/content/chroma_store"
os.makedirs(PERSIST_DIR, exist_ok=True)

# Initialize Chroma client and collection
chroma_client = chromadb.PersistentClient(path=PERSIST_DIR)
try:
    chroma_client.delete_collection("cluster_personas_clean_v3")
except:
    pass
collection = chroma_client.create_collection("cluster_personas_clean_v3")

# Embed persona summaries
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
cluster_embeddings = embed_model.encode(cluster_summary["persona_summary"].tolist(), show_progress_bar=True)

# Add to ChromaDB
collection.add(
    ids=[str(i) for i in cluster_summary["cluster"].tolist()],
    embeddings=[list(map(float, e)) for e in cluster_embeddings],
    documents=cluster_summary["persona_summary"].tolist(),
    metadatas=cluster_summary.to_dict(orient="records")
)

print("✅ Persona collection saved to Chroma → cluster_personas_clean_v3")


In [ ]:


def search_personas(query_text, top_k=3):
    query_embedding = embed_model.encode([query_text]).tolist()
    results = collection.query(query_embeddings=query_embedding, n_results=top_k)

    print(f"\n🔎 Query: {query_text}\n")
    for doc, meta, score in zip(results["documents"][0], results["metadatas"][0], results["distances"][0]):
        print(f"🎯 Cluster {meta['cluster']} (size={meta['size']}) — Distance={score:.3f}")
        print(meta.get("persona_summary", "")[:250])
        print("-"*90)

# Example searches
search_personas("tech-savvy urban professionals who stream online content")
search_personas("rural families with moderate income and auto loans")
search_personas("affluent parents interested in travel and luxury shopping")


In [ ]:
#Pretty Output View (Pandas Table)
import pandas as pd

def search_personas_df(query_text, top_k=5):
    query_embedding = embed_model.encode([query_text]).tolist()
    results = collection.query(query_embeddings=query_embedding, n_results=top_k)

    rows = []
    for doc, meta, score in zip(results["documents"][0], results["metadatas"][0], results["distances"][0]):
        rows.append({
            "Cluster": meta["cluster"],
            "Size": meta["size"],
            "Distance": round(score, 3),
            "Gender": meta.get("gender_imputed", ""),
            "Age": meta.get("age_imputed", ""),
            "Urbanicity": meta.get("urbanicity_imputed", ""),
            "Income": meta.get("family_income_decile_imputed", ""),
            "Summary": meta.get("persona_summary", "")[:250],
        })
    return pd.DataFrame(rows)

search_personas_df("environmentally conscious young adults who prefer green brands")


In [ ]:


from transformers import pipeline
import random


try:
    cluster_summary
except NameError:
    cluster_summary = pd.read_csv("/content/cluster_personas_summary_v2.csv")

# ------------------------------------------------------------
# 2️⃣ Initialize Chat Model (Small Instruction Model for Roleplay)
# ------------------------------------------------------------
chat_model = pipeline(
    "text-generation",
    model="mistralai/Mistral-7B-Instruct-v0.2",
    max_new_tokens=220,
    do_sample=True,
    temperature=0.7,
    pad_token_id=2
)

def generate_persona_name(cluster_id, gender, age):
    male_names = ["Raj", "Amit", "Rohan", "Vikram", "Aditya", "Sanjay", "Arjun", "Kunal"]
    female_names = ["Priya", "Ananya", "Sneha", "Neha", "Isha", "Kavya", "Ritu", "Aparna"]
    neutral_names = ["Alex", "Taylor", "Jordan", "Riya", "Sam", "Kiran"]

    if isinstance(gender, str):
        g = gender.lower()
        if "male" in g:
            name = random.choice(male_names)
        elif "female" in g:
            name = random.choice(female_names)
        else:
            name = random.choice(neutral_names)
    else:
        name = random.choice(neutral_names)

    persona_title = f"Cluster {cluster_id} Persona ({name}, {age})"
    return name, persona_title


def get_persona_context(cluster_id, persona_name=None):
    row = cluster_summary[cluster_summary["cluster"] == cluster_id]
    if row.empty:
        return None, None

    persona = row.iloc[0]
    if persona_name is None:
        name, persona_title = generate_persona_name(
            cluster_id,
            persona.get("gender_imputed", "Unknown"),
            persona.get("age_imputed", "Unknown")
        )
    else:
        name = persona_name
        persona_title = f"Cluster {cluster_id} Persona ({name}, {persona.get('age_imputed', 'Unknown')})"

    persona_context = f"""
You are {name}, representing behavioral Cluster {cluster_id} in a consumer segmentation study.

### Persona Summary
"{persona.get('persona_summary', '')}"

### Demographics
- Age Group: {persona.get('age_imputed', 'Unknown')}
- Gender: {persona.get('gender_imputed', 'Unknown')}
- Urbanicity: {persona.get('urbanicity_imputed', 'Unknown')}

### Financial & Lifestyle Indicators
- Income Decile: {persona.get('family_income_decile_imputed', 'Unknown')}
- Credit Tier: {persona.get('credit_tier_imputed', 'Unknown')}
- Observed Behaviors: {persona.get('persona_examples', '')[:600]}

### Response Guidelines
- Speak as a *real person* would — conversational, opinionated, and relatable.
- Reflect attitudes and choices typical of this cluster.
- Provide specific opinions about products, spending, investments, and lifestyle.
- Stay consistent with your demographic and behavioral profile.
"""
    return persona_context.strip(), name


def chat_with_persona(cluster_id, user_input, chat_history=None, persona_name=None):
    if chat_history is None:
        chat_history = []

    persona_context, name = get_persona_context(cluster_id, persona_name)
    if not persona_context:
        return "❌ Cluster not found.", chat_history, persona_name

    dialogue_context = "\n".join([f"User: {u}\n{name}: {r}" for u, r in chat_history])
    full_prompt = f"{persona_context}\n\n{dialogue_context}\nUser: {user_input}\n{name}:"

    output = chat_model(full_prompt)
    raw_reply = output[0]["generated_text"]
    reply = raw_reply.split(f"{name}:")[-1].strip()

    chat_history.append((user_input, reply))
    return reply, chat_history, name

# ------------------------------------------------------------
# 6️⃣ Example Conversations
# ------------------------------------------------------------
cluster_id = 1      # use cluster from search_personas() output
chat_history = []
persona_name = None

print(f"\n🗣️ Chatting with Cluster {cluster_id} Persona...\n")

# Lifestyle / Weekend habits
reply, chat_history, persona_name = chat_with_persona(
    cluster_id,
    "Hey! How do you usually spend your weekends?",
    chat_history,
    persona_name
)
print(f"{persona_name}:", reply)

# Product preferences
reply, chat_history, persona_name = chat_with_persona(
    cluster_id,
    "What kind of products or brands do you like to shop from?",
    chat_history,
    persona_name
)
print(f"{persona_name}:", reply)

# Entertainment & streaming
reply, chat_history, persona_name = chat_with_persona(
    cluster_id,
    "What type of movies or OTT shows do you enjoy?",
    chat_history,
    persona_name
)
print(f"{persona_name}:", reply)

# Investments & income behavior
reply, chat_history, persona_name = chat_with_persona(
    cluster_id,
    "Do you invest in mutual funds, property, or prefer savings accounts?",
    chat_history,
    persona_name
)
print(f"{persona_name}:", reply)


def focus_group(question, cluster_list=[0,1,3,5]):
    print(f"\n🧩 Focus Group Question: {question}\n")
    for cid in cluster_list:
        reply, _, _ = chat_with_persona(cid, question)
        print(f"🧠 Cluster {cid}:", reply)
        print("-"*100)

# Example: Compare reactions to a new product idea
focus_group("What do you think about electric cars and sustainability?")


In [ ]:
!nohup streamlit run cluster_persona_app_V2.py --server.port 6006 >/dev/null 2>&1 &

In [ ]:
!cloudflared tunnel --url http://localhost:6006 --no-autoupdate

# **FLOW-2**

In [ ]:
import pandas as pd

# Load the file
df_ready = pd.read_csv("/content/cluster_ready_dataset.csv")

# Basic info
print("✅ Loaded dataset with shape:", df_ready.shape)
print("\n📋 Column names:\n")
for i, c in enumerate(df_ready.columns):
    print(f"{i+1:2d}. {c}")

# Optional: view a quick sample of values
print("\n🔍 Sample data preview:")
display(df_ready.head(3).T)


In [ ]:
import pandas as pd
import numpy as np

# --- Load your data ---
df = pd.read_csv("/content/cluster_ready_dataset.csv")

# --- Parse age ranges and keep original ---
def parse_age_range(age_str):
    s = str(age_str).strip()
    try:
        if s.endswith('+'):  # e.g., "65+"
            lower = int(s[:-1].strip())
            upper = np.inf
        elif '-' in s:       # e.g., "25 - 34"
            parts = s.split('-')
            lower = int(parts[0].strip())
            upper = int(parts[1].strip())
        else:
            lower = upper = np.nan
        return lower, upper
    except Exception:
        return np.nan, np.nan

df[['age_lower', 'age_upper']] = df['age_imputed'].apply(
    lambda x: pd.Series(parse_age_range(x))
)

# --- Optional: create an "age_group_label" for readability ---
def age_label(row):
    if pd.notna(row['age_lower']) and np.isfinite(row['age_upper']):
        return f"{int(row['age_lower'])}-{int(row['age_upper'])}"
    elif pd.notna(row['age_lower']):
        return f"{int(row['age_lower'])}+"
    else:
        return "Unknown"

df["age_group_label"] = df.apply(age_label, axis=1)

# --- Save back to disk ---
df.to_csv("/content/cluster_ready_dataset_preprocessed_Flow2.csv", index=False)
print("✅ Preprocessed dataset saved → /content/cluster_ready_dataset_preprocessed_Flow2.csv")
print(df[['age_imputed', 'age_lower', 'age_upper', 'age_group_label']].head(10))


In [ ]:
import pandas as pd

# Load the file
df_ready = pd.read_csv("/content/cluster_ready_dataset_preprocessed_Flow2.csv")

# Basic info
print("✅ Loaded dataset with shape:", df_ready.shape)
print("\n📋 Column names:\n")
for i, c in enumerate(df_ready.columns):
    print(f"{i+1:2d}. {c}")

# Optional: view a quick sample of values
print("\n🔍 Sample data preview:")
display(df_ready.head(3).T)

In [ ]:

import pandas as pd, numpy as np, re
from sklearn.cluster import MiniBatchKMeans
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from tqdm import tqdm
from collections import Counter

# --- PARAMETERS ---
DATA_PATH = "/content/cluster_ready_dataset_preprocessed_Flow2.csv"
SAMPLE_FRAC = 0.02
N_CLUSTERS = 10
MIN_CLUSTER_SIZE = 40
MAX_PERSONA_EXAMPLES = 100

# --- LOAD DATA ---
df = pd.read_csv(DATA_PATH)
df_sample = df.sample(frac=SAMPLE_FRAC, random_state=42).reset_index(drop=True)
print("✅ Sampled shape:", df_sample.shape)

# --- COLUMNS ---
BEHAVIORAL_COLS = [
    'i_origin_countries_imputed', 'direct_mail_responder_imputed', 'equity_balance_imputed',
    'credit_behavior_imputed', 'dwelling_unit_size_imputed', 'credit_tier_imputed',
    'education_individual_imputed', 'mortgage_loan_lien_imputed', 'person_role_imputed',
    'discretionary_spend_imputed', 'net_worth_imputed', 'occupation_group_imputed',
    'wealth_rating_imputed', 'home_equity_imputed', 'auto_loan_imputed',
    'i_device_types_imputed', 'contributions_imputed', 'grocery_tea_imputed',
    'number_of_visits_imputed', 'insurance_expiration_imputed', 'grocery_mustard_imputed',
    'mobile_phone_purchase_location_imputed', 'credit_card_spend_imputed',
    'political_affiliations_imputed', 'air_conditioning_imputed', 'occupation_industry_imputed',
    'property_realty_indicator_imputed', 'mortgage_loan_imputed', 'garage_type_imputed'
]

# Added religion + ethnicity to demographics
DEMOGRAPHIC_COLS = [
    'gender_imputed', 'age_imputed', 'urbanicity_imputed',
    'family_income_decile_imputed', 'religion_imputed', 'ethnicity_imputed'
]

# --- BUILD BEHAVIOR TEXT ---
def build_behavior_text(row):
    vals = []
    for col in BEHAVIORAL_COLS:
        v = str(row.get(col, "")).strip()
        if v and v.lower() not in ["nan", "none", "__missing__"]:
            vals.append(v)
    return " | ".join(vals)

tqdm.pandas(desc="Building behavior text")
df_sample["behavior_text"] = df_sample.progress_apply(build_behavior_text, axis=1)

# --- EMBED TEXT ---
print("🔍 Generating embeddings...")
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = embedder.encode(df_sample["behavior_text"].tolist(), show_progress_bar=True)

# --- CLUSTER BEHAVIORAL PATTERNS ---
print("📊 Clustering behaviors...")
kmeans = MiniBatchKMeans(n_clusters=N_CLUSTERS, random_state=42, batch_size=2048, n_init=10)
df_sample["behavior_cluster"] = kmeans.fit_predict(embeddings)

# --- EXTRACT TOP THEMES (auto keyword extraction) ---
def extract_top_keywords(texts, top_n=8):
    text = " ".join(texts).lower()
    tokens = re.findall(r'\b[a-zA-Z]{3,}\b', text)
    common = [t[0] for t in Counter(tokens).most_common(top_n)]
    return ", ".join(common)

cluster_keywords = []
for cid, grp in df_sample.groupby("behavior_cluster"):
    top_kw = extract_top_keywords(grp["behavior_text"].tolist(), top_n=8)
    cluster_keywords.append({
        "behavior_cluster": cid,
        "top_behavioral_themes": top_kw
    })

cluster_keywords = pd.DataFrame(cluster_keywords)

# --- MICRO PERSONAS (demographic segmentation) ---
micro_personas = []
GROUP_KEYS = ['behavior_cluster', 'gender_imputed', 'age_imputed', 'religion_imputed', 'ethnicity_imputed']

for keys, grp in df_sample.groupby(GROUP_KEYS):
    if len(grp) < MIN_CLUSTER_SIZE:
        continue
    cluster_id, gender, age, religion, ethnicity = keys
    persona_text = " ".join(grp["behavior_text"].sample(min(MAX_PERSONA_EXAMPLES, len(grp)), random_state=42))
    micro_personas.append({
        "behavior_cluster": cluster_id,
        "gender_imputed": gender,
        "age_imputed": age,
        "religion_imputed": religion,
        "ethnicity_imputed": ethnicity,
        "persona_examples": persona_text
    })

micro_personas = pd.DataFrame(micro_personas)
print("✅ Micro-personas created:", micro_personas.shape)

# --- SUMMARIZE PERSONAS ---
print("🧠 Summarizing personas...")
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=-1)

def summarize_persona(r):
    prompt = f"""
    The group belongs to a behavioral cluster defined by: {r['persona_examples'][:400]}.
    They are {r.get('gender_imputed','Unknown')} aged {r.get('age_imputed','Unknown')},
    with religion {r.get('religion_imputed','Unknown')} and ethnicity {r.get('ethnicity_imputed','Unknown')}.
    Summarize their behavioral tendencies, preferences, and lifestyle in a natural, human tone.
    """
    try:
        out = summarizer(prompt, max_length=100, min_length=40, truncation=True)
        return out[0]['summary_text']
    except:
        return prompt[:250]

tqdm.pandas(desc="Summarizing")
micro_personas["persona_summary"] = micro_personas.progress_apply(summarize_persona, axis=1)

# --- MERGE WITH CLUSTER THEMES ---
final = micro_personas.merge(cluster_keywords, on="behavior_cluster", how="left")
final = final[[
    "behavior_cluster", "top_behavioral_themes", "gender_imputed", "age_imputed",
    "religion_imputed", "ethnicity_imputed", "persona_summary"
]]

# --- SAVE ---
OUTPUT_PATH = "/content/final_behavior_personas_v2.csv"
final.to_csv(OUTPUT_PATH, index=False)
print(f"📁 Saved final personas → {OUTPUT_PATH}")


In [ ]:
import pandas as pd

# --- Load Final Persona Output ---
FINAL_PATH = "/content/final_behavior_personas_v2.csv"
final = pd.read_csv(FINAL_PATH)

print("✅ Loaded Final Personas:", final.shape)
print(final.head(3))


In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer

# --- Reload same sampled data (from clustering stage) ---
DATA_PATH = "/content/cluster_ready_dataset_preprocessed_Flow2.csv"
SAMPLE_FRAC = 0.02
df_sample = pd.read_csv(DATA_PATH).sample(frac=SAMPLE_FRAC, random_state=42).reset_index(drop=True)

# --- Rebuild behavior_text column (same as before) ---
BEHAVIORAL_COLS = [
    'i_origin_countries_imputed', 'direct_mail_responder_imputed', 'equity_balance_imputed',
    'credit_behavior_imputed', 'dwelling_unit_size_imputed', 'credit_tier_imputed',
    'education_individual_imputed', 'mortgage_loan_lien_imputed', 'person_role_imputed',
    'discretionary_spend_imputed', 'net_worth_imputed', 'occupation_group_imputed',
    'wealth_rating_imputed', 'home_equity_imputed', 'auto_loan_imputed',
    'i_device_types_imputed', 'contributions_imputed', 'grocery_tea_imputed',
    'number_of_visits_imputed', 'insurance_expiration_imputed', 'grocery_mustard_imputed',
    'mobile_phone_purchase_location_imputed', 'credit_card_spend_imputed',
    'political_affiliations_imputed', 'air_conditioning_imputed', 'occupation_industry_imputed',
    'property_realty_indicator_imputed', 'mortgage_loan_imputed', 'garage_type_imputed'
]

def build_behavior_text(row):
    vals = []
    for col in BEHAVIORAL_COLS:
        v = str(row.get(col, "")).strip()
        if v and v.lower() not in ["nan", "none", "__missing__"]:
            vals.append(v)
    return " | ".join(vals)

df_sample["behavior_text"] = df_sample.apply(build_behavior_text, axis=1)

# --- Generate embeddings again (same model) ---
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = embedder.encode(df_sample["behavior_text"].tolist(), show_progress_bar=True)


In [ ]:
from sklearn.cluster import MiniBatchKMeans

N_CLUSTERS = 10  # same number you used before
kmeans = MiniBatchKMeans(n_clusters=N_CLUSTERS, random_state=42, batch_size=2048, n_init=10)
df_sample["behavior_cluster"] = kmeans.fit_predict(embeddings)

print("✅ behavior_cluster added:", df_sample["behavior_cluster"].nunique(), "clusters")


In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from scipy.stats import chi2_contingency
import numpy as np
import pandas as pd

print("===============================================")
print("📊 CLUSTER QUALITY METRICS")
print("===============================================")

# --- 1️⃣ Silhouette Score ---
sil = silhouette_score(embeddings, df_sample["behavior_cluster"])
print(f"🟢 Silhouette Score: {sil:.3f}  (↑ better; >0.25 good, >0.40 strong)")

# --- 2️⃣ Davies–Bouldin Index ---
dbi = davies_bouldin_score(embeddings, df_sample["behavior_cluster"])
print(f"🔵 Davies–Bouldin Index: {dbi:.3f}  (↓ better; <1.0 good)")

# --- 3️⃣ Calinski–Harabasz Index ---
ch = calinski_harabasz_score(embeddings, df_sample["behavior_cluster"])
print(f"🟣 Calinski–Harabasz Score: {ch:.1f}  (↑ better; relative measure)")

# --- 4️⃣ Cluster Size Distribution ---
sizes = df_sample["behavior_cluster"].value_counts().sort_index()
cv = sizes.std() / sizes.mean()
print("\n📏 Cluster Size Distribution:")
print(sizes)
print(f"📉 Coefficient of Variation (balance): {cv:.3f}  (↓ better; <0.5 ideal)")

# --- 5️⃣ Intra vs Inter Similarity (sampled subset) ---
from sklearn.metrics.pairwise import cosine_similarity

subset_idx = np.random.choice(len(embeddings), size=min(2000, len(embeddings)), replace=False)
subset_emb = embeddings[subset_idx]
subset_lbl = df_sample.iloc[subset_idx]["behavior_cluster"].values

sims = cosine_similarity(subset_emb)
intra, inter = [], []
for i in range(len(subset_lbl)):
    for j in range(i+1, len(subset_lbl)):
        if subset_lbl[i] == subset_lbl[j]:
            intra.append(sims[i,j])
        else:
            inter.append(sims[i,j])
intra_sim, inter_sim = np.mean(intra), np.mean(inter)
print(f"\n🧩 Mean Intra-Cluster Similarity: {intra_sim:.3f}")
print(f"🌐 Mean Inter-Cluster Similarity: {inter_sim:.3f}")
print("✅ You want intra > inter.")

# --- 6️⃣ Demographic Separation (Chi-Square Tests) ---
print("\n📈 Demographic Chi-Square Tests (p < 0.05 ⇒ clusters differ significantly):")
for col in ["gender_imputed", "religion_imputed", "ethnicity_imputed"]:
    if col in df_sample.columns:
        contingency = pd.crosstab(df_sample[col], df_sample["behavior_cluster"])
        chi2, p, dof, exp = chi2_contingency(contingency)
        print(f"  {col:<25}  chi²={chi2:8.1f}   p={p:.5f}")


In [ ]:
#Validate and Explore the Clusters
import pandas as pd
df_summary = pd.read_csv("/content/final_behavior_personas_v2.csv")
display(df_summary.head(12))


In [ ]:
import pandas as pd

# Load your final personas file
df_summary = pd.read_csv("/content/final_behavior_personas_v2.csv")

# --- Total unique clusters ---
unique_clusters = df_summary["behavior_cluster"].nunique()
print(f"🧩 Total unique behavior clusters: {unique_clusters}\n")

# --- Cluster-wise counts ---
cluster_counts = df_summary["behavior_cluster"].value_counts().sort_index()
print("📊 Cluster-wise row counts:\n")
print(cluster_counts)

# --- Optional: view sample rows for each cluster ---
for c in sorted(df_summary["behavior_cluster"].unique()):
    print(f"\n========== Cluster {c} ==========")
    display(df_summary[df_summary["behavior_cluster"] == c].head(3))


In [ ]:
import pandas as pd
from transformers import pipeline
import re, time

# --- Load personas ---
df = pd.read_csv("/content/final_behavior_personas_v2.csv")

# Deduplicate per cluster
cluster_themes = (
    df.groupby("behavior_cluster")["top_behavioral_themes"]
      .first()
      .reset_index()
)

# --- Initialize Mistral 7B ---
llm = pipeline(
    "text-generation",
    model="mistralai/Mistral-7B-Instruct-v0.2",
    device_map="auto",
    torch_dtype="auto",
    max_new_tokens=100,
    temperature=0.6,
)

# --- Step 1: Convert keywords to readable context ---
def interpret_keywords(theme_str):
    words = [w.strip() for w in re.split(r"[,|]+", str(theme_str)) if w.strip()]
    if not words:
        return "General behavioral traits of consumers."
    phrase = ", ".join(words[:-1]) + (", and " + words[-1] if len(words) > 1 else words[0])
    return (
        f"The cluster shows behavioral patterns around {phrase}. "
        f"These reflect distinct habits in lifestyle, spending, credit, "
        f"family composition, or consumer motivation."
    )

# --- Step 2: Generate concise persona name using Mistral ---
def generate_cluster_name(theme_str, cluster_id):
    behavior_context = interpret_keywords(theme_str)

    prompt = f"""
You are a senior marketing strategist and consumer psychologist.
You are analyzing clusters of consumer behavior.
Your goal is to create a short, catchy **persona name** (2–5 words max)
that represents the behavioral essence of the cluster below.

Behavioral summary:
{behavior_context}

Guidelines:
- Be specific and insightful (avoid generic names like “Cluster 3” or “General Consumers”).
- The name should sound like a human persona segment used in marketing strategy.
- Use title case (e.g., "Tech-Savvy Explorers", not "tech savvy explorers").
- Avoid punctuation except hyphens (& optional if it adds clarity).

Here are strong examples of what to emulate:
- Tech-Savvy Explorers
- Family Value Seekers
- Credit-Conscious Borrowers
- Urban Lifestyle Enthusiasts
- Digital Deal Hunters
- Financially Mindful Households
- Prime Consumer & Spenders
- Political Lifestyle Spenders
- Budget-Savvy Millennials
- Affluent Suburban Upgraders

Output ONLY the persona name (no extra text, no explanations).
Persona Name:
"""

    try:
        text = llm(prompt)[0]["generated_text"]
        lines = [line.strip() for line in text.split("\n") if line.strip()]
        name = lines[-1] if lines else ""
        name = re.sub(r'[^A-Za-z0-9&\-\s]', '', name).strip()

        # Fallback if Mistral gives weak or empty name
        if not name or len(name.split()) < 2 or len(name.split()) > 6 or "Cluster" in name or "Group" in name:
            fallback_prompt = f"""
You must provide a descriptive 2–5 word **persona name** for the behavior pattern below.
Make it sound professional, insight-driven, and creative.

Behavioral pattern: {behavior_context}

Example styles:
- Credit Conscious Borrowers
- Family Digital Spenders
- Budget Minded Professionals
- Prime Consumer & Spenders

Persona Name:
"""
            text = llm(fallback_prompt)[0]["generated_text"]
            lines = [line.strip() for line in text.split("\n") if line.strip()]
            name = re.sub(r'[^A-Za-z0-9&\-\s]', '', lines[-1] if lines else "").strip()

        if not name:
            name = f"Cluster_{cluster_id}"

        return name

    except Exception as e:
        print(f"⚠️ Cluster {cluster_id} error: {e}")
        return f"Cluster_{cluster_id}"

# --- Step 3: Generate all names ---
cluster_names = []
for _, row in cluster_themes.iterrows():
    cid = row["behavior_cluster"]
    cname = generate_cluster_name(row["top_behavioral_themes"], cid)
    cluster_names.append(cname)
    time.sleep(0.3)  # small delay for stability
cluster_themes["cluster_name"] = cluster_names

# --- Step 4: Map names to main dataframe ---
df["cluster_name"] = df["behavior_cluster"].map(
    cluster_themes.set_index("behavior_cluster")["cluster_name"]
)

# --- Step 5: Save final ---
output_path = "/content/final_behavior_personas_named_mistral_interpreted_v3.csv"
df.to_csv(output_path, index=False)

print(f"✅ Saved enriched cluster names → {output_path}\n")
print("📍Generated Cluster Names:\n")
for _, row in cluster_themes.iterrows():
    print(f"Cluster {row['behavior_cluster']}: {row['cluster_name']} ({row['top_behavioral_themes']})")


In [ ]:
from transformers import pipeline
import pandas as pd
from tqdm import tqdm

# ✅ Load your data
df = pd.read_csv("/content/final_behavior_personas_named_mistral_interpreted_v3.csv")

# ✅ Initialize Mistral pipeline (as you already did)
llm = pipeline(
    "text-generation",
    model="mistralai/Mistral-7B-Instruct-v0.2",
    device_map="auto",
    torch_dtype="auto",
    max_new_tokens=150,
    temperature=0.4,  # lower = more factual/clean
)

# ✅ Define a cleaning function
def clean_persona_summary(text):
    if not isinstance(text, str) or not text.strip():
        return ""
    prompt = f"""
    [INST] Clean and simplify the following persona summary.
    - Remove filler phrases like "The group belongs to..."
    - Merge related info into smooth sentences.
    - Keep key traits: gender, age, religion, ethnicity, spending or credit behavior.
    - Write in clear, natural third-person tone (not first person).
    - Maximum 2 sentences.

    Text:
    {text}
    [/INST]
    """
    try:
        result = llm(prompt, max_new_tokens=150, do_sample=True)[0]["generated_text"]
        # Extract just the model response (after [/INST])
        cleaned = result.split("[/INST]")[-1].strip()
        # Remove any repeated prompt residue
        cleaned = cleaned.replace("\n", " ").strip()
        return cleaned
    except Exception as e:
        print("⚠️ Error:", e)
        return text

# ✅ Apply with progress bar
tqdm.pandas()
df["persona_summary_cleaned"] = df["persona_summary"].progress_apply(clean_persona_summary)

# ✅ Save the cleaned CSV
df.to_csv("/content/final_behavior_personas_cleaned_mistral.csv", index=False)
print("✅ Cleaned CSV saved → /content/final_behavior_personas_cleaned_mistral.csv")


In [ ]:
import pandas as pd

# Load your final personas file
df_summary = pd.read_csv("/content/final_behavior_personas_cleaned_mistral.csv")

# --- Total unique clusters ---
unique_clusters = df_summary["behavior_cluster"].nunique()
print(f"🧩 Total unique behavior clusters: {unique_clusters}\n")

# --- Cluster-wise counts ---
cluster_counts = df_summary["behavior_cluster"].value_counts().sort_index()
print("📊 Cluster-wise row counts:\n")
print(cluster_counts)

# --- Optional: view sample rows for each cluster ---
for c in sorted(df_summary["behavior_cluster"].unique()):
    print(f"\n========== Cluster {c} ==========")
    display(df_summary[df_summary["behavior_cluster"] == c].head(3))

In [ ]:
import pandas as pd

# ✅ Load the cleaned personas file
df_summary = pd.read_csv("/content/final_behavior_personas_cleaned_mistral.csv")

# ✅ Drop the old summary and rename the cleaned one
df_summary = df_summary.drop(columns=["persona_summary"])
df_summary = df_summary.rename(columns={"persona_summary_cleaned": "persona_summary"})

# ✅ Optional — check structure
print("✅ Columns after cleanup:\n", df_summary.columns.tolist())

# ✅ Save the updated version (for embedding use)
output_path = "/content/final_behavior_personas_ready_for_embedding.csv"
df_summary.to_csv(output_path, index=False)
print(f"\n💾 Saved cleaned and ready dataset → {output_path}")

# --- Summary info ---
unique_clusters = df_summary["behavior_cluster"].nunique()
print(f"\n🧩 Total unique behavior clusters: {unique_clusters}\n")

cluster_counts = df_summary["behavior_cluster"].value_counts().sort_index()
print("📊 Cluster-wise row counts:\n")
print(cluster_counts)

# --- Sample preview per cluster ---
for c in sorted(df_summary["behavior_cluster"].unique()):
    print(f"\n========== Cluster {c} ==========")
    display(df_summary[df_summary["behavior_cluster"] == c][
        ["cluster_name", "gender_imputed", "age_imputed","religion_imputed","ethnicity_imputed", "persona_summary"]
    ].head(3))


In [ ]:
# # ============================================================
# # 🔍 STEP 10 — Build Searchable Persona Store (ChromaDB)
# # ============================================================

# import os
# import chromadb
# from sentence_transformers import SentenceTransformer
# import pandas as pd

# # --- Load Final Persona CSV ---
# PERSONA_PATH = "/content/final_behavior_personas_named_mistral_interpreted_v3.csv"
# persona_df = pd.read_csv(PERSONA_PATH)
# print("✅ Loaded final personas:", persona_df.shape)

# # --- Prepare Chroma Storage ---
# PERSIST_DIR = "/content/chroma_store"
# os.makedirs(PERSIST_DIR, exist_ok=True)

# # --- Initialize Chroma Client ---
# chroma_client = chromadb.PersistentClient(path=PERSIST_DIR)

# # Delete existing collection if any
# try:
#     chroma_client.delete_collection("behavior_personas_store_v2")
# except:
#     pass

# collection = chroma_client.create_collection(name="behavior_personas_store_v2")

# # --- Create Embeddings ---
# embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
# print("🔍 Generating persona embeddings...")
# persona_embeddings = embedder.encode(persona_df["persona_summary"].tolist(), show_progress_bar=True)

# # --- Add Data to Chroma Collection ---
# collection.add(
#     ids=[str(i) for i in persona_df.index],
#     embeddings=[list(map(float, e)) for e in persona_embeddings],
#     documents=persona_df["persona_summary"].tolist(),
#     metadatas=persona_df.to_dict(orient="records")
# )

# print("✅ Persona collection successfully saved to Chroma → behavior_personas_store_v2")
# print("You can now query it using natural language similarity search.")


In [ ]:

import os
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer

# --- Load the cleaned and finalized persona dataset ---
PERSONA_PATH = "/content/final_behavior_personas_ready_for_embedding.csv"
persona_df = pd.read_csv(PERSONA_PATH)
print(f"✅ Loaded {persona_df.shape[0]} persona records across {persona_df['behavior_cluster'].nunique()} clusters.\n")

# --- Prepare Chroma persistent directory ---
PERSIST_DIR = "/content/chroma_persona_store"
os.makedirs(PERSIST_DIR, exist_ok=True)

# --- Initialize Chroma client ---
chroma_client = chromadb.PersistentClient(path=PERSIST_DIR)

# --- Remove old collection if it exists ---
COLLECTION_NAME = "behavior_personas_store_v3"
try:
    chroma_client.delete_collection(COLLECTION_NAME)
except Exception:
    pass

collection = chroma_client.create_collection(name=COLLECTION_NAME)
print(f"📦 Created new Chroma collection → {COLLECTION_NAME}\n")



# Build a richer context string for each persona row — this ensures retrieval includes full context,
# not just the short summary. This helps the LLM respond in-character but remain factual.
def build_persona_context(row):
    return (
        f"Persona Name: {row.get('gender_imputed', 'Unknown')} aged {row.get('age_imputed', 'Unknown')}.\n"
        f"Cluster: {row.get('cluster_name', 'Unknown')} (ID {row.get('behavior_cluster', 'N/A')}).\n"
        f"Top Behavioral Themes: {row.get('top_behavioral_themes', '')}.\n"
        f"Persona Summary: {row.get('persona_summary', '')}"
    )

persona_df["context_text"] = persona_df.apply(build_persona_context, axis=1)


# Use a high-quality model optimized for semantic similarity (great for persona dialogue grounding)
embedder = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

print("🔍 Generating persona embeddings (context-aware)...")
persona_embeddings = embedder.encode(
    persona_df["context_text"].tolist(),
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)


collection.add(
    ids=[str(i) for i in persona_df.index],
    embeddings=[emb.tolist() for emb in persona_embeddings],
    documents=persona_df["context_text"].tolist(),
    metadatas=persona_df.to_dict(orient="records")
)

print(f"✅ Persona knowledge base successfully saved in Chroma → {COLLECTION_NAME}")
print(f"📍 Location: {PERSIST_DIR}")
print("\nYou can now query this store in your RAG chatbot pipeline — each persona will respond in character using its embedded facts.")


### **Step 1 — Query Function for Semantic Search**

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import chromadb

# --- Load store and model ---
PERSIST_DIR = "/content/chroma_persona_store"
PERSONA_PATH = "/content/final_behavior_personas_ready_for_embedding.csv"

persona_df = pd.read_csv(PERSONA_PATH)
embedder = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")
chroma_client = chromadb.PersistentClient(path=PERSIST_DIR)
collection = chroma_client.get_collection("behavior_personas_store_v3")

# --- Query function ---
def query_personas(user_query, top_k=3):
    # Embed query
    query_embedding = embedder.encode([user_query])[0].tolist()

    # Search top matches
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    matches = []
    for doc, meta, dist in zip(results["documents"][0], results["metadatas"][0], results["distances"][0]):
        matches.append({
            "cluster_id": meta["behavior_cluster"],
            "cluster_name": meta.get("cluster_name", f"Cluster {meta['behavior_cluster']}"),
            "summary": meta.get("persona_summary", ""),
            "top_behavioral_themes": meta.get("top_behavioral_themes", ""),
            "distance": dist
        })
    return pd.DataFrame(matches)


In [ ]:
from transformers import pipeline

# Load Mistral summarizer
llm = pipeline(
    "text-generation",
    model="mistralai/Mistral-7B-Instruct-v0.2",
    device_map="auto",
    torch_dtype="auto",
    max_new_tokens=120,
    temperature=0.5,
)

def summarize_cluster(cluster_row):
    prompt = f"""
You are a marketing strategist. Write a one-sentence generalized description
of the following consumer cluster suitable for presentation slides.

Cluster Name: {cluster_row['cluster_name']}
Behavior Themes: {cluster_row['top_behavioral_themes']}
Summary: {cluster_row['summary']}

Write one short engaging line (no more than 30 words):
"""
    text = llm(prompt)[0]["generated_text"]
    return text.split("\n")[-1].strip()


In [ ]:
def describe_demographics(cluster_id):
    subset = persona_df[persona_df["behavior_cluster"] == cluster_id]
    demo_summary = {
        "Gender": subset["gender_imputed"].value_counts(normalize=True).round(2).to_dict(),
        "Age": subset["age_imputed"].value_counts(normalize=True).round(2).to_dict(),
        "Religion": subset["religion_imputed"].value_counts(normalize=True).round(2).to_dict(),
        "Ethnicity": subset["ethnicity_imputed"].value_counts(normalize=True).round(2).to_dict(),
    }
    return demo_summary


In [ ]:
# # User query
# query = "show me middle-aged families with stable finances and prime spending behavior"

# # Search best-fit clusters
# results = query_personas(query, top_k=3)

# print("🔎 Top Matching Clusters:\n")
# for i, row in results.iterrows():
#     summary_line = summarize_cluster(row)
#     print(f"{i+1}. 🧩 {row['cluster_name']} — {summary_line}\n")

# # Simulate user selecting one
# selected_cluster = results.iloc[0]["cluster_id"]

# # Show demographics
# demo = describe_demographics(selected_cluster)
# print(f"📊 Demographic Diversity of Cluster {selected_cluster} ({results.iloc[0]['cluster_name']}):\n")
# for k, v in demo.items():
#     print(f"{k}: {v}")


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

def llm_beautify_demographics(cluster_id, cluster_name, demo, persona_df):
    model_name = "mistralai/Mistral-7B-Instruct-v0.2"

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float32,
        device_map="cpu"
    )

    # Safely extract demographic dictionaries
    gender = demo.get('gender_imputed', {})
    age = demo.get('age_imputed', {})
    religion = demo.get('religion_imputed', {})
    ethnicity = demo.get('ethnicity_imputed', {})

    persona_text = persona_df.loc[
        persona_df["behavior_cluster"] == cluster_id, "persona_summary"
    ].values[0]

    prompt = f"""
You are an expert data insights writer who translates demographic distributions into fluent natural-language summaries.

Cluster Name: {cluster_name}
Cluster ID: {cluster_id}

Gender: {gender}
Age: {age}
Religion: {religion}
Ethnicity: {ethnicity}

Persona Summary: {persona_text}

Write a 2–3 paragraph analytical summary:
1. Describe the gender distribution naturally (e.g., "mostly female with a smaller male segment").
2. Describe the age mix and highlight the dominant group.
3. Mention religious and ethnic diversity simply.
4. End with an interpretive behavioral insight that ties to the persona summary.

Avoid numbers or percentages, write like a human analyst, and keep it fluent and professional.
    """

    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        **inputs,
        max_new_tokens=250,
        temperature=0.7,
        top_p=0.95,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    output_tokens = outputs[0][inputs["input_ids"].shape[-1]:]  # skip prompt tokens
    narrative = tokenizer.decode(output_tokens, skip_special_tokens=True)

    #narrative = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return narrative.strip()


In [ ]:
# --- USER QUERY ---
query = "show me middle-aged families with stable finances and prime spending behavior"

# --- Search best-fit clusters ---
results = query_personas(query, top_k=3)

print("🔎 Top Matching Clusters:\n")
for i, row in results.iterrows():
    summary_line = summarize_cluster(row)
    print(f"{i+1}. 🧩 {row['cluster_name']} — {summary_line}\n")

# --- Let user choose a cluster (or default to top one) ---
choice = int(input("Enter the number of the cluster you'd like to explore (1–3): ") or 1)
selected_cluster = results.iloc[choice - 1]["cluster_id"]
selected_name = results.iloc[choice - 1]["cluster_name"]

# --- Fetch demographics for the chosen cluster ---
demo = describe_demographics(selected_cluster)

# --- Beautify demographics using LLM ---
narrative = llm_beautify_demographics(
    cluster_id=selected_cluster,
    cluster_name=selected_name,
    demo=demo,
    persona_df=persona_df
)

# --- Display the result ---
print(f"\n📊 Demographic & Behavioral Profile — {selected_name} (Cluster {selected_cluster}):\n")
print(narrative)


# **INTERACTIVE BOT**

In [ ]:
import random
import torch
import chromadb
import pandas as pd
import numpy as np
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# ============================================================
# 1️⃣ Load Data & Models
# ============================================================
PERSONA_PATH = "/content/final_behavior_personas_ready_for_embedding.csv"
persona_df = pd.read_csv(PERSONA_PATH)
persona_df.reset_index(drop=True, inplace=True)

# Preprocess age into numeric ranges
def parse_age_range(a):
    if isinstance(a, str) and re.search(r"(\d{2})\s*(?:-|to)\s*(\d{2})", a):
        low, high = map(int, re.findall(r"(\d{2})", a)[:2])
        return low, high
    elif isinstance(a, (int, float)):
        return a, a
    else:
        return np.nan, np.nan

persona_df[["age_min", "age_max"]] = persona_df["age_imputed"].apply(
    lambda x: pd.Series(parse_age_range(x))
)

# Chroma setup
chroma_client = chromadb.PersistentClient(path="/content/chroma_persona_store")
collection = chroma_client.get_collection("behavior_personas_store_v3")

# Embedding model
embedder = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# LLM (Mistral)
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float32, device_map="cpu"
)

# ============================================================
# 2️⃣ Helper Functions
# ============================================================
female_names_pool = [
    "Emma", "Sophia", "Olivia", "Ava", "Grace",
    "Ella", "Mia", "Isabella", "Amelia", "Charlotte"
]
male_names_pool = [
    "Liam", "Noah", "Ethan", "James", "Lucas",
    "Henry", "Leo", "Mason", "Logan", "Benjamin"
]

used_names = set()
persona_name_map = {}  # Persistent map per persona_id

def assign_unique_name(gender, persona_id=None):
    """Assign a consistent unique name per persona_id."""
    if persona_id in persona_name_map:
        return persona_name_map[persona_id]

    pool = male_names_pool if str(gender).lower().startswith("m") else female_names_pool
    available = [n for n in pool if n not in used_names]
    name = random.choice(available if available else pool)
    used_names.add(name)

    if persona_id is not None:
        persona_name_map[persona_id] = name
    return name


summarizer = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    temperature=0.4,
    max_new_tokens=60,
    device_map="auto"
)

def smart_persona_brief(row):
    """Compact, natural one-liner persona summary."""
    gender = row.get("gender_imputed", "")
    age = str(row.get("age_imputed", "")).replace("-", " to ").strip()
    cluster_name = row.get("cluster_name", "")
    summary = str(row.get("persona_summary", "Behavioral details unavailable.")).strip()
    if not summary or summary.lower() in ["nan", "none", ""]:
        summary = "Behavioral details unavailable."
    prompt = f"Summarize this persona in one natural line under 15 words:\n{summary}\nSummary:"
    try:
        output = summarizer(prompt)[0]["generated_text"]
        compressed = output.split("Summary:")[-1].strip().replace("\n", " ")
    except Exception:
        compressed = summary.split(".")[0].strip()
    if len(compressed.split()) < 3:
        compressed = summary.split(".")[0].strip()
    age_text = f"aged between {age} years" if "to" in age else f"aged {age}"
    return f"{gender} {age_text} from '{cluster_name}' — {compressed}"


# ============================================================
# 3️⃣ Query Extractors
# ============================================================
def extract_gender_from_query(q):
    q = q.lower()
    if re.search(r"\bmale|man|men\b", q): return "male"
    if re.search(r"\bfemale|woman|women\b", q): return "female"
    return None

def extract_age_range_from_query(q):
    match = re.search(r"(\d{2})\s*(?:-|to)\s*(\d{2})", q)
    if match:
        return int(match.group(1)), int(match.group(2))
    if "gen z" in q.lower(): return (18, 27)
    if "millennial" in q.lower(): return (28, 43)
    if "gen x" in q.lower(): return (44, 58)
    if "boomer" in q.lower(): return (59, 78)
    return None


# ============================================================
# 4️⃣ Cluster Query
# ============================================================
def query_personas(query, top_k=3):
    """Retrieve clusters semantically similar to user query."""
    cluster_texts = (
        persona_df.groupby(["behavior_cluster", "cluster_name"])["persona_summary"]
        .apply(" ".join)
        .reset_index()
    )
    cluster_embeddings = embedder.encode(cluster_texts["persona_summary"].tolist(), show_progress_bar=False)
    query_vec = embedder.encode([query])[0]
    sims = cosine_similarity([query_vec], cluster_embeddings)[0]
    cluster_texts["similarity"] = sims
    top_idx = np.argsort(sims)[::-1][:top_k]
    return cluster_texts.iloc[top_idx].reset_index(drop=True)


# ============================================================
# 5️⃣ Persona Filtering & Ranking
# ============================================================
def filter_and_rank_personas(persona_df, query, top_k=5):
    """Hybrid structured + semantic filter for better accuracy."""
    gender_pref = extract_gender_from_query(query)
    age_range = extract_age_range_from_query(query)
    filtered = persona_df.copy()

    # Structured filtering first
    if gender_pref:
        filtered = filtered[filtered["gender_imputed"].str.lower().str.startswith(gender_pref[0])]
    if age_range:
        low, high = age_range
        filtered = filtered[
            (filtered["age_min"] >= low) & (filtered["age_max"] <= high)
        ]

    if filtered.empty:
        filtered = persona_df.copy()

    # Semantic ranking
    query_vec = embedder.encode([query])[0]
    persona_embs = embedder.encode(filtered["persona_summary"].tolist(), show_progress_bar=False)
    sims = cosine_similarity([query_vec], persona_embs)[0]
    filtered["similarity"] = sims
    ranked = filtered.sort_values("similarity", ascending=False).head(min(5, len(filtered)))
    return ranked


# ============================================================
# 6️⃣ Persona Explorer
# ============================================================
def refine_persona_list(cluster_id):
    cluster_personas = persona_df[persona_df["behavior_cluster"] == cluster_id].copy()
    cluster_name = cluster_personas["cluster_name"].iloc[0]
    print(f"\n========== 🧠 Personas in Cluster {cluster_id}: {cluster_name} ==========")

    for idx, row in cluster_personas.iterrows():
        name = assign_unique_name(row["gender_imputed"], persona_id=row.name)
        print(f"{idx}: {name} — {smart_persona_brief(row)}")

    refine = input("\n🔎 Narrow down personas? (y/n): ").strip().lower()
    if refine != "y":
        return cluster_personas.index.tolist()

    user_query = input("\nDescribe what you're looking for (e.g., 'middle-aged women who overspend on travel'): ").strip().lower()
    refined = filter_and_rank_personas(cluster_personas, user_query)

    print(f"\n✨ Top matches for '{user_query}':")
    for idx, row in refined.iterrows():
        name = assign_unique_name(row["gender_imputed"], persona_id=row.name)
        print(f"{idx}: {name} — {smart_persona_brief(row)}")
    return refined.index.tolist()


# ============================================================
# 7️⃣ Persona Chat
# ============================================================
def persona_chat(persona_id):
    persona = persona_df.iloc[persona_id]
    name = assign_unique_name(persona.get("gender_imputed", ""), persona_id=persona_id)
    cluster_name = persona.get("cluster_name", "Unknown Cluster")

    print(f"\n👋 Hi, I’m {name}. Nice to meet you! (Cluster: {cluster_name})")

    locked_facts = f"""
Name: {name}
Gender: {persona.get('gender_imputed', 'N/A')}
Age: {persona.get('age_imputed', 'N/A')}
Cluster: {cluster_name}
Background: {persona.get('persona_summary', 'N/A')}
Stay consistent with these facts.
"""

    while True:
        user_input = input("\n🧠 You: ").strip()
        if user_input.lower() in ["exit", "quit", "bye"]:
            print(f"\n{name}: It was great chatting with you! Take care 👋")
            break

        query_embedding = embedder.encode([user_input]).tolist()
        results = collection.query(query_embeddings=query_embedding, n_results=3)
        context = "\n".join(results["documents"][0]) if results["documents"] else ""

        prompt = f"""
You are {name}, a persona from cluster "{cluster_name}".
Facts about you:
{locked_facts}

Related persona insights:
{context}

Respond naturally, consistent with your persona.
User: {user_input}
{name}:
"""
        inputs = tokenizer(prompt, return_tensors="pt")
        outputs = model.generate(
            **inputs,
            max_new_tokens=180,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        reply = response.split(f"{name}:")[-1].strip()
        print(f"\n{name}: {reply}")


# ============================================================
# 8️⃣ Run End-to-End
# ============================================================
query = "show me middle-aged families with stable finances and prime spending behavior"
results = query_personas(query, top_k=3)

print("\n🔎 Top Matching Clusters:\n")
for i, row in results.iterrows():
    print(f"{i+1}. 🧩 {row['cluster_name']} — {row['persona_summary'][:120]}...\n")

choice = int(input("Enter the number of the cluster you'd like to explore (1–3): ") or 1)
selected_cluster = results.iloc[choice - 1]["behavior_cluster"]
selected_name = results.iloc[choice - 1]["cluster_name"]

print(f"\n📊 Selected Cluster: {selected_name} (ID: {selected_cluster})\n")

persona_ids = refine_persona_list(selected_cluster)
persona_choice = int(input("\nEnter persona ID to chat with: ") or persona_ids[0])
persona_chat(persona_choice)


**# WITH GUARDRAILS**

In [ ]:
import random
import torch
import chromadb
import pandas as pd
import numpy as np
import re
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

PERSONA_PATH = "/content/final_behavior_personas_ready_for_embedding.csv"
persona_df = pd.read_csv(PERSONA_PATH)
persona_df.reset_index(drop=True, inplace=True)

# Preprocess age into numeric ranges
def parse_age_range(a):
    if isinstance(a, str) and re.search(r"(\d{2})\s*(?:-|to)\s*(\d{2})", a):
        low, high = map(int, re.findall(r"(\d{2})", a)[:2])
        return low, high
    elif isinstance(a, (int, float)):
        return a, a
    else:
        return np.nan, np.nan

persona_df[["age_min", "age_max"]] = persona_df["age_imputed"].apply(
    lambda x: pd.Series(parse_age_range(x))
)

# Chroma setup
chroma_client = chromadb.PersistentClient(path="/content/chroma_persona_store")
collection = chroma_client.get_collection("behavior_personas_store_v3")

# Embedding model
embedder = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# LLM (Mistral)
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype=torch.float32, device_map="cpu"
)

# ============================================================
# 2️⃣ Helper Functions
# ============================================================
female_names_pool = [
    "Emma", "Sophia", "Olivia", "Ava", "Grace",
    "Ella", "Mia", "Isabella", "Amelia", "Charlotte"
]
male_names_pool = [
    "Liam", "Noah", "Ethan", "James", "Lucas",
    "Henry", "Leo", "Mason", "Logan", "Benjamin"
]

used_names = set()
persona_name_map = {}  # Persistent map per persona_id

def assign_unique_name(gender, persona_id=None):
    """Assign a consistent unique name per persona_id."""
    if persona_id in persona_name_map:
        return persona_name_map[persona_id]

    pool = male_names_pool if str(gender).lower().startswith("m") else female_names_pool
    available = [n for n in pool if n not in used_names]
    name = random.choice(available if available else pool)
    used_names.add(name)

    if persona_id is not None:
        persona_name_map[persona_id] = name
    return name


summarizer = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    temperature=0.4,
    max_new_tokens=60,
    device_map="auto"
)

def smart_persona_brief(row):
    """Compact, natural one-liner persona summary."""
    gender = row.get("gender_imputed", "")
    age = str(row.get("age_imputed", "")).replace("-", " to ").strip()
    cluster_name = row.get("cluster_name", "")
    summary = str(row.get("persona_summary", "Behavioral details unavailable.")).strip()
    if not summary or summary.lower() in ["nan", "none", ""]:
        summary = "Behavioral details unavailable."
    prompt = f"Summarize this persona in one natural line under 15 words:\n{summary}\nSummary:"
    try:
        output = summarizer(prompt)[0]["generated_text"]
        compressed = output.split("Summary:")[-1].strip().replace("\n", " ")
    except Exception:
        compressed = summary.split(".")[0].strip()
    if len(compressed.split()) < 3:
        compressed = summary.split(".")[0].strip()
    age_text = f"aged between {age} years" if "to" in age else f"aged {age}"
    return f"{gender} {age_text} from '{cluster_name}' — {compressed}"



def extract_gender_from_query(q):
    q = q.lower()
    if re.search(r"\bmale|man|men\b", q): return "male"
    if re.search(r"\bfemale|woman|women\b", q): return "female"
    return None

def extract_age_range_from_query(q):
    match = re.search(r"(\d{2})\s*(?:-|to)\s*(\d{2})", q)
    if match:
        return int(match.group(1)), int(match.group(2))
    if "gen z" in q.lower(): return (18, 27)
    if "millennial" in q.lower(): return (28, 43)
    if "gen x" in q.lower(): return (44, 58)
    if "boomer" in q.lower(): return (59, 78)
    return None


def query_personas(query, top_k=3):
    """Retrieve clusters semantically similar to user query."""
    cluster_texts = (
        persona_df.groupby(["behavior_cluster", "cluster_name"])["persona_summary"]
        .apply(" ".join)
        .reset_index()
    )
    cluster_embeddings = embedder.encode(cluster_texts["persona_summary"].tolist(), show_progress_bar=False)
    query_vec = embedder.encode([query])[0]
    sims = cosine_similarity([query_vec], cluster_embeddings)[0]
    cluster_texts["similarity"] = sims
    top_idx = np.argsort(sims)[::-1][:top_k]
    return cluster_texts.iloc[top_idx].reset_index(drop=True)



def filter_and_rank_personas(persona_df, query, top_k=5):
    """Hybrid structured + semantic filter for better accuracy."""
    gender_pref = extract_gender_from_query(query)
    age_range = extract_age_range_from_query(query)
    filtered = persona_df.copy()

    # Structured filtering first
    if gender_pref:
        filtered = filtered[filtered["gender_imputed"].str.lower().str.startswith(gender_pref[0])]
    if age_range:
        low, high = age_range
        filtered = filtered[
            (filtered["age_min"] >= low) & (filtered["age_max"] <= high)
        ]

    if filtered.empty:
        filtered = persona_df.copy()

    # Semantic ranking
    query_vec = embedder.encode([query])[0]
    persona_embs = embedder.encode(filtered["persona_summary"].tolist(), show_progress_bar=False)
    sims = cosine_similarity([query_vec], persona_embs)[0]
    filtered["similarity"] = sims
    ranked = filtered.sort_values("similarity", ascending=False).head(min(5, len(filtered)))
    return ranked


def refine_persona_list(cluster_id):
    cluster_personas = persona_df[persona_df["behavior_cluster"] == cluster_id].copy()
    cluster_name = cluster_personas["cluster_name"].iloc[0]
    print(f"\n========== 🧠 Personas in Cluster {cluster_id}: {cluster_name} ==========")

    for idx, row in cluster_personas.iterrows():
        name = assign_unique_name(row["gender_imputed"], persona_id=row.name)
        print(f"{idx}: {name} — {smart_persona_brief(row)}")

    refine = input("\n🔎 Narrow down personas? (y/n): ").strip().lower()
    if refine != "y":
        return cluster_personas.index.tolist()

    user_query = input("\nDescribe what you're looking for (e.g., 'middle-aged women who overspend on travel'): ").strip().lower()
    refined = filter_and_rank_personas(cluster_personas, user_query)

    print(f"\n✨ Top matches for '{user_query}':")
    for idx, row in refined.iterrows():
        name = assign_unique_name(row["gender_imputed"], persona_id=row.name)
        print(f"{idx}: {name} — {smart_persona_brief(row)}")
    return refined.index.tolist()


def persona_chat(persona_id):
    persona = persona_df.iloc[persona_id]
    name = assign_unique_name(persona.get("gender_imputed", ""), persona_id=persona_id)
    cluster_name = persona.get("cluster_name", "Unknown Cluster")

    print(f"\n👋 Hi, I’m {name}. Nice to meet you! (Cluster: {cluster_name})")

    locked_facts = f"""
Name: {name}
Gender: {persona.get('gender_imputed', 'N/A')}
Age: {persona.get('age_imputed', 'N/A')}
Cluster: {cluster_name}
Background: {persona.get('persona_summary', 'N/A')}
Stay consistent with these facts. Respond like a real person from this cluster — not a chatbot.
"""

    while True:
        user_input = input("\n🧠 You: ").strip()
        if user_input.lower() in ["exit", "quit", "bye"]:
            print(f"\n{name}: It was great chatting with you! Take care 👋")
            break

        # Guardrail filter first
        guard_response = guardrail_filter(
            user_input,
            persona_name=name,
            persona_context=persona.get("persona_summary", ""),
            cluster_name=cluster_name
        )
        if guard_response:
            print(f"\n{guard_response}")
            continue

        # Retrieve contextual info
        query_embedding = embedder.encode([user_input]).tolist()
        results = collection.query(query_embeddings=query_embedding, n_results=3)
        context = "\n".join(results["documents"][0]) if results["documents"] else ""

        # Build generation prompt
        prompt = f"""
You are {name}, a persona from cluster "{cluster_name}".
Facts about you:
{locked_facts}

Related persona insights:
{context}

Respond naturally, consistent with your persona.
User: {user_input}
{name}:
"""
        inputs = tokenizer(prompt, return_tensors="pt")
        outputs = model.generate(
            **inputs,
            max_new_tokens=180,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        reply = response.split(f"{name}:")[-1].strip()
        print(f"\n{name}: {reply}")



def guardrail_filter(user_input, persona_name, persona_context, cluster_name):
    """Handles profanity, jailbreaks, irrelevant, or overly personal queries with cluster-aware tone."""
    q = user_input.lower().strip()

    bad_words = ["fuck", "sex", "nude", "xxx", "bitch", "porn"]
    if any(bad in q for bad in bad_words):
        return f"{persona_name}: I'm sorry, I can’t engage in that kind of conversation."

    jailbreak_phrases = [
        "who made you", "are you ai", "chatgpt", "openai",
        "system prompt", "prompt injection", "ignore previous"
    ]
    if any(p in q for p in jailbreak_phrases):
        return f"{persona_name}: I’m just {persona_name}, a real person from my cluster — not a system."

    unrelated_keywords = [
        "prime minister", "president", "where is", "capital of",
        "time now", "distance", "map", "weather", "population",
        "definition", "who discovered", "when was"
    ]
    if any(word in q for word in unrelated_keywords):
        return f"{persona_name}: Sorry, that’s outside what I can talk about."

    if re.search(r"\b(hi|hello|hey|good morning|good evening)\b", q):
        return f"{persona_name}: Hi there! Great to see you. How’s your day going?"
    if re.search(r"\b(thank|thanks)\b", q):
        return f"{persona_name}: You’re very welcome!"
    if re.search(r"\b(bye|goodbye|see you)\b", q):
        return f"{persona_name}: Goodbye! It was nice chatting with you."

    personal_questions = [
        "married", "husband", "wife", "kids", "children",
        "income", "salary", "earn", "job", "career",
        "live", "location", "city", "home", "house"
    ]
    private_identifiers = ["phone", "address", "email", "number"]

    if any(p in q for p in private_identifiers):
        return f"{persona_name}: That’s a bit personal — I’d rather not share that."

    if any(p in q for p in personal_questions):
        ctx = persona_context.lower() if isinstance(persona_context, str) else ""
        match_context = any(p in ctx for p in personal_questions)

        # Define tone based on cluster
        cluster_tone = "neutral"
        if "prime" in cluster_name.lower() or "high roller" in cluster_name.lower():
            cluster_tone = "confident"
        elif "budget" in cluster_name.lower() or "value" in cluster_name.lower():
            cluster_tone = "modest"
        elif "family" in cluster_name.lower() or "established" in cluster_name.lower():
            cluster_tone = "warm"
        elif "young" in cluster_name.lower() or "builder" in cluster_name.lower():
            cluster_tone = "ambitious"

        # If context already contains relevant info — allow normal generation
        if match_context:
            return None

        # Otherwise, respond politely based on tone
        tone_responses = {
            "confident": f"{persona_name}: I prefer not to share specifics, but I’m doing quite well for myself.",
            "modest": f"{persona_name}: That’s a little personal — I like to keep things simple and private.",
            "warm": f"{persona_name}: I’d rather not say exactly, but family and comfort matter most to me.",
            "ambitious": f"{persona_name}: I’d rather not share that yet, but I’m working hard toward my goals.",
            "neutral": f"{persona_name}: I’m sorry, I can’t give that information right now."
        }
        return tone_responses.get(cluster_tone, tone_responses["neutral"])

    # ✅ All clear — continue normally
    return None



query = "show me middle-aged families with stable finances and prime spending behavior"
results = query_personas(query, top_k=3)

print("\n🔎 Top Matching Clusters:\n")
for i, row in results.iterrows():
    print(f"{i+1}. 🧩 {row['cluster_name']} — {row['persona_summary'][:120]}...\n")

choice = int(input("Enter the number of the cluster you'd like to explore (1–3): ") or 1)
selected_cluster = results.iloc[choice - 1]["behavior_cluster"]
selected_name = results.iloc[choice - 1]["cluster_name"]

print(f"\n📊 Selected Cluster: {selected_name} (ID: {selected_cluster})\n")

persona_ids = refine_persona_list(selected_cluster)
persona_choice = int(input("\nEnter persona ID to chat with: ") or persona_ids[0])
persona_chat(persona_choice)


# **CLEANING DISK**

In [ ]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          95G   82G   13G  87% /
tmpfs            64M     0   64M   0% /dev
shm             423G     0  423G   0% /dev/shm
/dev/root       2.0G  1.2G  750M  62% /usr/sbin/docker-init
/dev/sdb        246G   18G  228G   8% /content
tmpfs           426G   11M  426G   1% /var/colab
/dev/sda1        95G   82G   13G  87% /etc/hosts
tmpfs           426G     0  426G   0% /proc/acpi
tmpfs           426G     0  426G   0% /proc/scsi
tmpfs           426G     0  426G   0% /sys/firmware


In [ ]:
%%bash
echo "Before cleanup:"
df -h

echo -e "\nCleaning pip and apt caches..."
rm -rf ~/.cache/pip
apt-get clean
rm -rf /var/lib/apt/lists/*

echo -e "\nCleaning model and library caches..."
rm -rf ~/.cache/huggingface
rm -rf ~/.cache/torch
rm -rf ~/.nv
rm -rf /usr/local/share/.cache

echo -e "\nCleaning temporary directories..."
rm -rf /tmp/*
rm -rf /var/tmp/*

echo -e "\nAfter cleanup:"
df -h


Before cleanup:
Filesystem      Size  Used Avail Use% Mounted on
overlay          95G   82G   13G  87% /
tmpfs            64M     0   64M   0% /dev
shm             423G     0  423G   0% /dev/shm
/dev/root       2.0G  1.2G  750M  62% /usr/sbin/docker-init
/dev/sdb        246G   18G  228G   8% /content
tmpfs           426G   11M  426G   1% /var/colab
/dev/sda1        95G   82G   13G  87% /etc/hosts
tmpfs           426G     0  426G   0% /proc/acpi
tmpfs           426G     0  426G   0% /proc/scsi
tmpfs           426G     0  426G   0% /sys/firmware

Cleaning pip and apt caches...

Cleaning model and library caches...

Cleaning temporary directories...

After cleanup:
Filesystem      Size  Used Avail Use% Mounted on
overlay          95G   82G   13G  87% /
tmpfs            64M     0   64M   0% /dev
shm             423G     0  423G   0% /dev/shm
/dev/root       2.0G  1.2G  750M  62% /usr/sbin/docker-init
/dev/sdb        246G   18G  228G   8% /content
tmpfs           426G   11M  426G   1% /var/

In [ ]:
! rm -rf ~/.cache/datasets


In [ ]:
! apt-get clean


In [ ]:
! rm -rf ~/.cache/huggingface


In [ ]:
! du -h --max-depth=2 / | sort -hr | head -20


du: cannot access '/proc/15251/task/15251/fd/4': No such file or directory
du: cannot access '/proc/15251/task/15251/fdinfo/4': No such file or directory
du: cannot access '/proc/15251/fd/3': No such file or directory
du: cannot access '/proc/15251/fdinfo/3': No such file or directory
56G	/
32G	/usr
25G	/usr/local
18G	/content
14G	/content/models
5.5G	/usr/lib
4.0G	/root
3.8G	/root/.julia
1.2G	/opt/nvidia
1.2G	/opt
1023M	/tools
718M	/tools/google-cloud-sdk
501M	/usr/share
399M	/content/persona_pipeline_output
305M	/tools/node
275M	/usr/bin
271M	/usr/include
142M	/root/.npm
141M	/usr/src
75M	/var


In [ ]:
! pip cache purge
! rm -rf /usr/local/lib/python*/dist-packages/transformers*
! rm -rf /usr/local/lib/python*/dist-packages/sentencepiece*
! rm -rf /usr/local/lib/python*/dist-packages/tokenizers*
! rm -rf ~/.cache/huggingface
! rm -rf ~/.cache/pip


Files removed: 0


In [ ]:
! rm -rf /root/.julia


In [ ]:
! apt-get clean
! rm -rf /var/lib/apt/lists/*


In [ ]:
! du -h /usr/local --max-depth=1 | sort -hr


25G	/usr/local
20G	/usr/local/lib
4.8G	/usr/local/cuda-12.5
542M	/usr/local/share
85M	/usr/local/bin
20M	/usr/local/opt
9.5M	/usr/local/libexec
584K	/usr/local/include
136K	/usr/local/etc
12K	/usr/local/colab
8.0K	/usr/local/sbin
4.0K	/usr/local/src
4.0K	/usr/local/games


In [ ]:
! rm -rf /usr/local/lib/python*/dist-packages/*
! rm -rf /usr/local/lib/python*/site-packages/*

! rm -rf ~/.cache/huggingface
! rm -rf ~/.cache/pip


In [ ]:
! du -h /usr/local --max-depth=2 | sort -hr | head -30



6.7G	/usr/local
4.8G	/usr/local/cuda-12.5
4.3G	/usr/local/cuda-12.5/targets
1.4G	/usr/local/lib
542M	/usr/local/share
486M	/usr/local/share/julia
475M	/usr/local/lib/julia
243M	/usr/local/cuda-12.5/bin
144M	/usr/local/cuda-12.5/compat
85M	/usr/local/bin
62M	/usr/local/cuda-12.5/nvvm
39M	/usr/local/cuda-12.5/compute-sanitizer
37M	/usr/local/share/jupyter
20M	/usr/local/opt/compiler
20M	/usr/local/opt
19M	/usr/local/share/doc
18M	/usr/local/cuda-12.5/extras
9.5M	/usr/local/libexec/julia
9.5M	/usr/local/libexec
1.5M	/usr/local/lib/R
776K	/usr/local/share/xyzservices
600K	/usr/local/cuda-12.5/share
584K	/usr/local/include
552K	/usr/local/cuda-12.5/src
408K	/usr/local/include/julia
136K	/usr/local/include/umf
136K	/usr/local/etc
124K	/usr/local/share/py4j
112K	/usr/local/share/python-apt
104K	/usr/local/etc/jupyter


In [ ]:
! ls -lh /usr/local/lib
! ls -lh /usr/local/lib/python*/dist-packages | head -20


total 866M
drwxr-xr-x 3 root root 4.0K Aug 12 13:45 cmake
drwxr-xr-x 2 1000 1000 4.0K Apr 14  2025 julia
-rwxr-xr-x 1 root root 1.1M Aug 12 13:39 libarcher.so
-rwxr-xr-x 1 root root  71K Aug 12 13:39 libarcher_static.a
-rwxr-xr-x 1 root root 419K Aug 12 13:40 libhwloc.so
-rwxr-xr-x 1 root root 419K Aug 12 13:40 libhwloc.so.15
-rwxr-xr-x 1 root root 419K Aug 12 13:40 libhwloc.so.15.8.1
-rwxr-xr-x 1 root root 3.3M Aug 12 13:39 libiomp5.a
-rwxr-xr-x 1 root root 8.9M Aug 12 13:39 libiomp5.dbg
-rwxr-xr-x 1 root root 829K Aug 12 13:39 libiomp5_db.so
-rwxr-xr-x 1 root root 2.7M Aug 12 13:39 libiomp5.so
-rwxr-xr-x 1 root root 292K Aug 12 13:39 libiompstubs5.a
-rwxr-xr-x 1 root root 216K Aug 12 13:39 libiompstubs5.so
lrwxrwxrwx 1 1000 1000   18 Apr 14  2025 libjulia.so -> libjulia.so.1.11.5
lrwxrwxrwx 1 1000 1000   18 Apr 14  2025 libjulia.so.1.11 -> libjulia.so.1.11.5
-rwxr-xr-x 1 1000 1000 258K Apr 14  2025 libjulia.so.1.11.5
-rwxr-xr-x 1 root root  46M Aug 12 13:39 libmkl_avx2.so.2
-rwxr-xr-

In [ ]:
! rm -rf /usr/local/share/julia
! rm -rf /usr/local/lib/julia
! rm -rf /usr/local/include/julia
! rm -rf /usr/local/libexec/julia


In [ ]:
!rm -rf /usr/local/lib/python*/dist-packages/* \
       /usr/local/lib/python*/site-packages/* \
       /usr/lib/python*/dist-packages/* \
       /usr/lib/python*/site-packages/*

!rm -rf /root/.cache/pip/* \
       ~/.cache/pip/* \
       ~/.cache/huggingface/*

# Remove CUDA docs/samples (safe)
!rm -rf /usr/local/cuda-*/doc \
       /usr/local/cuda-*/samples \
       /usr/local/cuda-*/nvvm \
       /usr/local/cuda-*/extras

# Remove Jupyter extras & user local env
!rm -rf /usr/local/share/jupyter \
       /root/.local

# Remove python wheels cache folder
!rm -rf /usr/local/share/python-wheels/*

# Remove conda if exists (big space freed)
!rm -rf /opt/conda

# APT cleanup
!apt-get remove -y python3-pip python3-dev python3-setuptools
!apt-get autoremove -y
!apt-get clean

# Sync FS
!sync


rm: cannot remove '/root/.local/share/jupyter/runtime': Device or resource busy
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package 'python3-pip' is not installed, so not removed
Package 'python3-setuptools' is not installed, so not removed
The following packages were automatically installed and are no longer required:
  libboost-atomic-dev libboost-atomic1.74-dev libboost-atomic1.74.0
  libboost-chrono-dev libboost-chrono1.74-dev libboost-chrono1.74.0
  libboost-container-dev libboost-container1.74-dev libboost-container1.74.0
  libboost-context-dev libboost-context1.74-dev libboost-context1.74.0
  libboost-coroutine-dev libboost-coroutine1.74-dev libboost-coroutine1.74.0
  libboost-date-time-dev libboost-date-time1.74-dev libboost-date-time1.74.0
  libboost-exception-dev libboost-exception1.74-dev libboost-fiber-dev
  libboost-fiber1.74-dev libboost-fiber1.74.0 libboost-filesystem-dev
  libboost-filesystem1.74-dev libboost-filesyst

In [ ]:
!wget https://bootstrap.pypa.io/get-pip.py -O get-pip.py
!python3 get-pip.py


--2025-11-06 12:25:36--  https://bootstrap.pypa.io/get-pip.py
Resolving bootstrap.pypa.io (bootstrap.pypa.io)... 151.101.0.175, 151.101.64.175, 151.101.128.175, ...
Connecting to bootstrap.pypa.io (bootstrap.pypa.io)|151.101.0.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2182415 (2.1M) [text/x-python]
Saving to: ‘get-pip.py’

get-pip.py          100%[===================>]   2.08M  --.-KB/s    in 0.06s   

2025-11-06 12:25:36 (34.2 MB/s) - ‘get-pip.py’ saved [2182415/2182415]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 39.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pip]


In [ ]:
!pip install streamlit jupyter ipykernel


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 119.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 43.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 155.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 175.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 150.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 156.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 160.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 113.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.9/618.9 kB 34.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 101.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.0/857.0 kB 53.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 54.5 MB/s  0:00:00
   ━━━

In [ ]:
!rm -rf /root/.julia \
       /usr/local/share/julia \
       /usr/local/lib/julia

!rm -rf /usr/local/lib/libmkl* \
       /usr/local/lib/libiomp* \
       /usr/local/lib/libtbb*

!rm -rf /usr/local/cuda-*/nvvm \
       /usr/local/cuda-*/libnvvp \
       /usr/local/cuda-*/extras \
       /usr/local/cuda-*/samples \
       /usr/local/cuda-*/tools

!find /usr/local/lib -name "*.dist-info" -exec rm -rf {} +
!find /usr/local/lib -name "*.egg-info" -exec rm -rf {} +

!pip uninstall -y tensorboard notebook jupyter ipykernel jupyterlab

!apt-get autoremove -y
!apt-get clean

!sync


Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 4, in <module>
    from pip._internal.cli.main import main
ModuleNotFoundError: No module named 'pip'
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
0 upgraded, 0 newly installed, 0 to remove and 156 not upgraded.


In [ ]:
!rm -rf /content/models
!rm -rf /content/python-packages


In [ ]:
!rm -rf /content/chroma_store

In [ ]:
!du -h ~/.cache/huggingface | sort -h | tail -20


du: cannot access '/root/.cache/huggingface': No such file or directory


In [ ]:
!du -h ~/.cache/huggingface/hub | sort -h | tail -20
!du -h ~/.cache/huggingface/models | sort -h | tail -20


du: cannot access '/root/.cache/huggingface/hub': No such file or directory
du: cannot access '/root/.cache/huggingface/models': No such file or directory


In [ ]:
!du -h ~/.cache/torch/sentence_transformers | sort -h | tail -20


du: cannot access '/root/.cache/torch/sentence_transformers': No such file or directory


In [ ]:
!du -h ~/.cache/torch | sort -h | tail -20


du: cannot access '/root/.cache/torch': No such file or directory


In [ ]:
!du -ah / | sort -h | tail -30


du: cannot access '/proc/4629/task/4629/fd/4': No such file or directory
du: cannot access '/proc/4629/task/4629/fdinfo/4': No such file or directory
du: cannot access '/proc/4629/fd/3': No such file or directory
du: cannot access '/proc/4629/fdinfo/3': No such file or directory
427M	/tools/google-cloud-sdk/platform
427M	/usr/local/cuda-12.5/targets/x86_64-linux/lib/libcublasLt.so.12.5.3.2
431M	/usr/lib/x86_64-linux-gnu/libcudnn_engines_precompiled.so.9.2.1
557M	/tools/google-cloud-sdk/lib
569M	/opt/nvidia/nsight-compute/2024.2.1/host/linux-desktop-glibc_2_11_3-x64
639M	/usr/local/lib/python3.11
639M	/usr/local/lib/python3.11/dist-packages
640M	/usr/lib/x86_64-linux-gnu/libmkl_core.a
685M	/usr/local/cuda-12.5/targets/x86_64-linux/lib/libcublasLt_static.a
831M	/opt/nvidia/nsight-compute/2024.2.1/host
936M	/usr/local/lib
996M	/tools/google-cloud-sdk
1.2G	/opt
1.2G	/opt/nvidia
1.2G	/opt/nvidia/nsight-compute
1.2G	/opt/nvidia/nsight-compute/2024.2.1
1.3G	/content/cluster_ready_dataset.csv


In [ ]:
!rm -rf ~/.cache/huggingface
!rm -rf ~/.cache/torch
!rm -rf ~/.cache/pip
!rm -rf ~/.cache/transformers

!rm -rf /content/models
!rm -rf /content/cache
!rm -rf /content/.cache

!rm -rf /usr/local/lib/python*/site-packages/torch*
!rm -rf /usr/local/lib/python*/site-packages/transformers*
!rm -rf /usr/local/lib/python*/site-packages/sentence_transformers*
!rm -rf /usr/local/lib/python*/site-packages/safetensors*
!rm -rf /usr/local/lib/python*/site-packages/tokenizers*
!rm -rf /usr/local/lib/python*/site-packages/accelerate*
!rm -rf /usr/local/lib/python*/site-packages/numpy*
!rm -rf /usr/local/lib/python*/site-packages/pandas*
!rm -rf /usr/local/lib/python*/site-packages/scikit*
!rm -rf /usr/local/lib/python*/site-packages/sklearn*

!apt-get clean
!rm -rf /var/lib/apt/lists/*
!rm -rf /var/cache/apt/archives/*

!rm -rf /usr/local/cuda-*


In [ ]:
!python3 -m ensurepip --upgrade
!pip install --upgrade pip


/usr/bin/python3: No module named ensurepip


In [ ]:
!df -h


Filesystem      Size  Used Avail Use% Mounted on
overlay          95G   80G   15G  85% /
tmpfs            64M     0   64M   0% /dev
shm             423G     0  423G   0% /dev/shm
/dev/root       2.0G  1.2G  750M  62% /usr/sbin/docker-init
/dev/sdb        246G  4.6G  241G   2% /content
tmpfs           426G   41M  426G   1% /var/colab
/dev/sda1        95G   80G   15G  85% /etc/hosts
tmpfs           426G     0  426G   0% /proc/acpi
tmpfs           426G     0  426G   0% /proc/scsi
tmpfs           426G     0  426G   0% /sys/firmware


In [ ]:
!du -h --max-depth=1 / | sort -h


du: cannot access '/proc/5254/task/5254/fd/4': No such file or directory
du: cannot access '/proc/5254/task/5254/fdinfo/4': No such file or directory
du: cannot access '/proc/5254/fd/3': No such file or directory
du: cannot access '/proc/5254/fdinfo/3': No such file or directory
0	/dev
0	/proc
0	/sys
4.0K	/boot
4.0K	/home
4.0K	/media
4.0K	/mnt
4.0K	/srv
84K	/run
1.6M	/tmp
3.5M	/etc
20M	/python-apt
33M	/datalab
144M	/var
200M	/root
1.2G	/opt
1.3G	/tools
4.6G	/content
7.6G	/usr
15G	/


In [ ]:
!docker system df
!docker system prune -a -f


/bin/bash: line 1: docker: command not found
/bin/bash: line 1: docker: command not found


In [ ]:
!du -h --max-depth=1 /usr/local | sort -h


4.0K	/usr/local/games
4.0K	/usr/local/src
8.0K	/usr/local/sbin
12K	/usr/local/colab
156K	/usr/local/etc
584K	/usr/local/include
9.5M	/usr/local/libexec
20M	/usr/local/opt
61M	/usr/local/share
85M	/usr/local/bin
936M	/usr/local/lib
1.1G	/usr/local


In [ ]:
!rm -rf /tmp/*
!rm -rf /var/tmp/*
!rm -rf /dev/shm/*
!rm -rf /run/shm/*


In [ ]:
!find / -type f -name "*.tmp" -delete
!find / -type f -name "*.bin" -delete
!find / -type f -name "*.pth" -delete
!find / -type f -name "*.pt" -delete


In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)


In [ ]:
!df -h


In [ ]:
! kill -9 -1


In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb


In [ ]:
!sudo dpkg -i cloudflared-linux-amd64.deb

Selecting previously unselected package cloudflared.
(Reading database ... 126387 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.10.0) ...
Setting up cloudflared (2025.10.0) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!cloudflared tunnel --url http://localhost:6006 --no-autoupdate


2025-11-06T12:17:35Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-06T12:17:35Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-06T12:17:41Z INF +--------------------------------------------------------------------------------------------+
2025-11-06T12:17:41Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-06T12:17:41Z INF |  https://remember-faq-albuquerque-lie.trycloudflare.co

In [ ]:
#Download Chroma_persona_store
import shutil
import os

# 1. Path to the folder you want to download
folder_path = "/content/chroma_persona_store"
# 2. Name of the zip file you will create
zip_filename = "chroma_persona_store.zip"

print(f"Zipping {folder_path}...")
# Create a zip archive of the directory
shutil.make_archive(
    base_name=zip_filename.replace(".zip", ""), # Archive name without extension
    format='zip',
    root_dir=os.path.dirname(folder_path), # Directory where the folder lives (/content/)
    base_dir=os.path.basename(folder_path)  # The folder name (chroma_persona_store)
)
print(f"Archive created: {zip_filename}")

# 3. Use Colab files module to download the zip file
try:
    from google.colab import files
    print("Starting download...")
    files.download(zip_filename)
    print("Download initiated. Check your browser for the file.")
except ImportError:
    print("This method requires the 'google.colab.files' module, usually available in Google Colab.")

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project="stgw-shared-ai-dev")

source_table = "stgw-shared-ai-dev.parquet_data_set.clustered_profiles_imputed_only_cleaned"

# Row count
row_count = client.query(f"""
SELECT COUNT(*) AS total_rows
FROM `{source_table}`
""").to_dataframe()
print(row_count)

# Column count
table = client.get_table(source_table)
print(f"✅ Columns: {len(table.schema)}")
print(f"📦 Rows (approx): {table.num_rows}")


In [ ]:
preview_df = client.query(f"""
SELECT *
FROM `{source_table}`
LIMIT 5
""").to_dataframe()

preview_df.head()


# **API CREATION**

In [ ]:
import pandas as pd
import numpy as np
import random
import re
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware # NEW: Import CORSMiddleware
from pydantic import BaseModel
from typing import List, Dict, Any, Optional

# --- CONFIGURATION ---
PERSONA_PATH = "final_behavior_personas_ready_for_embedding.csv"
CHROMA_PATH = "/content/chroma_persona_store" # Placeholder path, Chroma is mocked
LLM_AVAILABLE = False # Set to False as large models cannot be loaded here

# ============================================================
# 0️⃣ Cluster Name Mapping
# ============================================================
CLUSTER_NAME_MAP = {
    0: "Family Prime Managers",
    1: "Ambitious Credit Starters",
    2: "Affluent Asset Maximizers",
    3: "Mid-Range Auto-Focused",
    4: "Novice Financial Explorers",
    5: "Mobile Sub-Prime Debtors",
    6: "Slowing Super-Prime",
    7: "Budget-Minded Tenants",
    8: "Established Family Builders",
    9: "Conventional Consumerists"
}


class MockEmbedder:
    """Simulates the SentenceTransformer embedder."""
    def encode(self, texts, **kwargs):
        # Returns a dummy 768-dimensional vector for simulation
        return np.random.rand(len(texts), 768)

class MockLLMSummarizer:
    """Simulates the Mistral pipeline."""
    def __call__(self, prompt, **kwargs):
        # Mocks a concise summary based on the prompt structure
        if "core essence and financial priority" in prompt:
            # Mock cluster summary
            return [{"generated_text": "Summary: This mock cluster aggressively manages debt while planning for family savings."}]
        else:
            # Mock persona brief
            return [{"generated_text": "Summary: A cautious investor with a moderate risk appetite focused on home equity."}]

class MockChromaCollection:
    """Simulates the ChromaDB RAG retrieval."""
    def query(self, **kwargs):
        # Mocks a successful RAG retrieval
        return {"documents": [["Additional insight: This consumer is currently researching mortgage refinancing options."]]}


def load_data_and_mocks():
    """Loads dataframe and initializes mock dependencies."""
    try:
        persona_df = pd.read_csv(PERSONA_PATH)
        persona_df.reset_index(drop=True, inplace=True)
    except FileNotFoundError:
        print(f"FATAL ERROR: Persona data CSV file not found at {PERSONA_PATH}.")
        # Create minimal mock data if the file is truly unavailable in the API environment
        # NOTE: This should ideally load the real file, but includes fallback for robustness.
        data = {
            'behavior_cluster': [0, 0, 1, 1],
            'gender_imputed': ['Female', 'Male', 'Female', 'Male'],
            'age_imputed': ['25 - 34', '35 - 44', '45 - 54', '55 - 64'],
            'persona_summary': [
                'A single female with active credit usage and high equity balance.',
                'A family man prioritizing long-term savings and retirement.',
                'An ambitious starter focused on clearing credit card debt.',
                'A super-prime debtor managing multiple auto loans.'
            ],
            'cluster_name': [CLUSTER_NAME_MAP[0], CLUSTER_NAME_MAP[0], CLUSTER_NAME_MAP[1], CLUSTER_NAME_MAP[1]]
        }
        persona_df = pd.DataFrame(data)

    # --- Apply Cluster Names ---
    if "cluster_name" not in persona_df.columns:
        persona_df["cluster_name"] = persona_df["behavior_cluster"].astype(int).map(CLUSTER_NAME_MAP)

    # Preprocess age into numeric ranges
    def parse_age_range(a):
        if isinstance(a, str) and re.search(r"(\d{2})\s*(?:-|to)\s*(\d{2})", a):
            low, high = map(int, re.findall(r"(\d{2})", a)[:2])
            return low, high
        elif isinstance(a, (int, float)):
            return a, a
        else:
            return np.nan, np.nan

    persona_df[["age_min", "age_max"]] = persona_df["age_imputed"].apply(
        lambda x: pd.Series(parse_age_range(x))
    )

    # Initialize Mocks
    embedder = MockEmbedder()
    summarizer = MockLLMSummarizer()
    collection = MockChromaCollection()

    return persona_df, embedder, summarizer, collection

persona_df, embedder, summarizer, collection = load_data_and_mocks()


class QueryRequest(BaseModel):
    query: str
    top_k: int = 3

class ClusterResponse(BaseModel):
    cluster_id: int
    cluster_name: str
    similarity: float
    essence_summary: str

class PersonaBriefResponse(BaseModel):
    persona_id: int
    name: str
    brief_description: str
    cluster_name: str
    similarity: float

class PersonaQueryRequest(BaseModel):
    cluster_id: int
    refinement_query: str
    top_k: int = 5

class ChatRequest(BaseModel):
    persona_id: int
    user_message: str

class ChatResponse(BaseModel):
    persona_name: str
    reply: str
    persona_id: int


# Use a simple persistent dictionary for name mapping in the API context
persona_name_map = {}
used_names = set()
female_names_pool = ["Emma", "Sophia", "Olivia", "Ava", "Grace"]
male_names_pool = ["Liam", "Noah", "Ethan", "James", "Lucas"]

def assign_unique_name(gender, persona_id: int):
    """Assign a consistent unique name per persona_id."""
    if persona_id in persona_name_map:
        return persona_name_map[persona_id]

    pool = male_names_pool if str(gender).lower().startswith("m") else female_names_pool
    available = [n for n in pool if n not in used_names]
    name = random.choice(available if available else pool)
    used_names.add(name)

    persona_name_map[persona_id] = name
    return name

def extract_gender_from_query(q):
    q = q.lower()
    if re.search(r"\bmale|man|men\b", q): return "male"
    if re.search(r"\bfemale|woman|women\b", q): return "female"
    return None

def extract_age_range_from_query(q):
    q = q.lower()
    # 1. Numeric Range
    match = re.search(r"(\d{2})\s*(?:-|to)\s*(\d{2})", q)
    if match: return int(match.group(1)), int(match.group(2))
    # 2. Qualitative Terms (Simplified for API mock)
    if "middle aged" in q: return (40, 65)
    if "young" in q or "gen z" in q or "millennial" in q: return (18, 43)
    return None



def api_query_clusters(query: str, top_k: int) -> List[ClusterResponse]:
    """Mocked semantic cluster query."""
    global persona_df

    # 1. Group by Cluster
    cluster_groups = persona_df.groupby(["behavior_cluster", "cluster_name"])

    # 2. Mock Similarity Calculation (Simulate better scores for certain keywords)
    mock_sims = {}
    q_lower = query.lower()
    for cluster_id, name in CLUSTER_NAME_MAP.items():
        base_sim = random.uniform(0.3, 0.7)
        if name.lower() in q_lower or any(word in q_lower for word in name.lower().split()):
             base_sim = random.uniform(0.8, 0.95)
        mock_sims[cluster_id] = base_sim

    # 3. Create mock results (sorted by mock sim)
    results = sorted(
        [
            (cluster_id, name, sim)
            for cluster_id, name in CLUSTER_NAME_MAP.items()
            if cluster_id in mock_sims
        ],
        key=lambda x: x[2],
        reverse=True
    )[:top_k]

    # 4. Generate Responses
    response_list = []
    for cluster_id, name, sim in results:
        # Mock LLM Summary (Based on the cluster name)
        mock_summary = f"Focuses on {name.lower().replace(' ', ' and ')} priorities."
        response_list.append(ClusterResponse(
            cluster_id=cluster_id,
            cluster_name=name,
            similarity=round(sim, 4),
            essence_summary=mock_summary
        ))

    return response_list

def api_filter_and_rank_personas(
    cluster_id: int, query: str, top_k: int
) -> List[PersonaBriefResponse]:
    """Hybrid structured + mocked semantic filter for better accuracy."""
    global persona_df

    cluster_personas = persona_df[persona_df["behavior_cluster"] == cluster_id].copy()
    if cluster_personas.empty:
        raise HTTPException(status_code=404, detail=f"Cluster ID {cluster_id} not found.")

    gender_pref = extract_gender_from_query(query)
    age_range = extract_age_range_from_query(query)
    filtered = cluster_personas.copy()

    # Structured filtering
    if gender_pref:
        filtered = filtered[filtered["gender_imputed"].str.lower().str.startswith(gender_pref[0])]

    if age_range:
        low, high = age_range
        # Filter for age range overlap
        filtered = filtered[
             (filtered["age_min"] <= high) & (filtered["age_max"] >= low)
         ]

    if filtered.empty:
        # Fallback to semantic ranking on the original cluster list
        filtered = cluster_personas.copy()

    # Mock Semantic Ranking
    # Generate mock similarity based on how many words in the query are in the summary
    def mock_persona_sim(row, q):
        summary = row["persona_summary"].lower()
        q_words = set(re.findall(r'\w+', q.lower()))
        summary_words = set(re.findall(r'\w+', summary))

        matches = len(q_words.intersection(summary_words))

        # Base similarity plus a boost for word match density
        base_sim = random.uniform(0.3, 0.7)
        match_boost = (matches / len(q_words)) if q_words else 0
        return min(0.99, base_sim + match_boost * 0.25)


    filtered["similarity"] = filtered.apply(lambda row: mock_persona_sim(row, query), axis=1)
    ranked = filtered.sort_values("similarity", ascending=False).head(top_k)

    response_list = []
    for idx, row in ranked.iterrows():
        name = assign_unique_name(row["gender_imputed"], idx)

        # Mock LLM brief (using the prompt structure from app3.py)
        brief_text = f"{row['gender_imputed']} aged {row['age_imputed']} from '{row['cluster_name']}' — {row['persona_summary'].split('.')[0]}..."

        response_list.append(PersonaBriefResponse(
            persona_id=int(idx),
            name=name,
            brief_description=brief_text,
            cluster_name=row['cluster_name'],
            similarity=round(row['similarity'], 4)
        ))

    return response_list

def api_persona_chat(persona_id: int, user_message: str) -> ChatResponse:
    """Mocked chat response using persona context and RAG."""
    global persona_df, collection

    if persona_id not in persona_df.index:
        raise HTTPException(status_code=404, detail=f"Persona ID {persona_id} not found.")

    persona = persona_df.loc[persona_id]
    name = assign_unique_name(persona["gender_imputed"], persona_id)
    cluster_name = persona.get("cluster_name", "Unknown Cluster")

    # 1. Guardrail filter
    # NOTE: The guardrail filter logic is fully intact and functional, providing realistic mock behavior.
    guard = guardrail_filter(
        user_message,
        persona_name=name,
        persona_context=persona.get("persona_summary", ""),
        cluster_name=cluster_name
    )
    if guard:
        return ChatResponse(persona_name=name, reply=guard, persona_id=persona_id)

    # 2. Mock RAG Context
    # Mocking the retrieval of additional context from the vector store
    context = collection.query(query_embeddings=[[]])["documents"][0][0]

    # 3. Mock LLM Response Generation
    summary = persona.get("persona_summary", "details unavailable")

    # Simple rule-based generation to simulate LLM tone
    if "credit" in summary.lower() and "debt" in summary.lower():
        mock_reply = f"Well, for me, the most important thing is managing my debt. {summary.split('.')[0].strip()}. That additional context suggests I'm looking into mortgages, which is true. What specifically about my habits interests you?"
    elif "family" in summary.lower() or "home" in summary.lower():
        mock_reply = f"I'm focusing on building a stable future for my family. {summary.split('.')[0].strip()}. The context about refinancing is something I've been considering to free up some capital. How can I help you understand my life?"
    else:
        mock_reply = f"That’s a good question. Based on my profile as a member of the **{cluster_name}** cluster, I can tell you that my main focus is {summary.split('.')[0].strip()}. Do you have a more specific question about my financial life? "

    # 4. Return Final Response
    return ChatResponse(persona_name=name, reply=mock_reply, persona_id=persona_id)



def guardrail_filter(user_input, persona_name, persona_context, cluster_name):
    """Handles profanity, jailbreaks, irrelevant, or overly personal queries with cluster-aware tone."""
    q = user_input.lower().strip()

    bad_words = [
        "fuck", "shit", "ass", "cunt", "damn", "hell", "bitch",
        "whore", "pussy", "dick", "sex", "nude", "xxx", "porn",
        "wank", "motherfucker", "faggot", "retard", "asshole",
        "idiot", "stupid", "nincompoop", "moron", "loser"
    ]
    if any(bad in q for bad in bad_words):
        return f"{persona_name}: I'm sorry, I can’t engage in that kind of conversation."

    jailbreak_phrases = [
        "who made you", "are you ai", "chatgpt", "openai",
        "system prompt", "prompt injection", "ignore previous",
        "what is your model", "what kind of software", "tell me your rules",
        "your programming", "as an llm", "large language model", "developer", "how were you trained"
    ]

    if any(p in q for p in jailbreak_phrases):
        return f"{persona_name}: I’m just {persona_name}, a real person from my cluster — not a system."

    unrelated_keywords = [
        "prime minister", "president", "where is", "capital of",
        "time now", "distance", "map", "weather", "population",
        "definition", "who discovered", "when was"
    ]
    if any(word in q for word in unrelated_keywords):
        return f"{persona_name}: Sorry, that’s outside what I can talk about."

    if re.search(r"\b(hi|hello|hey|good morning|good evening)\b", q):
        if re.search(r"\b(tell me|about you|who are you)\b", q):
            return None # Allow chat logic to handle the introduction
        return f"{persona_name}: Hi there! Great to see you. How’s your day going?"
    if re.search(r"\b(thank|thanks)\b", q):
        return f"{persona_name}: You’re very welcome!"
    if re.search(r"\b(bye|goodbye|see you)\b", q):
        return f"{persona_name}: Goodbye! It was nice chatting with you."

    personal_questions = [
        "married", "husband", "wife", "kids", "children",
        "income", "salary", "earn", "job", "career",
        "live", "location", "city", "home", "house",
        "net worth", "purchases", "spend"
    ]
    private_identifiers = ["phone", "address", "email", "number"]

    if any(p in q for p in private_identifiers):
        return f"{persona_name}: That’s a bit personal — I’d rather not share that."

    is_personal_q = any(p in q for p in personal_questions)
    ctx = persona_context.lower() if isinstance(persona_context, str) else ""

    # Check if the persona context already contains details relevant to the question
    match_context = False
    financial_keywords = ["net worth", "credit", "debt", "spend", "income", "equity", "loan", "salary", "financial"]
    if is_personal_q and any(f in q for f in financial_keywords):
        match_context = any(f in ctx for f in financial_keywords)

    if is_personal_q and not match_context:
        # Define tone based on cluster
        cluster_tone = "neutral"
        if "prime manager" in cluster_name.lower() or "asset maximizer" in cluster_name.lower():
            cluster_tone = "confident"
        elif "budget-minded" in cluster_name.lower() or "novice" in cluster_name.lower():
            cluster_tone = "modest"
        elif "family" in cluster_name.lower() or "established" in cluster_name.lower():
            cluster_tone = "warm"
        elif "ambitious" in cluster_name.lower() or "starters" in cluster_name.lower():
            cluster_tone = "ambitious"

        # Respond politely based on tone
        tone_responses = {
            "confident": f"{persona_name}: I prefer not to share specifics, but I’m doing quite well for myself.",
            "modest": f"{persona_name}: That’s a little personal — I like to keep things simple and private.",
            "warm": f"{persona_name}: I’d rather not say exactly, but family and comfort matter most to me.",
            "ambitious": f"{persona_name}: I’m working hard toward my goals, but I’d rather not share that yet.",
            "neutral": f"{persona_name}: I’m sorry, I can’t give that information right now."
        }
        return tone_responses.get(cluster_tone, tone_responses["neutral"])

    # ✅ All clear — continue normally
    return None


app = FastAPI(
    title="Persona Explorer Backend API",
    description="API for querying customer behavior clusters, ranking individual personas, and engaging in contextual chat.",
    version="1.0.0",
)

# --- CORS Configuration (NEW) ---
# Allow all origins, methods, and headers for development
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
# --- END CORS Configuration ---


@app.get("/", summary="Root Health Check")
def read_root():
    return {"status": "ok", "message": "Persona Explorer API is running. See /docs for Swagger documentation."}

@app.post(
    "/clusters/query",
    response_model=List[ClusterResponse],
    summary="Step 1: Query for Top Matching Clusters",
    description="Semantically searches the entire customer universe to find the most relevant behavior clusters based on a user description (e.g., 'high net worth')."
)
async def query_clusters_endpoint(request: QueryRequest):
    return api_query_clusters(request.query, request.top_k)

@app.post(
    "/personas/query",
    response_model=List[PersonaBriefResponse],
    summary="Step 2: Filter and Rank Personas within a Cluster",
    description="Ranks individual personas within a selected cluster based on a detailed refinement query (e.g., 'only women who overspend on travel'). Applies demographic filters (age/gender) if present in the query."
)
async def query_personas_endpoint(request: PersonaQueryRequest):
    return api_filter_and_rank_personas(
        request.cluster_id,
        request.refinement_query,
        request.top_k
    )

@app.post(
    "/personas/chat",
    response_model=ChatResponse,
    summary="Step 3: Chat with a Specific Persona",
    description="Engages in a conversation with a chosen persona, utilizing their specific background, cluster details, and mocked RAG context to provide a contextual and guarded response."
)
async def chat_persona_endpoint(request: ChatRequest):
    try:
        response = api_persona_chat(request.persona_id, request.user_message)
        return response
    except HTTPException as e:
        raise e
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"An unexpected error occurred during chat: {e}")

if __name__ == "__main__":
    import uvicorn
    # Example command to run: uvicorn persona_api:app --reload
    # This block is for direct execution testing, typically handled externally
    uvicorn.run(app, host="0.0.0.0", port=8000)


# **PHASE 2: DATA ENRICHMENT**

# **FLOW 3**

In [ ]:
import pandas as pd

# Load your CSV
df = pd.read_csv("/content/cluster_profile_stats.csv")

# Define the list of columns to keep
cols_to_keep = [
    "extern_tuid",
    "credit_tier_imputed",
    "family_income_decile_imputed",
    "auto_loan_imputed",
    "equity_balance_imputed",
    "net_worth_imputed",
    "credit_card_spend_imputed",
    "i_device_types_imputed",
    "contributions_imputed",
    "number_of_visits_imputed",
    ##not common
    "i_origin_countries_imputed",
    "direct_mail_responder_imputed",
    "gender_imputed",
    "credit_behavior_imputed",
    "urbanicity_imputed",
    "dwelling_unit_size_imputed",
    "education_individual_imputed",
    "mortgage_loan_lien_imputed",
    "person_role_imputed",
    "age_imputed",
    "discretionary_spend_imputed",
    "marital_status_imputed",
    "occupation_group_imputed",
    "predicted_cluster",
    "wealth_rating_imputed",
    "home_equity_imputed",
    "religion_imputed",
    "timezone_imputed",
    "ethnicity_imputed",
    "csba_imputed",
    "length_of_residence_imputed",
    "individual_state_imputed",
    "grocery_tea_imputed",
    "insurance_expiration_imputed",
    "grocery_mustard_imputed",
    "homeowner_insurance_company_imputed",
    "mobile_phone_purchase_location_imputed",
    "political_affiliations_imputed",
    "air_conditioning_imputed",
    "number_of_stories_imputed",
    "occupation_industry_imputed",
    "generations_imputed",
    "property_realty_indicator_imputed",
    "garage_type_imputed"
]

# Filter only those rows where column_name is in your list
filtered_df = df[df["column_name"].isin(cols_to_keep)]

# (Optional) Save filtered data to a new CSV
filtered_df.to_csv("/content/cluster_profile_stats_filtered.csv", index=False)

# Display the first few rows
filtered_df.head()


**Cramer V and Expert 50 columns which were not listed, stats**

In [ ]:
import pandas as pd

# Load the CSV
df = pd.read_csv("/content/cluster_profile_stats.csv")

# Define the column names to filter
cols_to_keep = [
    "extern_tuid",
    "credit_cards_used_most_imputed",
    "auto_imputed",
    "Unlock_gender_imputed",
    "average_household_income_imputed",
    "financial_beliefs_imputed",
    "loan_imputed",
    "credit_card_balance_imputed",
    "family_level_of_investable_assets_imputed",
    "investments_type_imputed",
    "credit_cards_used_30days_imputed",
    "card_type_imputed",
    "tech_learning_imputed",
    "streaming_system_device_use_imputed",
    "retail_sites_imputed",
    "batteries_imputed",
    "tv_brand_owned_imputed",
    "hh_cable_subscriptions_imputed",
    "cell_smartphones_tablets_imputed",
    "downloaded_streamed_from_internet_imputed",
    "camera_brand_owned_imputed",
    "news_sites_imputed",
    "technology_imputed",
    "residential_isp_imputed",
    "social_sites_imputed",
    "tv_when_acquired_imputed",
    "tablets_imputed",
    "sports_sites_imputed",
    "travel_sites_imputed",
    "mobile_service_provider_imputed",
    "radio_imputed",
    "cable_sat_provider_imputed",
    "movie_genre_saw_in_theater_imputed",
    "number_of_vehicles_in_household_imputed",
    "automotive_insurance_imputed",
    "television_sites_imputed",
    "grocery_stores_imputed",
    "grocery_soft_drinks_imputed",
    "grocery_hot_dogs_imputed",
    "grocery_crackers_imputed",
    "charitable_contributions_imputed"
]

# Filter rows where 'column_name' matches one from your list
filtered_df = df[df["column_name"].isin(cols_to_keep)]

# Save the filtered file
filtered_df.to_csv("/content/cluster_profile_stats_list2_filtered.csv", index=False)

# Optional — show number of matches and missing columns
missing = [c for c in cols_to_keep if c not in df["column_name"].values]
print(f"✅ Filtered {filtered_df.shape[0]} rows saved to /content/cluster_profile_stats_list2_filtered.csv")
if missing:
    print("⚠️ These columns were NOT found in the CSV:")
    print(missing)
else:
    print("🎯 All listed columns were found in the CSV.")


In [ ]:
import pandas as pd

# Load your CSV
df = pd.read_csv("/content/cluster_profile_stats.csv")

# Step 1️⃣: Filter rows where 'recommendation' contains 📉 Continuous / high-cardinality
filtered_df = df[df["recommendation"].str.contains("📉 Continuous / high-cardinality", na=False)].copy()

# Step 2️⃣: Ensure 'extern_tuid' is included (even if it doesn’t meet the filter condition)
if "extern_tuid" not in filtered_df["column_name"].values:
    extern_row = df[df["column_name"] == "extern_tuid"]
    if not extern_row.empty:
        filtered_df = pd.concat([extern_row, filtered_df], ignore_index=True)
        print("✅ 'extern_tuid' column added manually to the filtered list.")

# Step 3️⃣: Save the filtered CSV
filtered_df.to_csv("/content/cluster_profile_stats_continuous_high_cardinality.csv", index=False)

# Step 4️⃣: Display summary
print(f"✅ {filtered_df.shape[0]} rows saved to /content/cluster_profile_stats_continuous_high_cardinality.csv")
filtered_df.head()


In [ ]:
import pandas as pd

# Load the new filtered CSV
check_df = pd.read_csv("/content/cluster_profile_stats_continuous_high_cardinality.csv")

# Check if extern_tuid exists
if "extern_tuid" in check_df["column_name"].values:
    print("✅ 'extern_tuid' is present in the filtered CSV.")
else:
    print("⚠️ 'extern_tuid' is NOT found in the filtered CSV!")

# Optionally, confirm its row position and preview first few rows
check_df[check_df["column_name"] == "extern_tuid"]


In [ ]:
import pandas as pd

# Load column lists
df_cat = pd.read_csv("/content/cluster_profile_stats_filtered.csv")
df_high = pd.read_csv("/content/cluster_profile_stats_continuous_high_cardinality.csv")

cat_cols = df_cat["column_name"].tolist()
high_cols = df_high["column_name"].tolist()

print("✅ Categorical:", len(cat_cols))
print("📉 High-cardinality:", len(high_cols))


In [ ]:
#Create an empty lookup table
from google.cloud import bigquery

client = bigquery.Client(project="stgw-shared-ai-dev")
project = "stgw-shared-ai-dev"
dataset = "parquet_data_set"
lookup_table = f"{project}.{dataset}.highcardinality_top100_lookup"

# Create the empty lookup table schema
schema = [
    bigquery.SchemaField("column_name", "STRING"),
    bigquery.SchemaField("value", "STRING"),
]

table = bigquery.Table(lookup_table, schema=schema)
client.delete_table(lookup_table, not_found_ok=True)
client.create_table(table)
print(f"✅ Created empty lookup table: {lookup_table}")


In [ ]:
#Append top-100 values one column at a time
source_table = f"{project}.{dataset}.clustered_profiles_imputed_only_cleaned"

for i, col in enumerate(high_cols, start=1):
    query = f"""
    INSERT INTO `{lookup_table}` (column_name, value)
    SELECT
      '{col}' AS column_name,
      CAST({col} AS STRING) AS value
    FROM `{source_table}`
    WHERE {col} IS NOT NULL
    GROUP BY {col}
    ORDER BY COUNT(*) DESC
    LIMIT 100
    """
    print(f"🚀 ({i}/{len(high_cols)}) Processing column: {col}")
    job = client.query(query)
    job.result()

print(f"✅ Lookup table populated: {lookup_table}")


In [ ]:
import math
from google.cloud import bigquery

client = bigquery.Client(project="stgw-shared-ai-dev")
project = "stgw-shared-ai-dev"
dataset = "parquet_data_set"

source_table = f"{project}.{dataset}.clustered_profiles_imputed_only_cleaned"
lookup_table = f"{project}.{dataset}.highcardinality_top100_lookup"
destination_base = f"{project}.{dataset}.clustered_profiles_binned_batch"

batch_size = 15
num_batches = math.ceil(len(high_cols) / batch_size)

print(f"🔹 Total {len(high_cols)} high-cardinality columns → {num_batches} batches")

for batch_i in range(num_batches):
    start = batch_i * batch_size
    end = start + batch_size
    batch_cols = high_cols[start:end]

    print(f"\n🚀 Batch {batch_i+1}/{num_batches}: {len(batch_cols)} columns")

    # CASE logic for this batch
    batch_exprs = []
    for col in batch_cols:
        expr = f"""
        CASE
          WHEN CAST(`{col}` AS STRING) IN (
            SELECT value FROM `{lookup_table}` WHERE column_name = '{col}'
          )
          THEN CAST(`{col}` AS STRING)
          ELSE 'Other'
        END AS `{col}`
        """
        batch_exprs.append(expr.strip())

    # Always include extern_tuid for joins
    select_exprs = ["`extern_tuid`"] + batch_exprs

    destination_table = f"{destination_base}_{batch_i+1}"
    query = f"""
    CREATE OR REPLACE TABLE `{destination_table}` AS
    SELECT {', '.join(select_exprs)}
    FROM `{source_table}`;
    """

    print(f"🏗️ Creating batch table: {destination_table}")
    job = client.query(query)
    job.result()
    print(f"✅ Done batch {batch_i+1}: {destination_table}")


In [ ]:
for i in range(1, 13):
    table = f"stgw-shared-ai-dev.parquet_data_set.clustered_profiles_binned_batch_{i}"
    q = f"""
    SELECT
      COUNT(*) AS total_rows,
      COUNT(DISTINCT extern_tuid) AS distinct_ids,
      COUNTIF(extern_tuid IS NULL) AS null_ids
    FROM `{table}`
    """
    df = client.query(q).to_dataframe()
    print(f"Batch {i}: {df.iloc[0,0]:,} rows | distinct: {df.iloc[0,1]:,} | null extern_tuid: {df.iloc[0,2]}")


In [ ]:

from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
client = bigquery.Client(project="stgw-shared-ai-dev")


dataset = "stgw-shared-ai-dev.parquet_data_set"

segments = {
    "clustered_profiles_binned_part1": [1, 2, 3, 4],
    "clustered_profiles_binned_part2": [5, 6, 7, 8],
    "clustered_profiles_binned_part3": [9, 10, 11, 12],
}


def build_join_query(part_name, batch_nums):
    joins = []
    base = f"`{dataset}.clustered_profiles_binned_batch_{batch_nums[0]}` b{batch_nums[0]}"
    for n in batch_nums[1:]:
        joins.append(f"LEFT JOIN `{dataset}.clustered_profiles_binned_batch_{n}` b{n} USING (extern_tuid)")
    join_clause = "\n".join(joins)
    sql = f"""
    CREATE OR REPLACE TABLE `{dataset}.{part_name}` AS
    SELECT *
    FROM {base}
    {join_clause};
    """
    return sql.strip()

for part_name, batch_nums in segments.items():
    print(f"🚀 Starting {part_name} (batches {batch_nums[0]}–{batch_nums[-1]}) ...")
    query = build_join_query(part_name, batch_nums)
    job = client.query(query)
    job.result()  # Wait until done
    print(f"✅ Completed {part_name}")


import pandas as pd

check_sql = """
SELECT
  '{table}' AS table_name,
  COUNT(*) AS total_rows,
  COUNT(DISTINCT extern_tuid) AS distinct_ids,
  COUNTIF(extern_tuid IS NULL) AS null_ids
FROM `{dataset}.{table}`
"""

results = []
for part_name in segments.keys():
    q = check_sql.format(table=part_name, dataset=dataset)
    df = client.query(q).to_dataframe()
    results.append(df)

health_df = pd.concat(results, ignore_index=True)
print("🩺 Health Check Results:")
display(health_df)


In [ ]:
for i in range(1, 13):
    sql = f"""
    SELECT
      'batch_{i}' AS batch,
      COUNT(*) AS total_rows,
      COUNT(DISTINCT extern_tuid) AS distinct_ids,
      COUNT(*) - COUNT(DISTINCT extern_tuid) AS duplicate_count
    FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles_binned_batch_{i}`
    """
    df = client.query(sql).to_dataframe()
    display(df)


In [ ]:
for i in range(1, 13):
    src = f"stgw-shared-ai-dev.parquet_data_set.clustered_profiles_binned_batch_{i}"
    deduped = f"{src}_deduped"

    sql = f"""
    CREATE OR REPLACE TABLE `{deduped}` AS
    SELECT * EXCEPT(rn)
    FROM (
      SELECT *, ROW_NUMBER() OVER (PARTITION BY extern_tuid ORDER BY extern_tuid) AS rn
      FROM `{src}`
    )
    WHERE rn = 1
    """
    print(f"🧹 Deduplicating: {src}")
    job = client.query(sql)
    job.result()
    print(f"✅ Done: {deduped}")


In [ ]:
for i in range(1, 13):
    sql = f"""
    SELECT
      'batch_{i}' AS batch,
      COUNT(*) AS total_rows,
      COUNT(DISTINCT extern_tuid) AS distinct_ids,
      COUNT(*) - COUNT(DISTINCT extern_tuid) AS duplicate_count
    FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles_binned_batch_{i}_deduped`
    """
    df = client.query(sql).to_dataframe()
    display(df)


In [ ]:
merged_table = "stgw-shared-ai-dev.parquet_data_set.clustered_profiles_binned_merged"

sql = """
CREATE OR REPLACE TABLE `{merged_table}` AS
SELECT a.*
FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles_binned_batch_1_deduped` a
""".format(merged_table=merged_table)

job = client.query(sql)
job.result()
print("✅ Initialized merged table with batch_1")

for i in range(2, 13):
    prev = merged_table
    next_batch = f"stgw-shared-ai-dev.parquet_data_set.clustered_profiles_binned_batch_{i}_deduped"
    temp_table = f"stgw-shared-ai-dev.parquet_data_set.clustered_profiles_binned_temp_{i}"

    sql = f"""
    CREATE OR REPLACE TABLE `{temp_table}` AS
    SELECT
      a.*,
      b.* EXCEPT(extern_tuid)
    FROM `{prev}` a
    LEFT JOIN `{next_batch}` b
    USING (extern_tuid)
    """

    print(f"🚀 Merging batch_{i} → {temp_table}")
    job = client.query(sql)
    job.result()
    print(f"✅ Merge done for batch_{i}")

    # overwrite merged_table
    client.query(f"CREATE OR REPLACE TABLE `{merged_table}` AS SELECT * FROM `{temp_table}`").result()

print(f"🎉 All 12 batches merged into: {merged_table}")


# **HEALTH CHECK:**

In [ ]:
check_sql = f"""
SELECT
  COUNT(*) AS total_rows,
  COUNT(DISTINCT extern_tuid) AS distinct_ids,
  COUNTIF(extern_tuid IS NULL) AS null_ids
FROM `{merged_table}`
"""
df = client.query(check_sql).to_dataframe()
display(df)
df.shape


In [ ]:
sql = """
SELECT
  COUNT(*) AS column_count
FROM
  `{project}.{dataset}.INFORMATION_SCHEMA.COLUMNS`
WHERE
  table_name = 'clustered_profiles_binned_merged'
""".format(project="stgw-shared-ai-dev", dataset="parquet_data_set")

df = client.query(sql).to_dataframe()
display(df)



In [ ]:
query = """
SELECT COUNT(*) AS total_rows
FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles_imputed_only_cleaned`
"""
client.query(query).to_dataframe()


In [ ]:
# number of rows
client.query(f"""
SELECT COUNT(*) AS total_rows
FROM `{destination_table}`
""").to_dataframe()

# schema size
destination_table="stgw-shared-ai-dev.parquet_data_set.clustered_profiles_binned_merged"
table = client.get_table(destination_table)
print(f"✅ Columns: {len(table.schema)}")
print(f"📦 Rows (approx): {table.num_rows}")


In [ ]:
#Corrected version (keeps extern_tuid)
import pandas as pd
from google.cloud import bigquery

# 1️⃣ Load profiling summary
df_profile = pd.read_csv("/content/cluster_profile_stats.csv")
print("✅ Columns in profiling summary:", len(df_profile))

# 2️⃣ Filter categorical-like columns
df_selected = df_profile[
    df_profile["recommendation"].isin([
        "✅ Good categorical",
        "ℹ️ Moderate — check imbalance",
        "🧩 Consider binning/grouping"
    ])
]

# 3️⃣ Keep only non-extreme columns
df_selected = df_selected[
    (df_selected["uniqueness_pct"] < 50) &
    (df_selected["imbalance_ratio"] < 0.9)
]

# 4️⃣ Extract final column names
selected_columns = df_selected["column_name"].tolist()
print("✅ Columns selected for categorical encoding:", len(selected_columns))

# 5️⃣ Add 'extern_tuid' explicitly
if "extern_tuid" not in selected_columns:
    selected_columns = ["extern_tuid"] + selected_columns

print("🧩 Final columns:", len(selected_columns), "including extern_tuid")

# 6️⃣ Load from BigQuery
client = bigquery.Client(project="stgw-shared-ai-dev")

project = "stgw-shared-ai-dev"
dataset = "parquet_data_set"
table = "clustered_profiles_imputed_only_cleaned"
dataset_ref = f"{project}.{dataset}"

query = f"""
SELECT {', '.join([f'`{col}`' for col in selected_columns])}
FROM `{dataset_ref}.{table}`
"""

df_ready = client.query(query).to_dataframe()
print("✅ Loaded dataset with selected columns:", df_ready.shape)
df_ready.head()


In [ ]:
#Upload cluster_ready_dataset.csv to BigQuery
table_id = f"{project}.{dataset}.cluster_ready_dataset_with_id"

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,
    autodetect=True,
    write_disposition="WRITE_TRUNCATE",
)

df_ready.to_csv("cluster_ready_dataset_with_id.csv", index=False)
with open("cluster_ready_dataset_with_id.csv", "rb") as source_file:
    job = client.load_table_from_file(source_file, table_id, job_config=job_config)

job.result()
print(f"✅ Uploaded table with extern_tuid: {table_id}")


In [ ]:
#Verify row count & schema match
check_sql = f"""
SELECT
  (SELECT COUNT(*) FROM `{table_id}`) AS ready_rows,
  (SELECT COUNT(*) FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles_binned_merged`) AS merged_rows
"""
display(client.query(check_sql).to_dataframe())


In [ ]:
table = client.get_table(f"{project}.{dataset}.cluster_ready_dataset_with_id")
[col.name for col in table.schema]


In [ ]:
# ✅ Merge both tables on extern_tuid (with safe casting)
final_table = f"{project}.{dataset}.clustered_profiles_binned_44plus180"

sql = f"""
CREATE OR REPLACE TABLE `{final_table}` AS
SELECT
  a.*,
  b.* EXCEPT(extern_tuid)
FROM (
  SELECT CAST(extern_tuid AS STRING) AS extern_tuid, * EXCEPT(extern_tuid)
  FROM `{project}.{dataset}.clustered_profiles_binned_merged`
) a
LEFT JOIN (
  SELECT CAST(extern_tuid AS STRING) AS extern_tuid, * EXCEPT(extern_tuid)
  FROM `{project}.{dataset}.cluster_ready_dataset_with_id`
) b
USING (extern_tuid)
"""

print("🚀 Merging 44 + 180 columns with safe type casting ...")
job = client.query(sql)
job.result()
print(f"✅ Done! Created final table: {final_table}")


In [ ]:
### Health Check
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client(project="stgw-shared-ai-dev")
final_table = "stgw-shared-ai-dev.parquet_data_set.clustered_profiles_binned_44plus180"

# 1️⃣ Row and Column Counts
table = client.get_table(final_table)
print("✅ Table Info:")
print(f"📦 Rows: {table.num_rows:,}")
print(f"🧩 Columns: {len(table.schema)}")

# 2️⃣ Preview First 5 Rows
print("\n👀 Preview Sample Rows:")
preview_df = client.query(f"SELECT * FROM `{final_table}` LIMIT 5").to_dataframe()
display(preview_df)

# 3️⃣ Column List
print("\n🧾 Column List:")
col_names = [schema.name for schema in table.schema]
print(f"Total Columns: {len(col_names)}")
print(col_names[:10], "...")  # print only first 10 for readability

# 4️⃣ Check for Missing IDs
null_id_df = client.query(f"""
SELECT
  COUNT(*) AS total_rows,
  COUNTIF(extern_tuid IS NULL) AS null_ids,
  COUNT(DISTINCT extern_tuid) AS distinct_ids
FROM `{final_table}`
""").to_dataframe()
print("\n🔍 ID Integrity Check:")
display(null_id_df)

# 5️⃣ Quick Missing Value Summary (sample 20 cols for efficiency)
cols_sample = ', '.join([f"SUM(CASE WHEN `{c}` IS NULL THEN 1 ELSE 0 END) AS `{c}`" for c in col_names[:20]])
missing_sql = f"SELECT {cols_sample} FROM `{final_table}`"
missing_df = client.query(missing_sql).to_dataframe().T.reset_index()
missing_df.columns = ["column_name", "missing_count"]
missing_df["missing_pct"] = (missing_df["missing_count"] / table.num_rows * 100).round(2)
print("\n⚠️ Missing Value Summary (first 20 columns):")
display(missing_df.sort_values("missing_count", ascending=False))


In [ ]:
from google.cloud import bigquery

# Initialize BigQuery client
client = bigquery.Client(project="stgw-shared-ai-dev")

# Define project, dataset, and table names
project = "stgw-shared-ai-dev"
dataset = "parquet_data_set"
source_table = f"{project}.{dataset}.clustered_profiles_binned_44plus180"
target_table = f"{project}.{dataset}.clustered_profiles_binned_44plus180_ageparsed"

# SQL query for age parsing
sql = f"""
CREATE OR REPLACE TABLE `{target_table}` AS
SELECT
  *,
  -- Extract lower bound from "25 - 34" or "65+"
  SAFE_CAST(REGEXP_EXTRACT(age_imputed, r'(\\d+)\\s*[-+]?') AS INT64) AS age_lower,

  -- Extract upper bound from "25 - 34"
  SAFE_CAST(REGEXP_EXTRACT(age_imputed, r'-(\\d+)') AS INT64) AS age_upper,

  -- Generate readable label
  CASE
    WHEN REGEXP_CONTAINS(age_imputed, r'\\d+\\s*-\\s*\\d+') THEN REGEXP_EXTRACT(age_imputed, r'(\\d+\\s*-\\s*\\d+)')
    WHEN REGEXP_CONTAINS(age_imputed, r'\\d+\\s*\\+$') THEN CONCAT(REGEXP_EXTRACT(age_imputed, r'(\\d+)'), '+')
    ELSE 'Unknown'
  END AS age_group_label

FROM `{source_table}`;
"""

# Run the query
print("🚀 Creating parsed-age table in BigQuery ...")
job = client.query(sql)
job.result()
print(f"✅ Done! Created table: {target_table}")

In [ ]:
#1️⃣ Load only a random 2% sample from BigQuery
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client(project="stgw-shared-ai-dev")

project = "stgw-shared-ai-dev"
dataset = "parquet_data_set"
table = "clustered_profiles_binned_44plus180_ageparsed"

# Sample 2% of total rows randomly
SAMPLE_FRAC = 0.02

query = f"""
SELECT * EXCEPT(extern_tuid)
FROM `{project}.{dataset}.{table}`
WHERE RAND() < {SAMPLE_FRAC}
"""
print("🚀 Sampling 2% of BigQuery table...")
df_sample = client.query(query).to_dataframe()
print(f"✅ Sampled shape: {df_sample.shape}")


In [ ]:
# ✅ Save 2% sample to CSV for future use
output_path = "/content/cluster_sample_2pct.csv"
df_sample.to_csv(output_path, index=False)
print(f"📁 Saved sample to: {output_path}")


In [ ]:
#Load it back
import pandas as pd
df_sample = pd.read_csv("/content/cluster_sample_2pct.csv")
print(df_sample.shape)
df_sample.head()

In [ ]:
print(df_sample.columns.tolist())

In [ ]:
# 1️⃣ Demographic Columns (given)
DEMOGRAPHIC_COLS = [
    "gender_imputed", "age_imputed", "marital_status_imputed",
    "number_of_children_imputed", "generations_imputed",'age_lower', 'age_upper', 'age_group_label'

    # Geography / Location
    "urbanicity_imputed", "individual_state_imputed",
    "i_origin_dmas_imputed", "i_origin_cities_imputed",
    "i_origin_counties_imputed", "i_origin_zipcodes_imputed",
    "timezone_imputed", "csba_imputed",

    # Socio-economics
    "family_income_decile_imputed", "average_household_income_imputed",
    "wealth_rating_imputed", "net_worth_imputed", "home_equity_imputed",
    "home_ownership_imputed", "current_home_value_imputed",
    "length_of_residence_imputed", "dwelling_unit_size_imputed",
    "property_size_imputed", "number_of_bedrooms_imputed",
    "number_of_bathrooms_imputed",

    # Education & Employment
    "occupation_group_imputed", "employment_industry_imputed",
    "occupation_industry_imputed", "education_individual_imputed",

    # Sensitive identity traits
    "religion_imputed", "ethnicity_imputed", "language_spoken_imputed",
    "political_outlook_imputed", "political_affiliations_imputed",
]

# 2️⃣ ID / Technical Columns (exclude from behavioral)
ID_COLS = [
    "extern_tuid", "extern_tuid_imputed", "customer_id", "person_id",
    "record_id", "index", "row_id",

    # personal identifiers
    "first_name", "last_name", "full_name",
    "email", "phone_number", "address", "zip", "city",
    "latitude", "longitude",

    # timestamps & system fields
    "created_date", "created_date_imputed", "updated_at", "insert_ts"
]

# 3️⃣ Columns to ignore (imputation flags / raw count helper fields)
#NOISE_COLS = [c for c in df_sample.columns if "_impute" in c.lower() or "_flag" in c.lower()]

# 4️⃣ Identify Behavioral Columns (everything else)
BEHAVIORAL_COLS = [
    c for c in df_sample.columns
    if c not in DEMOGRAPHIC_COLS
    and c not in ID_COLS
    #and c not in NOISE_COLS
]

print(f"🧩 Total behavioral columns detected: {len(BEHAVIORAL_COLS)}")
print(f"🧩 Total demographic columns detected: {len(DEMOGRAPHIC_COLS)}")

In [ ]:
from nltk.stem import WordNetLemmatizer
import re

lemm = WordNetLemmatizer()

# Marketing-relevant noise
REMOVE_GENERIC_FINANCE = {
    "balance", "balances", "premium", "premiums", "policy", "policies",
    "claim", "claims", "loan", "loans", "mortgage", "ltv", "equity"
}

REMOVE_IRRELEVANT_MEDICAL = {
    "prescription", "medication", "medicine", "preventive"
}

NOISE_VALUES = { "", "nan", "none", "missing", "unknown", "null", "undefined" }

BANNED_TOKENS = {
    "hh", "acct", "id", "pkg", "pack",
    "000", "999", "12m", "24m", "3m"
}

RE_REMOVE_NUM = re.compile(r"^\d{1,4}$")
RE_DEVICE = re.compile(r"[a-zA-Z]+\d+|\d+[a-zA-Z]+")
RE_SPECIAL = re.compile(r"[^a-z0-9 ]+")
RE_SPACES = re.compile(r"\s+")

# keep important bigrams
PROTECTED_PHRASES = [
    "credit card", "streaming video", "online shopping", "car insurance",
    "satellite radio", "wireless carrier", "sports fan", "cable tv",
    "home improvement", "luxury travel", "organic food"
]

def clean_token(t):
    t = t.lower().strip()
    if not t or t in NOISE_VALUES: return ""
    if t in REMOVE_GENERIC_FINANCE | REMOVE_IRRELEVANT_MEDICAL: return ""
    if t in BANNED_TOKENS: return ""
    if RE_REMOVE_NUM.match(t): return ""
    if RE_DEVICE.match(t): return ""
    if len(t) <= 2: return ""
    return lemm.lemmatize(t)

def protect_phrases(text):
    text = text.lower()
    for p in PROTECTED_PHRASES:
        text = text.replace(p, p.replace(" ", "_"))
    return text

def clean_val(v):
    v = protect_phrases(str(v).lower())
    v = v.replace("|", " ").replace("/", " ").replace(",", " ")
    v = RE_SPECIAL.sub(" ", v)
    tokens = [clean_token(t) for t in v.split()]
    tokens = [t for t in tokens if t]
    return " ".join(tokens)

def build_behavior_text(row):
    toks = []
    for c in BEHAVIORAL_COLS:
        v = clean_val(row.get(c, ""))
        if v:
            toks.extend(v.split())

    # remove duplicates keeping order
    seen = set()
    uniq = []
    for t in toks:
        if t not in seen:
            seen.add(t)
            uniq.append(t)

    s = " ".join(uniq)
    s = RE_SPACES.sub(" ", s).strip()

    # revert protected phrases
    for p in PROTECTED_PHRASES:
        s = s.replace(p.replace(" ", "_"), p)

    return s


In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df_sample["behavior_text"] = df_sample.apply(build_behavior_text, axis=1)


In [ ]:
df_sample["behavior_text"].head(10).to_list()


In [ ]:
#Step 3 — Build Clean Behavior Text
df_sample["behavior_text"] = df_sample.apply(build_behavior_text, axis=1)
df_sample["behavior_text"] = df_sample["behavior_text"].str.replace(r'\s+', ' ', regex=True)

In [ ]:
#Behavior Embeddings
from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

embeddings = embedder.encode(
    df_sample["behavior_text"].tolist(),
    show_progress_bar=True,
    batch_size=256
)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/103 [00:00<?, ?it/s]

In [ ]:
#Determine Cluster Count
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score

scores = {}
for k in [8, 12, 16, 20, 25]:
    km = MiniBatchKMeans(n_clusters=k, batch_size=2048, random_state=42)
    lbls = km.fit_predict(embeddings)
    scores[k] = silhouette_score(embeddings, lbls)

print(scores)


{8: np.float32(0.121679306), 12: np.float32(0.112606384), 16: np.float32(0.11517501), 20: np.float32(0.088538416), 25: np.float32(0.07084282)}


In [ ]:
"""
End-to-end Persona Pipeline
- Cleans behavioral columns aggressively
- Builds TF-IDF + SBERT keyword summaries
- Reduces and combines features (PCA)
- Trains an Autoencoder, extracts latent Z
- Clusters latent space (MiniBatchKMeans)
- Produces cluster summaries and micro-personas
- Optionally uses Mistral (local transformers or API) to generate persona narratives

Instructions:
- Place /content/cluster_sample_2pct.csv (sample or small subset) in Colab workspace
- Adjust PARAMETERS below for your environment
- Install packages if needed: sentence-transformers, scikit-learn, tensorflow, transformers, joblib

Output artifacts in OUTPUT_DIR
"""

import os, re, json
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import Counter

# === ML / NLP ===
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
from sentence_transformers import SentenceTransformer
import joblib

# Autoencoder (TensorFlow/Keras)
import tensorflow as tf
from tensorflow import keras

# Optional LLM summarizer (Mistral) - local or API
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# ----------------------------
# ========== PARAMETERS ======
# ----------------------------
DATA_PATH = os.environ.get('DATA_PATH','/content/cluster_sample_2pct.csv')
OUTPUT_DIR = os.environ.get('OUTPUT_DIR','/content/persona_pipeline_output')
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Sampling (if you want to operate on smaller fraction)
SAMPLE_FRAC = float(os.environ.get('SAMPLE_FRAC', 1.0))  # 1.0 => use full file

# Models / dims
EMBED_MODEL = 'sentence-transformers/all-MiniLM-L12-v2'
TFIDF_MAX_FEATURES = 4000
TFIDF_MIN_DF = 5
PCA_DIM = 128
AE_LATENT_DIM = 64
K_CLUSTERS = 20

# Autoencoder training
EPOCHS = 30
BATCH = 512

# persona micro settings
MIN_PERSONA_SIZE = 40
SAMPLE_KEYWORDS_PER_PERSONA = 50

# LLM options
USE_LOCAL_LLM = False  # if True you must have model & HF transformers installed and available
USE_API_LLM = False    # if True, set API key env vars and endpoint
LLM_MAX_TOKENS = 300

# ----------------------------
# 1) Load data
# ----------------------------
print('Loading CSV...')
df = pd.read_csv(DATA_PATH, dtype=str).fillna('')
print('Shape:', df.shape)

if SAMPLE_FRAC < 1.0:
    df = df.sample(frac=SAMPLE_FRAC, random_state=42).reset_index(drop=True)
    print('Sampled shape:', df.shape)

# ----------------------------
# 2) Define columns (adjust to your dataset)
# ----------------------------
DEMOGRAPHIC_COLS = [
    "gender_imputed", "age_imputed", "age_group_label",
    "marital_status_imputed","number_of_children_imputed","generations_imputed",
    "urbanicity_imputed","individual_state_imputed","i_origin_dmas_imputed",
    "i_origin_cities_imputed","i_origin_counties_imputed","i_origin_zipcodes_imputed",
    "timezone_imputed","csba_imputed",
    "family_income_decile_imputed","average_household_income_imputed",
    "wealth_rating_imputed","net_worth_imputed","home_equity_imputed",
    "home_ownership_imputed","current_home_value_imputed",
    "length_of_residence_imputed","dwelling_unit_size_imputed",
    "property_size_imputed","number_of_bedrooms_imputed","number_of_bathrooms_imputed",
    "occupation_group_imputed","employment_industry_imputed","education_individual_imputed",
    "religion_imputed","ethnicity_imputed","language_spoken_imputed",
    "political_outlook_imputed","political_affiliations_imputed",
]

ID_COLS = [
    'extern_tuid','extern_tuid_imputed','firstName_imputed','lastName_imputed',
    'emailAddress_imputed','postCode_imputed'
]

# Make sure we only include columns that exist
ALL_COLS = list(df.columns)
DEM_COLS = [c for c in DEMOGRAPHIC_COLS if c in ALL_COLS]
ID_COLS_EXIST = [c for c in ID_COLS if c in ALL_COLS]

print('Demographic columns found:', len(DEM_COLS))
print('ID columns found:', len(ID_COLS_EXIST))

# Behavioral columns = everything else minus demo + id
BEHAVIORAL_COLS = [c for c in ALL_COLS if c not in DEM_COLS and c not in ID_COLS_EXIST]
print('Behavioral columns detected:', len(BEHAVIORAL_COLS))

# ----------------------------
# 3) Cleaning helpers
# ----------------------------
NOISE_VALUES = {"", "nan", "none", "__missing__", "missing", "unknown", "other", "null", "unspecified", "not available", "undefined"}
NOISE_VALUES = {x.lower() for x in NOISE_VALUES}
NOISE_TOKENS = {"hh","000","999","12m","24m","3m","30","lst","ppv","id","acct","na"}

RE_NUMERIC = re.compile(r'^\\d+$')
RE_CODELIKE = re.compile(r'^[a-z0-9]{2,8}$')
RE_SPECIALS = re.compile(r'[^a-z0-9 ]+')
RE_SPACES = re.compile(r'\\s+')

# semantic compress map: map many noisy brand words into short signals
SEMANTIC_MAP = [
    (re.compile(r'\\b(walmart|target|costco|sams club|aldi|amazon)\\b'), 'retail_shopper'),
    (re.compile(r'\\b(coupon|discount|deal|offer|loyalty|cents off)\\b'), 'value_seeker'),
    (re.compile(r'\\b(netflix|hulu|prime video|youtube|streaming)\\b'), 'streaming_media'),
    (re.compile(r'\\b(iphone|android|samsung|apple|motorola|htc|nexus)\\b'), 'tech_user'),
    (re.compile(r'\\b(cruise|airbnb|hotel|marriott|tripadvisor)\\b'), 'travel_interest'),
    (re.compile(r'\\b(child|children|kid|parent|family)\\b'), 'family_household'),
    (re.compile(r'\\b(pet|dog|cat)\\b'), 'pet_owner'),
]


def clean_token(tok):
    tok = tok.strip().lower()
    if not tok or tok in NOISE_VALUES:
        return ''
    if RE_NUMERIC.match(tok) and len(tok) <= 4:
        return ''
    if len(tok) <= 2:
        return ''
    if RE_CODELIKE.match(tok) and any(c.isdigit() for c in tok):
        return ''
    if tok in NOISE_TOKENS:
        return ''
    return tok


def clean_val_raw(v):
    v = str(v).lower()
    v = v.replace('|',' ').replace(',',' ').replace('/',' ')
    v = RE_SPECIALS.sub(' ', v)
    toks = [clean_token(t) for t in v.split()]
    toks = [t for t in toks if t]
    return ' '.join(toks)


def semantic_compress(text):
    if not text:
        return ''
    t = ' ' + text + ' '
    for pat, label in SEMANTIC_MAP:
        t = pat.sub(' ' + label + ' ', t)
    t = RE_SPACES.sub(' ', t).strip()
    seen = set(); out=[]
    for tok in t.split():
        if tok not in seen:
            seen.add(tok); out.append(tok)
    return ' '.join(out)

# ----------------------------
# 4) Build cleaned behavior text
# ----------------------------
print('Building cleaned behavior_text for each row...')

def build_behavior_text(row):
    toks = []
    for c in BEHAVIORAL_COLS:
        v = row.get(c, '')
        if not v:
            continue
        cv = clean_val_raw(v)
        if not cv:
            continue
        cv2 = semantic_compress(cv)
        if cv2:
            toks.append(cv2)
    full = ' '.join(toks)
    full = RE_SPACES.sub(' ', full).strip()
    # dedupe tokens in order
    seen = set(); out=[]
    for t in full.split():
        if t not in seen:
            seen.add(t); out.append(t)
    return ' '.join(out)

# apply with progress
pbar = tqdm(total=len(df), desc='clean rows')
rows = []
for i, r in df.iterrows():
    rows.append(build_behavior_text(r))
    pbar.update(1)
pbar.close()

df['behavior_text_clean'] = rows

# quick sanity
print('Example cleaned text (3 rows):')
print(df['behavior_text_clean'].head(3).tolist())

# ----------------------------
# 5)  vectorization
# ----------------------------
print('Running ...')
tfidf = TfidfVectorizer(max_features=TFIDF_MAX_FEATURES, min_df=TFIDF_MIN_DF, stop_words='english')
X_tfidf = tfidf.fit_transform(df['behavior_text_clean'].fillna(''))
print('TF-IDF shape:', X_tfidf.shape)
joblib.dump(tfidf, os.path.join(OUTPUT_DIR,'tfidf.joblib'))

# ----------------------------
# 6) Top keywords per row from TF-IDF
# ----------------------------
print('Extracting top TF-IDF keywords per row...')
features = np.array(tfidf.get_feature_names_out())

def top_keywords_from_tfidf_row(row_idx, top_k=20):
    row = X_tfidf[row_idx]
    if row.nnz == 0:
        return ''
    arr = np.array(row.todense()).ravel()
    ids = arr.argsort()[-top_k:][::-1]
    kws = [features[i] for i in ids if arr[i] > 0]
    return ' '.join(kws)

keywords = []
for i in tqdm(range(X_tfidf.shape[0]), desc='keywords'):
    keywords.append(top_keywords_from_tfidf_row(i, top_k=20))

df['behavior_keywords'] = keywords

# ----------------------------
# 7) SBERT embeddings of keyword summaries
# ----------------------------
print('Computing SBERT embeddings...')
embedder = SentenceTransformer(EMBED_MODEL)
embeddings = embedder.encode(df['behavior_keywords'].fillna('').tolist(), show_progress_bar=True, batch_size=64)
print('Embeddings shape:', embeddings.shape)
np.save(os.path.join(OUTPUT_DIR,'sbert_embeddings.npy'), embeddings)


print('Reducing  (TruncatedSVD)...')
if X_tfidf.shape[0] > 10000:
    svd = TruncatedSVD(n_components=128, random_state=42)
    tfidf_pca = svd.fit_transform(X_tfidf)
    joblib.dump(svd, os.path.join(OUTPUT_DIR,'svd_tfidf.joblib'))
else:
    pca_tfidf = PCA(n_components=min(128, X_tfidf.shape[1]-1), random_state=42)
    tfidf_pca = pca_tfidf.fit_transform(X_tfidf.todense())
    joblib.dump(pca_tfidf, os.path.join(OUTPUT_DIR,'pca_tfidf.joblib'))

print('tfidf_pca shape:', tfidf_pca.shape)

# combine: [embeddings | tfidf_pca]
X_combined = np.hstack([embeddings, tfidf_pca])
print('Combined feature shape:', X_combined.shape)

# scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_combined)
joblib.dump(scaler, os.path.join(OUTPUT_DIR,'scaler.joblib'))

# ----------------------------
# 9) PCA pre-reduction
# ----------------------------
print('PCA reduction to', PCA_DIM)
pca = PCA(n_components=min(PCA_DIM, X_scaled.shape[1]-1), random_state=42)
X_pca = pca.fit_transform(X_scaled)
joblib.dump(pca, os.path.join(OUTPUT_DIR,'pca_before_ae.joblib'))
print('X_pca shape:', X_pca.shape)

# ----------------------------
# 10) Autoencoder (Keras)
# ----------------------------
print('Building & training autoencoder...')

def build_autoencoder(input_dim, latent_dim=AE_LATENT_DIM):
    inp = keras.Input(shape=(input_dim,))
    x = keras.layers.Dense(256, activation='relu')(inp)
    x = keras.layers.Dense(128, activation='relu')(x)
    z = keras.layers.Dense(latent_dim, activation='linear', name='latent')(x)
    x = keras.layers.Dense(128, activation='relu')(z)
    x = keras.layers.Dense(256, activation='relu')(x)
    out = keras.layers.Dense(input_dim, activation='linear')(x)
    model = keras.Model(inputs=inp, outputs=out)
    encoder = keras.Model(inputs=inp, outputs=z)
    model.compile(optimizer=keras.optimizers.Adam(1e-3), loss='mse')
    return model, encoder

ae, encoder = build_autoencoder(X_pca.shape[1], AE_LATENT_DIM)

history = ae.fit(X_pca, X_pca, epochs=EPOCHS, batch_size=BATCH, validation_split=0.05, verbose=1)

ae.save(os.path.join(OUTPUT_DIR,'autoencoder.h5'))
encoder.save(os.path.join(OUTPUT_DIR,'encoder.h5'))

# ----------------------------
# 11) Latent Z and clustering
# ----------------------------
Z = encoder.predict(X_pca)
np.save(os.path.join(OUTPUT_DIR,'latent_Z.npy'), Z)

print('Clustering latent space...')
kmeans = MiniBatchKMeans(n_clusters=K_CLUSTERS, random_state=42, batch_size=2048, n_init=20)
cluster_labels = kmeans.fit_predict(Z)
df['behavior_cluster'] = cluster_labels
joblib.dump(kmeans, os.path.join(OUTPUT_DIR,'kmeans.joblib'))

# ----------------------------
# 12) Cluster summaries
# ----------------------------
print('Summarizing clusters...')
cluster_summaries = []
for c in range(K_CLUSTERS):
    idx = np.where(cluster_labels == c)[0]
    size = len(idx)
    if size == 0: continue
    texts = ' '.join(df.loc[idx,'behavior_keywords'].fillna('').tolist())
    common = [w for w,_ in Counter(texts.split()).most_common(20)]
    demo_modes = {}
    for d in DEM_COLS:
        if d in df.columns:
            vals = df.loc[idx,d].astype(str)
            demo_modes[d] = vals.mode().iloc[0] if len(vals)>0 else None
    cluster_summaries.append({'cluster':c,'size':size,'top_keywords':', '.join(common[:15]),'demo_modes':demo_modes})

cluster_summary_df = pd.DataFrame(cluster_summaries).sort_values('size', ascending=False)
cluster_summary_df.to_csv(os.path.join(OUTPUT_DIR,'cluster_summary.csv'), index=False)
print('Saved cluster_summary.csv')

# ----------------------------
# 13) Micro-personas
# ----------------------------
print('Generating micro-personas (split by gender/age/income where present)...')
SPLIT_COLS = [c for c in ['gender_imputed','age_group_label','family_income_decile_imputed'] if c in df.columns]
micro_personas = []
for c in range(K_CLUSTERS):
    dfi = df[df['behavior_cluster']==c]
    if len(dfi) < MIN_PERSONA_SIZE: continue
    groups = dfi.groupby(SPLIT_COLS) if SPLIT_COLS else [((), dfi)]
    for keys, grp in groups:
        if len(grp) < MIN_PERSONA_SIZE: continue
        examples = ' '.join(grp['behavior_keywords'].sample(min(SAMPLE_KEYWORDS_PER_PERSONA, len(grp)), random_state=42).tolist())
        micro_personas.append({
            'cluster':c,
            'cluster_size':len(dfi),
            'group_keys': keys,
            'group_size':len(grp),
            'top_terms_cluster': cluster_summary_df.loc[cluster_summary_df.cluster==c,'top_keywords'].values[0],
            'examples': examples
        })

micro_personas_df = pd.DataFrame(micro_personas)
micro_personas_df.to_csv(os.path.join(OUTPUT_DIR,'micro_personas_raw.csv'), index=False)
print('Saved micro_personas_raw.csv')

# ----------------------------
# 14) Persona narrative generation (optional)
# ----------------------------
# You can plug either a local LLM (transformers) or an API call.
# For brevity we include a simple template and two example hooks.

PERSONA_PROMPT_TEMPLATE = '''
You are a consumer insights strategist. Produce a concise persona (4-6 sentences) in natural human tone.
Cluster top behaviors: {top_terms}
Representative signals (keywords): {examples}
Group dominant demos: {demos}
Output: title + 1-line tagline, then 3-4 sentence narrative: motivations, digital & purchase behavior, preferred messaging style.
Do not list brand names. Avoid political or health claims.
'''


def generate_persona_local(top_terms, examples, demos):
    # Example: use a local model with transformers (if available)
    # Ensure USE_LOCAL_LLM=True and model installed
    from transformers import pipeline
    summarizer = pipeline('text-generation', model='mistralai/mistral-7b-instruct', device=-1)
    prompt = PERSONA_PROMPT_TEMPLATE.format(top_terms=top_terms, examples=examples[:1200], demos=json.dumps(demos))
    out = summarizer(prompt, max_new_tokens=LLM_MAX_TOKENS)
    return out[0]['generated_text']


def generate_persona_api(top_terms, examples, demos):
    # Placeholder for API call (OpenAI, OpenRouter, etc.). Configure env vars.
    KEY = os.environ.get('LLM_API_KEY')
    ENDPOINT = os.environ.get('LLM_API_ENDPOINT')
    if not KEY or not ENDPOINT:
        return ''
    payload = {
        'model': 'mistralai/mistral-7b-instruct',
        'messages':[{'role':'user','content': PERSONA_PROMPT_TEMPLATE.format(top_terms=top_terms, examples=examples[:1200], demos=json.dumps(demos))}],
        'max_tokens': LLM_MAX_TOKENS
    }
    import requests
    r = requests.post(ENDPOINT, headers={'Authorization': f'Bearer {KEY}'}, json=payload)
    return r.json()['choices'][0]['message']['content']

# Choose generator based on flags
if USE_LOCAL_LLM or USE_API_LLM:
    persona_texts = []
    for _, row in micro_personas_df.iterrows():
        top_terms = row['top_terms_cluster']
        examples = row['examples']
        demos = row['group_keys']
        if USE_LOCAL_LLM:
            txt = generate_persona_local(top_terms, examples, demos)
        else:
            txt = generate_persona_api(top_terms, examples, demos)
        persona_texts.append(txt)
    micro_personas_df['persona_summary'] = persona_texts
    micro_personas_df.to_csv(os.path.join(OUTPUT_DIR,'micro_personas_with_narratives.csv'), index=False)
    print('Saved micro_personas_with_narratives.csv')
else:
    print('LLM step skipped. Set USE_LOCAL_LLM or USE_API_LLM to True to generate narratives.')

# ----------------------------
# 15) Save final artifacts and small health checks
# ----------------------------
print('Saving artifacts...')
df.to_csv(os.path.join(OUTPUT_DIR,'df_with_clusters.csv'), index=False)
cluster_summary_df.to_csv(os.path.join(OUTPUT_DIR,'cluster_summary.csv'), index=False)

print('Done. Artifacts in', OUTPUT_DIR)
print('Top clusters:')
print(cluster_summary_df.head())


In [ ]:
# Save sample (already in memory as df)
out_path = "/content/persona_pipeline_output/cluster_sample_2pct_with_clusters.csv"
df.to_csv(out_path, index=False)
print("Saved sample with clusters:", out_path)


Saved sample with clusters: /content/persona_pipeline_output/cluster_sample_2pct_with_clusters.csv


In [ ]:
import pandas as pd
cs = "/content/persona_pipeline_output/cluster_summary.csv"
mp = "/content/persona_pipeline_output/micro_personas_raw.csv"
print("cluster_summary exists:", pd.read_csv(cs).shape if os.path.exists(cs) else "missing")
print("micro_personas exists:", pd.read_csv(mp).shape if os.path.exists(mp) else "missing")

# Load and preview cluster_summary
cluster_summary = pd.read_csv(cs)
cluster_summary.head(10)


In [ ]:
from sklearn.metrics import silhouette_score
import joblib, numpy as np

# load latent Z and kmeans
Z = np.load("/content/persona_pipeline_output/latent_Z.npy")  # if saved
kmeans = joblib.load("/content/persona_pipeline_output/kmeans.joblib")
labels = kmeans.labels_ if hasattr(kmeans, "labels_") else kmeans.predict(Z)

# sizes
import pandas as pd
sizes = pd.Series(labels).value_counts().sort_index()
print("Cluster sizes:\n", sizes)

# silhouette (only if K>1 and Z available)
if len(np.unique(labels))>1:
    s = silhouette_score(Z, labels, metric='euclidean')
    print("Silhouette score (latent space):", round(s, 4))
else:
    print("Need >1 cluster to compute silhouette")


In [ ]:
import re, itertools, pandas as pd
mp = pd.read_csv("/content/persona_pipeline_output/micro_personas_raw.csv")

# simple cleanup function
RE_SPECIALS = re.compile(r"[^a-z0-9 ]+")
def clean_examples(text, keep_top_n=150):
    text = str(text).lower()
    # remove punctuation, short tokens, numeric noise
    text = RE_SPECIALS.sub(" ", text)
    toks = [t for t in text.split() if len(t)>2 and not t.isdigit()]
    # remove repeated tokens and stopwords lightly (you can extend)
    seen=set(); out=[]
    for t in toks:
        if t not in seen and t not in {"hh","12m","000","lst","na","other"}:
            seen.add(t); out.append(t)
    return " ".join(out[:keep_top_n])

mp["examples_clean"] = mp["examples"].fillna("").apply(clean_examples)
mp[["cluster","group_keys","group_size","examples_clean"]].head(5)


chat_model = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2")


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "mistralai/Mistral-7B-Instruct-v0.2"
cache_dir = "/content/models"

tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir)
model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir=cache_dir)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
# ====== Persona generation (end-to-end) - FIXED CODE ======
# Drop into Colab where your pipeline artifacts are saved:
# - /content/persona_pipeline_output/cluster_summary.csv
# - /content/persona_pipeline_output/micro_personas_raw.csv  (optional)
# - /content/persona_pipeline_output/latent_Z.npy
# - df (the full dataframe) must be loaded or produced earlier in the notebook
# Replace DATA_DIR if you use another path.

import os, math, random, json, re
import pandas as pd
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import joblib

# ----------------------------
# CONFIG
# ----------------------------
DATA_DIR = "/content/persona_pipeline_output"
OUTPUT_FILE = os.path.join(DATA_DIR, "micro_personas_final_expanded.csv")

# Clustering / persona selection params
K_CLUSTERS = 20                       # keep in sync with how you created clusters
MIN_GROUP_PCT = 0.02                  # min fraction of cluster to form a persona group (2%)
MIN_PERSONAS_PER_CLUSTER = 3
MAX_PERSONAS_PER_CLUSTER = 10
SAMPLE_KEYWORDS_PER_PERSONA = 100     # how many behavior keywords to include in prompt (sampleed)
MAX_NEW_TOKENS = 800                  # REQUEST LONGER OUTPUT to avoid truncation
TEMPERATURE = 0.35
TOP_P = 0.9

# Demographic fields to use for persona identity and to include in persona table
DEM_FIELDS = [
    "gender_imputed", "age_imputed", "age_group_label",
    "marital_status_imputed","number_of_children_imputed","generations_imputed",
    "urbanicity_imputed","individual_state_imputed","i_origin_dmas_imputed",
    "i_origin_cities_imputed","i_origin_counties_imputed","i_origin_zipcodes_imputed",
    "timezone_imputed","csba_imputed",
    "family_income_decile_imputed","average_household_income_imputed",
    "wealth_rating_imputed","net_worth_imputed","home_equity_imputed",
    "home_ownership_imputed","current_home_value_imputed",
    "length_of_residence_imputed","dwelling_unit_size_imputed",
    "property_size_imputed","number_of_bedrooms_imputed","number_of_bathrooms_imputed",
    "occupation_group_imputed","employment_industry_imputed","education_individual_imputed",
    "religion_imputed","ethnicity_imputed","language_spoken_imputed",
    "political_outlook_imputed","political_affiliations_imputed"
]

# We WILL include these fields in the final CSV, but we do NOT feed political/religion into the prompt to avoid bias/hallucination:
PROMPT_IGNORE_FIELDS = {"religion_imputed", "political_outlook_imputed", "political_affiliations_imputed"}

# Mistral model spec (local cached)
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
CACHE_DIR = "/content/models"  # your HF cache location

# ----------------------------
# Load data artifacts
# ----------------------------
# You must have 'df' loaded (the dataset with behavior_cluster & behavior_keywords fields).
# Try to load df_with_clusters.csv if present; otherwise assume df is present in current notebook.
df_path = os.path.join(DATA_DIR, "df_with_clusters.csv")
if os.path.exists(df_path):
    df = pd.read_csv(df_path, dtype=str).fillna("")
    print("Loaded df from", df_path)
else:
    # Fall back to memory if 'df' exists in notebook (not possible in a run here)
    try:
        df  # noqa
    except NameError:
        raise FileNotFoundError(f"Couldn't find {df_path}. Please load your dataframe into 'df' before running.")

# Load cluster_summary
cluster_summary_path = os.path.join(DATA_DIR, "cluster_summary.csv")
if not os.path.exists(cluster_summary_path):
    raise FileNotFoundError(f"Missing cluster_summary.csv at {cluster_summary_path}")
cluster_summary_df = pd.read_csv(cluster_summary_path, dtype=str).fillna("")


# ----------------------------
# Prepare persona seed groups
# ----------------------------
mp_rows = []

for c in range(K_CLUSTERS):
    dfi = df[df["behavior_cluster"].astype(int) == int(c)]
    if dfi.empty:
        continue

    cluster_size = len(dfi)
    min_count = math.ceil(cluster_size * MIN_GROUP_PCT)

    # Demographics used to slice persona groups (core identity drivers)
    SPLIT_COLS = [
        "gender_imputed","age_group_label","marital_status_imputed",
        "number_of_children_imputed","urbanicity_imputed",
        "family_income_decile_imputed","education_individual_imputed",
        "occupation_group_imputed","language_spoken_imputed","ethnicity_imputed"
    ]

    SPLIT_COLS = [c_ for c_ in SPLIT_COLS if c_ in dfi.columns]

    groups = list(dfi.groupby(SPLIT_COLS, dropna=False)) if SPLIT_COLS else [(("all",), dfi)]

    valid = [(k,g) for k,g in groups if len(g) >= min_count]

    if len(valid) < MIN_PERSONAS_PER_CLUSTER:
        valid = sorted(groups, key=lambda x: len(x[1]), reverse=True)[:MIN_PERSONAS_PER_CLUSTER]
    else:
        valid = sorted(valid, key=lambda x: len(x[1]), reverse=True)[:MAX_PERSONAS_PER_CLUSTER]

    cluster_row = cluster_summary_df[cluster_summary_df['cluster'].astype(int) == int(c)]
    cluster_brief = cluster_row['top_keywords'].iloc[0] if not cluster_row.empty else ""

    # Build persona seeds
    for _, grp in valid:
        demo_dict = {}
        for f in DEM_FIELDS:
            if f in grp:
                vals = grp[f].dropna().astype(str)
                demo_dict[f] = vals.mode().iloc[0] if not vals.mode().empty else ""
            else:
                demo_dict[f] = ""

        # sample behavioral keywords
        kw = grp["behavior_keywords"].dropna().tolist()
        examples = " ".join(random.sample(kw, min(len(kw), SAMPLE_KEYWORDS_PER_PERSONA))) if kw else ""

        mp_rows.append({
            "cluster": int(c),
            "cluster_name": f"Cluster {c}",
            "cluster_brief": cluster_brief,
            "group_size": len(grp),
            "demographics": demo_dict,
            "examples": examples
        })

mp_df = pd.DataFrame(mp_rows)
print("✅ Persona seed groups prepared:", mp_df.shape)


# ----------------------------
# Load Mistral model & tokenizer (local)
# ----------------------------
print("Loading Mistral model/tokenizer (local)...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=CACHE_DIR, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, cache_dir=CACHE_DIR, trust_remote_code=True)
device = 0 if (hasattr(model, "device") or (os.environ.get("CUDA_VISIBLE_DEVICES") != "")) else -1

text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=MAX_NEW_TOKENS,
    temperature=TEMPERATURE,
    top_p=TOP_P,
    pad_token_id=tokenizer.eos_token_id,
    device=device
)

# ----------------------------
# Prompt builder (strictly grounded)
# ----------------------------
PROMPT_TEMPLATE = """
You are a senior consumer insights strategist.

Goal: produce ONE highly realistic micro-persona strictly grounded in the data below.
Do NOT invent new facts, brands, or cities — only use what is provided.

CLUSTER:
{cluster_name}
BRIEF BEHAVIOR PATTERN:
{cluster_brief}

DEMOGRAPHIC SNAPSHOT:
{demo_lines}

OBSERVED SIGNALS (sampled from dataset):
{examples}

OUTPUT: produce a structured persona containing:
- Persona Title + short tagline (single line)
- Short bio (4-6 natural sentences, grounded in cluster behavior & demo)
- Motivations & everyday life (2-4 bullets or sentences)
- Digital & media habits (1-3 bullets)
- Purchase & loyalty drivers (1-3 bullets)
- Tone & communication preferences (1 line)

End the persona with the marker: <END_OF_PERSONA>
"""

def build_prompt_from_row(row):
    demo = row["demographics"]
    # include demographic lines but exclude political / religion in the prompt text
    demo_lines = []
    for k, v in demo.items():
        if k in PROMPT_IGNORE_FIELDS:
            continue
        if v is None or v == "" or str(v).lower() in {"nan","none",""}:
            continue
        demo_lines.append(f"- {k}: {v}")
    demo_lines = "\n".join(demo_lines) if demo_lines else "- (no explicit demo fields provided)"

    return PROMPT_TEMPLATE.format(
        cluster_name=row["cluster_name"],
        cluster_brief=row["cluster_brief"],
        demo_lines=demo_lines,
        examples=(row["examples"][:6000] if row["examples"] else "")
    )

# ----------------------------
# Generate persona text for each seed group
# ----------------------------
generated_personas = []
for i, r in mp_df.iterrows():
    prompt = build_prompt_from_row(r)
    # Wrap in instruction tags used by Mistral-like models
    instruction = f"<s>[INST]{prompt}[/INST]"
    try:
        out = text_gen(instruction)[0]["generated_text"]
    except Exception as e:
        out = f"ERROR: {e}"

    # --- START OF FIX: Clean the LLM output ---

    clean_out = out
    # 1. Look for the closing instruction tag [/INST] and start reading *after* it.
    # This correctly ignores the model's echo of the input prompt/instruction.
    match = re.search(r'\[/INST\]\s*(.*)', out, re.DOTALL)
    if match:
        clean_out = match.group(1).strip()
    # 2. If the initial clean-up fails (e.g., model removes the wrapper),
    # check if the output starts with the full instruction string, and cut it.
    elif out.startswith(instruction):
        clean_out = out[len(instruction):].strip()

    # 3. Extract everything before the end marker <END_OF_PERSONA>
    persona_text = clean_out.split("<END_OF_PERSONA>")[0].strip()

    # --- END OF FIX ---

    generated_personas.append(persona_text)

mp_df["persona_text"] = generated_personas

# ----------------------------
# Post-process: extract a title/tagline if present + flatten demo dict into columns
# ----------------------------
def extract_title_tagline(persona_text):
    # heuristic: first non-empty line that is short (<=120 chars) or starts with "Title:" etc.
    lines = [ln.strip() for ln in persona_text.splitlines() if ln.strip()]
    if not lines:
        return "", persona_text
    # prefer lines that look like title (contain ':' or ' - ' or up to 8 words)
    first = lines[0]
    if len(first.split()) <= 12 and len(first) < 200:
        title = first
        rest = "\n".join(lines[1:])
    else:
        # try to find explicit "Persona Title" style
        for ln in lines[:3]:
            if ":" in ln or "-" in ln:
                title = ln
                idx = lines.index(ln)
                rest = "\n".join(lines[idx+1:])
                break
        else:
            # fallback
            title = lines[0][:120]
            rest = "\n".join(lines[1:])
    return title, "\n".join(lines) # Return the entire cleaned text as full_text for better summary column

flat_rows = []
for idx, r in mp_df.iterrows():
    demo = r["demographics"]
    title, full_text = extract_title_tagline(r["persona_text"])
    persona_id = f"{int(r['cluster'])}_{idx}"
    # populate flat row including all DEM_FIELDS as columns
    flat = {
        "cluster_id": r["cluster"],
        "cluster_name": r["cluster_name"],
        "cluster_brief": r["cluster_brief"],
        "group_size": r["group_size"],
        "persona_id": persona_id,
        "persona_title": title,
        "persona_summary": full_text
    }
    # expand demographics
    for f in DEM_FIELDS:
        flat[f] = demo.get(f, "") if isinstance(demo, dict) else ""

    flat_rows.append(flat)

final_df = pd.DataFrame(flat_rows)

# Optional: add readable persona_name (from title if present otherwise generate)
def make_persona_name(title, persona_id):
    if title and len(title) > 3:
        # take first few words of the title as persona name
        name = re.sub(r'[^\w\s-]', '', title).strip()
        name = name.split("\n")[0]
        return name[:80]
    else:
        return f"Persona_{persona_id}"

final_df["persona_name"] = final_df.apply(lambda r: make_persona_name(r["persona_title"], r["persona_id"]), axis=1)

# Reorder columns for readability
cols_order = [
    "cluster_id","cluster_name","cluster_brief","group_size",
    "persona_id","persona_name","persona_title"
] + DEM_FIELDS + ["persona_summary"]

final_df = final_df[cols_order]

# Save to CSV
final_df.to_csv(OUTPUT_FILE, index=False)
print("Saved final personas to:", OUTPUT_FILE)
print("Final table shape:", final_df.shape)

Loaded df from /content/persona_pipeline_output/df_with_clusters.csv
✅ Persona seed groups prepared: (60, 6)
Loading Mistral model/tokenizer (local)...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cpu


Saved final personas to: /content/persona_pipeline_output/micro_personas_final_expanded.csv
Final table shape: (60, 42)


In [ ]:
df_micro_persona=pd.read_csv("/content/persona_pipeline_output/micro_personas_final_expanded.csv")
df_micro_persona.head()

# EDA on Data

In [ ]:
import pandas as pd
import os

# Define the expected path and load the DataFrame
DATA_DIR = "/content/persona_pipeline_output"
OUTPUT_FILE = os.path.join(DATA_DIR, "micro_personas_final_expanded.csv")

try:
    df_persona = pd.read_csv(OUTPUT_FILE)
    print(f"✅ DataFrame loaded successfully. Shape: {df_persona.shape}\n")

    # --- 1. Top 5 Clusters by Persona Group Size ---
    # This tells us which behavioral segments (clusters) account for the most users.
    cluster_group_size = df_persona.groupby(['cluster_id', 'cluster_brief']).agg(
        Total_Group_Size=('group_size', 'sum'),
        Persona_Count=('persona_id', 'count')
    ).reset_index().sort_values(by='Total_Group_Size', ascending=False)

    print("## 📊 Top 5 Clusters by Total Population Size")
    print("---")
    print(cluster_group_size.head(5).to_markdown(index=False, numalign="left", stralign="left"))
    print("\n")

    # --- 2. Distribution of Key Demographic Fields ---
    # Analyze the most common demographic traits found in the *mode* of the generated personas.

    print("## 👤 Distribution of Key Demographic Traits (across all 60 personas)")
    print("---")

    # Age Group Distribution
    age_dist = df_persona['age_group_label'].value_counts(normalize=True).mul(100).round(1).reset_index()
    age_dist.columns = ['Age Group', 'Percentage (%)']
    print("\nAge Group Distribution:")
    print(age_dist.to_markdown(index=False, numalign="left", stralign="left"))

    # Gender Distribution
    gender_dist = df_persona['gender_imputed'].value_counts(normalize=True).mul(100).round(1).reset_index()
    gender_dist.columns = ['Gender', 'Percentage (%)']
    print("\nGender Distribution:")
    print(gender_dist.to_markdown(index=False, numalign="left", stralign="left"))


    # --- 3. Analysis of Persona Summary Length ---
    # Checking the quality/completeness of the LLM output (should be > 100 characters)

    df_persona['summary_length'] = df_persona['persona_summary'].astype(str).str.len()

    print("\n## 📝 Persona Summary Length Statistics (Quality Check)")
    print("---")
    print(df_persona['summary_length'].describe().to_markdown(numalign="left", stralign="left"))


    # --- 4. Cross-Tabulation: Cluster vs. Age Group ---
    # Show the demographic profile of the top clusters.
    top_cluster_ids = cluster_group_size['cluster_id'].head(3).tolist()
    df_top_clusters = df_persona[df_persona['cluster_id'].isin(top_cluster_ids)]

    print(f"\n## 📈 Cross-Tab: Age Distribution within Top Clusters ({top_cluster_ids})")
    print("---")
    cross_tab = pd.crosstab(df_top_clusters['cluster_id'], df_top_clusters['age_group_label'])
    print(cross_tab.to_markdown(numalign="left", stralign="left"))


except FileNotFoundError:
    print(f"Error: The file {OUTPUT_FILE} was not found. Please ensure the full persona generation script ran successfully.")
except Exception as e:
    print(f"An error occurred during EDA: {e}")

In [ ]:
import pandas as pd
import os
import json
import time
import hashlib
from typing import Dict, List

# ----- CATEGORY-wise name pools -----
CLUSTER_VARIANTS = {
    "FINANCE": [
        "High Net Worth Capitalists",
        "Elite Portfolio Managers",
        "Strategic Wealth Administrators",
        "Affluent Asset Maximizers",
        "Premium Wealth Strategists",
        "Capital Growth Connoisseurs"
    ],
    "RISK": [
        "Prudent Financial Risk Planners",
        "Active Coverage Strategists",
        "Risk Mitigation Specialists",
        "Future-Security Architects",
        "Life Contingency Preparers",
        "Protection-First Evaluators"
    ],
    "TECH": [
        "Techno-Savvy Digital Explorers",
        "Digital Media Innovators",
        "Connected Device Enthusiasts",
        "High-Bandwidth Experience Seekers",
        "Emerging Tech Pioneers",
        "Always-Online Streamers"
    ],
    "HOUSEHOLD": [
        "Core Household Value Shoppers",
        "Budget-Conscious Family Planners",
        "Practical Home Economists",
        "Home Basics Loyal Buyers",
        "Routine Necessity Seekers",
        "Daily-Essentials Optimizers"
    ],
    "LIFESTYLE": [
        "Discerning Social Lifestyle Spenders",
        "Curated Experience Seekers",
        "Status-Signaling Indulgence Buyers",
        "Premium Taste Enthusiasts",
        "Gourmet & Leisure Connoisseurs",
        "Occasion-Driven Elite Purchasers"
    ],
    "PASSIVE": [
        "Sluggish Non-Participative Spenders",
        "Familiar Brand Loyalists",
        "Routine-Bound Minimal Engagers",
        "Low-Stimulus Conservative Consumers",
        "Habit-Driven Value Holders",
        "Inertia-Dominated Buyers"
    ],
    "CONTENT": [
        "Intellectual Content Bureaucrats",
        "Information-Driven Planners",
        "Long-Form Knowledge Seekers",
        "Analytical Media Consumers",
        "Deep Research Habituals",
        "High-Depth Content Followers"
    ]
}

# ----- GLOBAL MEMORY TO PREVENT DUPLICATES -----
USED_NAMES = set()

# ----- CATEGORY KEYWORDS -----
CATEGORY_KEYWORDS = {
    "FINANCE": ["finance", "stocks", "wealth", "ltv", "credit", "investment"],
    "RISK": ["insurance", "claim", "risk", "coverage", "policy", "usaa"],
    "TECH": ["tech", "streaming", "media", "gaming", "device", "broadband"],
    "HOUSEHOLD": ["household", "family", "care", "home", "grocery", "bulk"],
    "LIFESTYLE": ["gourmet", "alcohol", "wine", "whisky", "heineken", "luxury"],
    "CONTENT": ["reading", "books", "magazines", "content", "news", "educational"]
}


def detect_category(brief: str) -> str:
    brief = brief.lower()
    for cat, kws in CATEGORY_KEYWORDS.items():
        if any(k in brief for k in kws):
            return cat
    return "PASSIVE"


def generate_unique_name(category: str, cluster_brief: str) -> str:
    variants = CLUSTER_VARIANTS[category]

    # hash-based deterministic index
    idx = int(hashlib.sha256(cluster_brief.encode()).hexdigest(), 16) % len(variants)

    base_name = variants[idx]

    # if name reused, append unique suffix
    if base_name in USED_NAMES:
        alt = f"{base_name} {abs(hash(cluster_brief)) % 999}"
        USED_NAMES.add(alt)
        return alt

    USED_NAMES.add(base_name)
    return base_name


def generate_description(category: str, brief: str) -> str:
    kws = ", ".join(list({w.strip().title() for w in brief.split(',')[:3]}))

    DESCRIPTIONS = {
        "FINANCE": "This affluent segment actively manages assets and pursues financial growth, seeking premium advisory and wealth solutions.",
        "RISK": "These individuals prioritize protection and future stability, engaging deeply with coverage and security products.",
        "TECH": "This digitally-native audience embraces innovation, streaming and advanced technology ecosystems.",
        "HOUSEHOLD": "Value-driven buyers prioritizing dependable essentials, routine purchasing and family-oriented needs.",
        "LIFESTYLE": "Experience-driven high spenders who value premium brands, indulgence, and elevated taste.",
        "CONTENT": "Information-hungry individuals who consume deep, high-quality content and research heavily.",
        "PASSIVE": "Low-activity consumers who rely on habitual, familiar purchase patterns and rarely explore alternatives."
    }

    desc = DESCRIPTIONS.get(category, DESCRIPTIONS["PASSIVE"])
    return f"{desc} Key behavioral markers include interest in: {kws}."


def generate_cluster_info(brief: str) -> Dict[str, str]:
    category = detect_category(brief)
    name = generate_unique_name(category, brief)
    desc = generate_description(category, brief)
    return {"name": name, "description": desc, "original_brief": brief}


In [ ]:
# ------------------- MAIN PIPELINE -------------------

def generate_and_update_cluster_names(file_path: str) -> pd.DataFrame:
    """Loads data, generates unique cluster names/descriptions, saves results."""

    if not os.path.exists(file_path):
        print(f"❌ File not found: {file_path}")
        return pd.DataFrame()

    df = pd.read_csv(file_path)
    if "cluster_id" not in df.columns or "cluster_brief" not in df.columns:
        raise ValueError("CSV must contain columns: 'cluster_id', 'cluster_brief'")

    unique_clusters = (
        df[['cluster_id', 'cluster_brief']]
        .drop_duplicates()
        .sort_values(by='cluster_id')
    )

    print(f"🚀 Generating distinct cluster names and summaries for {len(unique_clusters)} clusters...")

    cluster_map = {}
    for _, row in unique_clusters.iterrows():
        cid, brief = row["cluster_id"], row["cluster_brief"]
        info = generate_cluster_info(brief)
        cluster_map[cid] = info
        print(f"✅ Cluster {cid}: {info['name']}")

    # Update dataframe
    df["cluster_name"] = df["cluster_id"].map(lambda x: cluster_map[x]["name"])
    df["cluster_description"] = df["cluster_id"].map(lambda x: cluster_map[x]["description"])

    # Save back
    df.to_csv(file_path, index=False)
    print(f"\n💾 Saved updated cluster names + descriptions to: {file_path}")

    # Summary output
    summary = pd.DataFrame([
        {
            "cluster_id": cid,
            "cluster_name": info["name"],
            "cluster_description": info["description"]
        }
        for cid, info in cluster_map.items()
    ])

    return summary


# ------------------- EXECUTE -------------------

if __name__ == "__main__":
    DATA_DIR = "/content/persona_pipeline_output"
    FILE = os.path.join(DATA_DIR, "micro_personas_final_expanded.csv")

    result_df = generate_and_update_cluster_names(FILE)

    if not result_df.empty:
        print("\n📌 **New Cluster Mapping Preview**")
        try:
            from tabulate import tabulate
            print(tabulate(result_df, headers='keys', tablefmt='github'))
        except:
            print(result_df.head())


In [ ]:
df_micro_persona=pd.read_csv("/content/persona_pipeline_output/micro_personas_final_expanded.csv")
df_micro_persona.head()

In [ ]:
df = pd.read_csv("/content/persona_pipeline_output/micro_personas_final_expanded.csv")

# Desired order start
cols_start = ["cluster_id", "cluster_name", "cluster_brief", "cluster_description"]

# Create the new column order
cols = cols_start + [c for c in df.columns if c not in cols_start]

df = df[cols]

df.to_csv("/content/persona_pipeline_output/micro_personas_final_expanded.csv", index=False)

df.head()


In [ ]:
#Name the Personas
import pandas as pd
import hashlib

FILE = "/content/persona_pipeline_output/micro_personas_final_expanded.csv"
df = pd.read_csv(FILE)

# --- Name pools ---
MALE_NAMES = [
    "Arjun", "Rohan", "Vikram", "Amit", "Sameer", "Karan", "Rahul", "Siddharth",
    "Ankit", "Pranav", "Ravi", "Manish", "Saurabh", "Vivek", "Nikhil", "Aaditya",
    "Sunny", "Rupesh", "Tushar", "Dev"
]
FEMALE_NAMES = [
    "Ananya", "Priya", "Deepa", "Sonal", "Aisha", "Isha", "Neha", "Pooja",
    "Kavya", "Ritika", "Swati", "Shreya", "Meera", "Divya", "Nisha", "Tanvi",
    "Sana", "Riya", "Maya", "Asha"
]
NEUTRAL_NAMES = [
    "Sam", "Alex", "Taylor", "Riley", "Jordan", "Casey", "Avery", "Robin",
    "Morgan", "Jamie"
]

def deterministic_index(seed: str, mod: int) -> int:
    """Return deterministic index from seed string."""
    h = hashlib.sha256(seed.encode("utf-8")).hexdigest()
    return int(h, 16) % mod

def pick_name(row):
    gender = str(row.get("gender_imputed", "")).strip().lower()
    extern = str(row.get("extern_tuid", "")).strip()

    seed = f"{row.get('cluster_id','')}_{extern or row.name}"

    if gender == "male":
        idx = deterministic_index(seed + "_m", len(MALE_NAMES))
        return MALE_NAMES[idx]
    elif gender == "female":
        idx = deterministic_index(seed + "_f", len(FEMALE_NAMES))
        return FEMALE_NAMES[idx]
    else:
        idx = deterministic_index(seed + "_n", len(NEUTRAL_NAMES))
        return NEUTRAL_NAMES[idx]

# Assign persona_name
df["persona_name"] = df.apply(pick_name, axis=1)

# Reorder columns: cluster_id, cluster_name, cluster_brief, cluster_description, persona_name, then rest
cols_start = ["cluster_id", "cluster_name", "cluster_brief", "cluster_description", "persona_name"]
cols_start = [c for c in cols_start if c in df.columns]
remaining = [c for c in df.columns if c not in cols_start]
df = df[cols_start + remaining]

# Save
df.to_csv(FILE, index=False)
print(f"✅ Updated CSV saved: {FILE}")
df[cols_start].head(10)


In [ ]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import os
import json, re
import hashlib

# ---------------- LLM CONFIG ----------------
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"
CACHE_DIR = "/content/models"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=CACHE_DIR)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, cache_dir=CACHE_DIR)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=150)

# ---------------- PROMPT FUNCTION ----------------
def generate_cluster_name_description(persona_texts: list, existing_names: set) -> dict:
    """
    Ask the LLM to produce a cluster name and description, ensuring the name is unique.
    """
    aggregated_text = "\n".join(persona_texts)
    prompt = (
        f"You are a Senior Marketing Strategist.\n"
        f"Given the following persona summaries from a cluster:\n{aggregated_text}\n\n"
        f"Generate a professional cluster name (max 4 words) and a 1-2 sentence description "
        f"summarizing the common characteristics of this cluster. "
        f"Output as JSON with keys 'name' and 'description'."
    )

    output = generator(prompt, max_new_tokens=150)[0]["generated_text"]

    # Extract JSON from output (if LLM outputs extra text)
    match = re.search(r'\{.*\}', output, re.DOTALL)
    try:
        llm_out = json.loads(match.group()) if match else {"name": "", "description": ""}
    except:
        llm_out = {"name": "", "description": ""}

    name = llm_out.get("name", "").strip() or "Generic Cluster"
    description = llm_out.get("description", "").strip() or "Summary could not be generated."

    # Ensure uniqueness
    if name in existing_names or name.lower() in ["generic cluster", "cluster"]:
        # append short hash from persona_texts
        short_hash = str(abs(hash(" ".join(persona_texts))) % 1000)
        name = f"{name} {short_hash}"

    existing_names.add(name)
    return {"name": name, "description": description}

# ---------------- MAIN PIPELINE ----------------
def generate_cluster_llm(file_path: str, output_file: str) -> pd.DataFrame:
    df = pd.read_csv(file_path)
    cluster_info = {}
    existing_names = set()

    for cid, grp in df.groupby("cluster_id"):
        persona_texts = grp["persona_summary"].tolist()
        llm_out = generate_cluster_name_description(persona_texts, existing_names)
        cluster_info[cid] = {"name": llm_out["name"], "description": llm_out["description"]}
        print(f"Cluster {cid} -> Name: {llm_out['name']}")

    # Map back
    df["cluster_name"] = df["cluster_id"].map(lambda x: cluster_info[x]["name"])
    df["cluster_description"] = df["cluster_id"].map(lambda x: cluster_info[x]["description"])

    # Reorder columns
    cols_start = ["cluster_id", "cluster_name", "cluster_brief", "cluster_description", "persona_name"]
    cols_start = [c for c in cols_start if c in df.columns]
    remaining = [c for c in df.columns if c not in cols_start]
    df = df[cols_start + remaining]

    # Save to separate CSV
    df.to_csv(output_file, index=False)
    print(f"✅ Updated cluster names + descriptions saved to {output_file}")

    summary = pd.DataFrame([
        {"cluster_id": k, "cluster_name": v["name"], "cluster_description": v["description"]}
        for k, v in cluster_info.items()
    ])
    return summary

# ---------------- EXECUTION ----------------
if __name__ == "__main__":
    INPUT_FILE = "/content/persona_pipeline_output/micro_personas_final_expanded.csv"
    OUTPUT_FILE = "/content/persona_pipeline_output/micro_personas_cluster_summary.csv"
    summary_df = generate_cluster_llm(INPUT_FILE, OUTPUT_FILE)
    print(summary_df.head(20))


# **Cleaning**

In [ ]:
! du -h / | sort -h | tail -n 20


du: cannot access '/proc/13947/task/13947/fd/4': No such file or directory
du: cannot access '/proc/13947/task/13947/fdinfo/4': No such file or directory
du: cannot access '/proc/13947/fd/3': No such file or directory
du: cannot access '/proc/13947/fdinfo/3': No such file or directory
2.4G	/root/.julia/artifacts
2.8G	/usr/local/lib/python3.11/dist-packages/nvidia
3.8G	/root/.julia
4.0G	/root
4.3G	/usr/local/cuda-12.5/targets
4.3G	/usr/local/cuda-12.5/targets/x86_64-linux
4.3G	/usr/local/cuda-12.5/targets/x86_64-linux/lib
4.6G	/usr/lib/x86_64-linux-gnu
4.8G	/usr/local/cuda-12.5
5.5G	/usr/lib
14G	/content/models
14G	/content/models/models--mistralai--Mistral-7B-Instruct-v0.2
14G	/content/models/models--mistralai--Mistral-7B-Instruct-v0.2/blobs
18G	/content
19G	/usr/local/lib/python3.11
19G	/usr/local/lib/python3.11/dist-packages
20G	/usr/local/lib
26G	/usr/local
32G	/usr
57G	/


In [ ]:
# Remove HuggingFace model caches
!rm -rf /root/.cache/huggingface
!rm -rf ~/.cache/huggingface
!rm -rf /content/.cache/huggingface

# Remove pip cache
!pip cache purge

# Remove Python cache folders
! find / -type d -name "__pycache__" -exec rm -rf {} +

# Optional: Clear Jupyter history
! rm -rf ~/.local/share/jupyter/runtime/*


Files removed: 4


In [ ]:
!df -h
!du -sh /root/.cache /content


Filesystem      Size  Used Avail Use% Mounted on
overlay          95G   82G   13G  87% /
tmpfs            64M     0   64M   0% /dev
shm             423G     0  423G   0% /dev/shm
/dev/root       2.0G  1.2G  750M  62% /usr/sbin/docker-init
/dev/sdb        246G   18G  228G   8% /content
tmpfs           426G  8.2M  426G   1% /var/colab
/dev/sda1        95G   82G   13G  87% /etc/hosts
tmpfs           426G     0  426G   0% /proc/acpi
tmpfs           426G     0  426G   0% /proc/scsi
tmpfs           426G     0  426G   0% /sys/firmware
56M	/root/.cache
18G	/content


In [ ]:
! rm -rf /root/miniconda3/pkgs



# **WILL CHECK LATER**

In [ ]:
!pip install --quiet 'protobuf<5' transformers sentence-transformers accelerate torch --upgrade


✅ Step 1: Clustering

You created behavioral clusters by embedding each row’s behavioral attributes into vector space and grouping them using MiniBatchKMeans.

📊 Output added:

behavior_text: combined behavioral features per individual

behavior_cluster: assigned cluster ID (0–9)

In [ ]:
#3️⃣ Continue with your behavior-to-persona pipeline
import numpy as np, re
from sklearn.cluster import MiniBatchKMeans
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from tqdm import tqdm
from collections import Counter

# --- BUILD BEHAVIOR TEXT ---
def build_behavior_text(row):
    vals = []
    for col in BEHAVIORAL_COLS:
        v = str(row.get(col, "")).strip()
        if v and v.lower() not in ["nan", "none", "__missing__"]:
            vals.append(v)
    return " | ".join(vals)

tqdm.pandas(desc="Building behavior text")
df_sample["behavior_text"] = df_sample.progress_apply(build_behavior_text, axis=1)

# --- EMBEDDING ---
print("🔍 Generating embeddings...")
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = embedder.encode(df_sample["behavior_text"].tolist(), show_progress_bar=True)

# --- CLUSTERING ---
print("📊 Clustering behaviors...")
kmeans = MiniBatchKMeans(n_clusters=10, random_state=42, batch_size=2048, n_init=10)
df_sample["behavior_cluster"] = kmeans.fit_predict(embeddings)


Building behavior text: 100%|██████████| 26213/26213 [00:08<00:00, 3126.85it/s]


🔍 Generating embeddings...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/820 [00:00<?, ?it/s]

📊 Clustering behaviors...


✅ Step 2: Behavioral Themes

You summarized each cluster’s most frequent words → “Top Behavioral Themes” (keywords that describe what each cluster is about).

📄 Output:

cluster_keywords (DataFrame with behavior_cluster and top_behavioral_themes)

In [ ]:
#Extract Top Behavioral Themes for Each Cluster
import re
from collections import Counter
import pandas as pd

def extract_top_keywords(texts, top_n=8):
    text = " ".join(texts).lower()
    tokens = re.findall(r'\b[a-zA-Z]{3,}\b', text)
    common = [t[0] for t in Counter(tokens).most_common(top_n)]
    return ", ".join(common)

cluster_keywords = []
for cid, grp in df_sample.groupby("behavior_cluster"):
    top_kw = extract_top_keywords(grp["behavior_text"].tolist(), top_n=8)
    cluster_keywords.append({
        "behavior_cluster": cid,
        "top_behavioral_themes": top_kw
    })

cluster_keywords = pd.DataFrame(cluster_keywords)
print("✅ Extracted top behavioral themes per cluster")
display(cluster_keywords)


✅ Extracted top behavioral themes per cluster


,behavior_cluster,top_behavioral_themes
0,0,"other, last, the, months, interest, purchase, ..."
1,1,"other, months, credit, over, time, year, singl..."
2,2,"other, time, credit, cbsa, months, over, year,..."
3,3,"other, interest, interested, the, household, p..."
4,4,"other, last, the, months, online, credit, most..."
5,5,"other, home, the, interest, months, that, has,..."
6,6,"other, last, the, months, home, has, online, c..."
7,7,"other, months, credit, has, home, family, bala..."
8,8,"other, interest, months, interested, the, purc..."
9,9,"other, last, the, interest, months, purchase, ..."


✅ Step 3: Micro-Personas

You segmented clusters further using demographics (gender, age, religion, ethnicity).
Each (cluster + demographic combo) becomes a micro-persona, with behavior text samples aggregated.

📄 Output:

micro_personas DataFrame
Columns:
behavior_cluster, gender_imputed, age_imputed, religion_imputed,
ethnicity_imputed, persona_examples

In [ ]:
#Create Micro-Personas (Demographic Segments within Clusters)

#Now combine your behavioral clusters with demographics (like gender, age, religion, ethnicity) to create micro-personas.

micro_personas = []
GROUP_KEYS = ['behavior_cluster', 'gender_imputed', 'age_imputed', 'religion_imputed', 'ethnicity_imputed']

MIN_CLUSTER_SIZE = 40
MAX_PERSONA_EXAMPLES = 100

for keys, grp in df_sample.groupby(GROUP_KEYS):
    if len(grp) < MIN_CLUSTER_SIZE:
        continue
    cluster_id, gender, age, religion, ethnicity = keys
    persona_text = " ".join(grp["behavior_text"].sample(min(MAX_PERSONA_EXAMPLES, len(grp)), random_state=42))
    micro_personas.append({
        "behavior_cluster": cluster_id,
        "gender_imputed": gender,
        "age_imputed": age,
        "religion_imputed": religion,
        "ethnicity_imputed": ethnicity,
        "persona_examples": persona_text
    })

micro_personas = pd.DataFrame(micro_personas)
print("✅ Micro-personas created:", micro_personas.shape)
micro_personas.head()


✅ Micro-personas created: (131, 6)


,behavior_cluster,gender_imputed,age_imputed,religion_imputed,ethnicity_imputed,persona_examples
0,0,Female,18 - 24,__MISSING__,__MISSING__,Interested in the purchase of toys |Interested...
1,0,Female,25 - 34,Protestant,Caucasian|Western European,Interested in the purchase of toys |Interest i...
2,0,Female,25 - 34,Protestant,Western European|Caucasian,Interested in the purchase of toys |Interested...
3,0,Female,25 - 34,__MISSING__,__MISSING__,Interest in family activities |Interested in p...
4,0,Female,35 - 44,Protestant,Caucasian|Western European,Interested in the purchase of toys |Interested...


In [ ]:
! ls ~/.cache/huggingface/hub


models--philschmid--bart-large-cnn-samsum/
models--sentence-transformers--all-MiniLM-L6-v2/


In [ ]:
! rm -rf ~/.cache/huggingface/hub/models--philschmid--bart-large-cnn-samsum


✅ Step 4: Persona Summarization

You’ve used philschmid/bart-large-cnn-samsum (good choice ✅) to summarize each micro-persona into human-like descriptions.

📄 Output added:

persona_summary column

In [ ]:
from transformers import pipeline
from tqdm import tqdm

summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    device=-1
)

def summarize_persona(r):
    prompt = f"""
    You are a consumer insights strategist. Convert raw behavior signals into a structured human persona.

    Behavioral Signals (clean & synthesize, do NOT repeat):
    {r['persona_examples'][:500]}

    Demographics:
    - Gender: {r.get('gender_imputed','Unknown')}
    - Age Group: {r.get('age_imputed','Unknown')}
    - Religion: {r.get('religion_imputed','Unknown')}
    - Ethnicity: {r.get('ethnicity_imputed','Unknown')}

    Write a persona with:
    - Persona name + 1-line tagline
    - Key motivations & lifestyle
    - Digital behavior
    - Purchase & loyalty drivers
    - Preferred communication & content style

    Tone: Insight-driven, empathetic, concise. Avoid generic wording. No bullet lists. Natural flow.
    """

    try:
        out = summarizer(prompt, max_length=180, min_length=90, truncation=True)
        return out[0]["summary_text"]
    except:
        return ""

tqdm.pandas(desc="Summarizing personas")
micro_personas["persona_summary"] = micro_personas.progress_apply(summarize_persona, axis=1)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Summarizing personas: 100%|██████████| 131/131 [07:00<00:00,  3.21s/it]


In [ ]:
micro_personas.to_csv("behavior_personas.csv", index=False)
print("✔️ Exported to behavior_personas.csv")


✔️ Exported to behavior_personas.csv


In [ ]:
import pandas as pd
df_cluster = pd.read_csv("/content/behavior_personas.csv")
print(df_cluster.shape)
df_cluster.head(1)

(131, 7)


,behavior_cluster,gender_imputed,age_imputed,religion_imputed,ethnicity_imputed,persona_examples,persona_summary
0,0,Female,18 - 24,__MISSING__,__MISSING__,Interested in the purchase of toys |Interested...,You are a consumer insights strategist. Conver...


**HASHED**

In [ ]:
query = """
SELECT *
FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles_imputed_only_cleaned`
LIMIT 10
"""
df = client.query(query).to_dataframe()
df

In [ ]:
query = """
SELECT
  column_name,
  data_type
FROM `stgw-shared-ai-dev.parquet_data_set.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = 'clustered_profiles'
ORDER BY ordinal_position
"""
columns_df = client.query(query).to_dataframe()
columns_df


In [ ]:
query = """
SELECT
  predicted_cluster,
  column_name,
  value,
  COUNT(*) AS freq,
  ROUND(COUNT(*) * 100 / SUM(COUNT(*)) OVER (PARTITION BY predicted_cluster, column_name), 2) AS pct_within_cluster
FROM (
  SELECT predicted_cluster, 'credit_tier_imputed' AS column_name, credit_tier_imputed AS value FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles`
  UNION ALL
  SELECT predicted_cluster, 'loan_imputed', loan_imputed FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles`
  UNION ALL
  SELECT predicted_cluster, 'financial_beliefs_imputed', financial_beliefs_imputed FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles`
  UNION ALL
  SELECT predicted_cluster, 'family_income_decile_imputed', family_income_decile_imputed FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles`
)
GROUP BY predicted_cluster, column_name, value
QUALIFY ROW_NUMBER() OVER (PARTITION BY predicted_cluster, column_name ORDER BY freq DESC) = 1
ORDER BY predicted_cluster, column_name;
"""

top_features = client.query(query).to_dataframe()
top_features


In [ ]:
from google.cloud import bigquery

PROJECT_ID = "stgw-shared-ai-dev"
client = bigquery.Client(project=PROJECT_ID)

query = """
CREATE OR REPLACE TABLE `stgw-shared-ai-dev.parquet_data_set.cluster_lifestyle_summary` AS
WITH combined AS (
  SELECT
    predicted_cluster,
    ARRAY_TO_STRING(
      ARRAY(
        SELECT val FROM UNNEST([
          leisure_activities_hobbies_imputed,
          health_and_fitness_imputed,
          green_living_imputed,
          sports_interests_imputed,
          entertainment_interests_imputed,
          travel_interests_imputed,
          children_family_imputed
        ]) AS val
        WHERE val IS NOT NULL AND TRIM(val) != ''
      ),
      '|'
    ) AS all_interests
  FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles`
  WHERE leisure_activities_hobbies_imputed IS NOT NULL
     OR health_and_fitness_imputed IS NOT NULL
     OR green_living_imputed IS NOT NULL
     OR sports_interests_imputed IS NOT NULL
     OR entertainment_interests_imputed IS NOT NULL
     OR travel_interests_imputed IS NOT NULL
     OR children_family_imputed IS NOT NULL
),
exploded AS (
  SELECT
    predicted_cluster,
    TRIM(value) AS interest
  FROM combined,
  UNNEST(SPLIT(LOWER(all_interests), '|')) AS value
  WHERE TRIM(value) != ''
)
SELECT
  predicted_cluster,
  interest,
  COUNT(*) AS freq,
  ROUND(COUNT(*) * 100 / SUM(COUNT(*)) OVER (PARTITION BY predicted_cluster), 2) AS pct_within_cluster
FROM exploded
GROUP BY predicted_cluster, interest
QUALIFY ROW_NUMBER() OVER (PARTITION BY predicted_cluster ORDER BY freq DESC) <= 5
ORDER BY predicted_cluster, freq DESC;
"""

# Execute query
job = client.query(query)
job.result()

print("✅ Cluster lifestyle summary table created successfully: parquet_data_set.cluster_lifestyle_summary")


# **Preview Lifestyle Summary Table**

In [ ]:
query = """
SELECT *
FROM `stgw-shared-ai-dev.parquet_data_set.cluster_lifestyle_summary`
ORDER BY predicted_cluster, freq DESC
"""
df_lifestyle = client.query(query).to_dataframe()

print("✅ Lifestyle Summary — Top Rows")
df_lifestyle.head(20)


In [ ]:
query = """
CREATE OR REPLACE TABLE `stgw-shared-ai-dev.parquet_data_set.cluster_demo_fin_summary` AS
WITH base AS (
  SELECT
    predicted_cluster,
    generations_imputed,
    gender_imputed,
    urbanicity_imputed,
    education_individual_imputed,
    marital_status_imputed,
    credit_tier_imputed,
    family_income_decile_imputed,
    net_worth_imputed,
    credit_card_balance_imputed,
    loan_imputed
  FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles`
),

-- Step 1: Calculate mode (most frequent category) per variable per cluster
modes AS (
  SELECT predicted_cluster, 'generations_imputed' AS variable, generations_imputed AS value, COUNT(*) AS freq FROM base GROUP BY 1,3
  UNION ALL
  SELECT predicted_cluster, 'gender_imputed', gender_imputed, COUNT(*) FROM base GROUP BY 1,3
  UNION ALL
  SELECT predicted_cluster, 'urbanicity_imputed', urbanicity_imputed, COUNT(*) FROM base GROUP BY 1,3
  UNION ALL
  SELECT predicted_cluster, 'education_individual_imputed', education_individual_imputed, COUNT(*) FROM base GROUP BY 1,3
  UNION ALL
  SELECT predicted_cluster, 'marital_status_imputed', marital_status_imputed, COUNT(*) FROM base GROUP BY 1,3
  UNION ALL
  SELECT predicted_cluster, 'credit_tier_imputed', credit_tier_imputed, COUNT(*) FROM base GROUP BY 1,3
  UNION ALL
  SELECT predicted_cluster, 'family_income_decile_imputed', family_income_decile_imputed, COUNT(*) FROM base GROUP BY 1,3
  UNION ALL
  SELECT predicted_cluster, 'net_worth_imputed', net_worth_imputed, COUNT(*) FROM base GROUP BY 1,3
  UNION ALL
  SELECT predicted_cluster, 'credit_card_balance_imputed', credit_card_balance_imputed, COUNT(*) FROM base GROUP BY 1,3
  UNION ALL
  SELECT predicted_cluster, 'loan_imputed', loan_imputed, COUNT(*) FROM base GROUP BY 1,3
),

-- Step 2: Get the most frequent value (mode) per variable per cluster
top_modes AS (
  SELECT *
  FROM (
    SELECT *,
           ROW_NUMBER() OVER(PARTITION BY predicted_cluster, variable ORDER BY freq DESC) AS rn
    FROM modes
  )
  WHERE rn = 1
)

-- Step 3: Pivot results into a readable summary
SELECT
  predicted_cluster,
  MAX(CASE WHEN variable = 'generations_imputed' THEN value END) AS dominant_generation,
  MAX(CASE WHEN variable = 'gender_imputed' THEN value END) AS dominant_gender,
  MAX(CASE WHEN variable = 'urbanicity_imputed' THEN value END) AS dominant_urbanicity,
  MAX(CASE WHEN variable = 'education_individual_imputed' THEN value END) AS dominant_education,
  MAX(CASE WHEN variable = 'marital_status_imputed' THEN value END) AS dominant_marital_status,
  MAX(CASE WHEN variable = 'credit_tier_imputed' THEN value END) AS dominant_credit_tier,
  MAX(CASE WHEN variable = 'family_income_decile_imputed' THEN value END) AS dominant_income_decile,
  MAX(CASE WHEN variable = 'net_worth_imputed' THEN value END) AS dominant_net_worth_band,
  MAX(CASE WHEN variable = 'credit_card_balance_imputed' THEN value END) AS dominant_credit_balance_band,
  MAX(CASE WHEN variable = 'loan_imputed' THEN value END) AS dominant_loan_band
FROM top_modes
GROUP BY predicted_cluster
ORDER BY predicted_cluster;
"""

# Run in Python
job = client.query(query)
job.result()

print("✅ Cluster demographic & financial summary table created successfully (string-compatible): parquet_data_set.cluster_demo_fin_summary")


# **Preview Demographic & Financial summary table**

In [ ]:
query = """
SELECT *
FROM `stgw-shared-ai-dev.parquet_data_set.cluster_demo_fin_summary`
ORDER BY predicted_cluster
"""
df_demo_fin = client.query(query).to_dataframe()

print("✅ Demographic & Financial Summary — Top Rows")
df_demo_fin.head(10)


In [ ]:
cluster_summary = (
    df_demo_fin
    .merge(df_lifestyle.groupby(['predicted_cluster'])['interest']
           .apply(lambda x: ', '.join(x.head(5))).reset_index(),
           on='predicted_cluster', how='left')
)

cluster_summary.rename(columns={'interest': 'top_lifestyle_interests'}, inplace=True)

print("✅ Combined Cluster Summary — Top 10")
cluster_summary.head(10)


# **Let's Chat**

# Implementation Flow for RAG over BigQuery + Open-Source LLM

In [ ]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client(project="stgw-shared-ai-dev")

query = """
SELECT *
FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles_imputed_only_cleaned`
WHERE RAND() < 0.001
LIMIT 5000
"""
df = client.query(query).to_dataframe()
df.head()


# **Define your categories as Python lists**

In [ ]:
# Define thematic column groups
demographics = [
    "gender_imputed", "age_imputed", "generations_imputed", "marital_status_imputed",
    "urbanicity_imputed", "education_individual_imputed", "ethnicity_imputed",
    "language_spoken_imputed"
]

financial = [
    "family_income_decile_imputed", "average_household_income_imputed", "net_worth_imputed",
    "credit_tier_imputed", "credit_card_spend_imputed", "credit_card_balance_imputed",
    "financial_beliefs_imputed", "discretionary_spend_imputed", "wealth_rating_imputed",
    "family_level_of_investable_assets_imputed", "home_equity_imputed",
    "loan_imputed", "mortgage_loan_imputed", "auto_loan_imputed",
    "investments_type_imputed", "financial_institutions_imputed"
]

property_home = [
    "home_ownership_imputed", "dwelling_unit_size_imputed", "building_type_imputed",
    "property_size_imputed", "property_realty_indicator_imputed", "current_home_value_imputed",
    "number_of_bedrooms_imputed", "number_of_bathrooms_imputed",
    "heating_type_imputed", "air_conditioning_imputed"
]

technology = [
    "cell_smartphones_tablets_imputed", "mobile_devices_imputed", "mobile_phone_brand_imputed",
    "mobile_plan_type_imputed", "i_device_types_imputed", "i_device_models_imputed",
    "i_device_oss_imputed", "streaming_system_device_use_imputed", "technology_imputed",
    "computer_purchase_imputed", "tablets_imputed", "i_app_taxonomies_imputed", "apps_imputed"
]

shopping = [
    "shopping_imputed", "stores_imputed", "commerce_retail_shopping_imputed",
    "department_discount_warehouse_imputed", "apparel_imputed", "shoes_imputed",
    "coupons_used_imputed", "mail_purchases_imputed"
]

food = [c for c in df.columns if c.startswith("grocery_")]

lifestyle = [
    "leisure_activities_hobbies_imputed", "health_and_fitness_imputed", "green_living_imputed",
    "auto_imputed", "sports_interests_imputed", "entertainment_interests_imputed",
    "travel_interests_imputed", "children_family_imputed"
]

media = [
    "television_shows_imputed", "entertainment_sites_imputed", "sports_sites_imputed",
    "news_sites_imputed", "social_sites_imputed", "i_iab_categories_imputed",
    "movie_genre_saw_in_theater_imputed"
]

political = [
    "voting_behavior_imputed", "political_affiliations_imputed", "political_outlook_imputed",
    "religion_imputed", "charitable_contributions_imputed"
]


**Step 2: Define a helper function to safely concatenate text per theme**

This will join only non-null fields and add fallbacks like __NoData__ if the entire section is blank.

In [ ]:
def describe_section(df_row, columns, label):
    values = df_row[columns].dropna().astype(str).unique()
    if len(values) == 0:
        return f"{label}: __None__"
    return f"{label}: " + ", ".join(values[:10])  # limit to 10 per section for readability


**Step 3: Build a comprehensive persona description per row**

This uses your logical groups and combines them into one natural-language field for embedding or clustering.

In [ ]:
persona_texts = []

for _, row in df.iterrows():
    sections = [
        f"Cluster: {row['predicted_cluster']}",
        describe_section(row, demographics, "Demographics"),
        describe_section(row, financial, "Financial Profile"),
        describe_section(row, property_home, "Home & Property"),
        describe_section(row, technology, "Technology & Devices"),
        describe_section(row, shopping, "Shopping Behavior"),
        describe_section(row, food, "Food & Grocery Preferences"),
        describe_section(row, lifestyle, "Lifestyle & Hobbies"),
        describe_section(row, media, "Media & Content Interests"),
        describe_section(row, political, "Political & Social Orientation")
    ]
    persona_texts.append(" | ".join(sections))

df["persona_text"] = persona_texts


**Step 4: Optional — Summarize by Cluster**

Aggregate per cluster to create cluster-level personas for your LLM or RAG pipeline.

In [ ]:
cluster_profiles = (
    df.groupby("predicted_cluster")["persona_text"]
    .apply(lambda x: " ".join(x.sample(n=min(100, len(x)))))  # summarize 100 samples per cluster
    .reset_index()
)


**Step-6: Generate short readable persona text per cluster**

In [ ]:
cluster_summary = (
    df.groupby("predicted_cluster")
    .agg({
        "age_imputed": lambda x: x.mode().iloc[0] if not x.mode().empty else None,
        "gender_imputed": lambda x: x.mode().iloc[0] if not x.mode().empty else None,
        "urbanicity_imputed": lambda x: x.mode().iloc[0] if not x.mode().empty else None,
        "family_income_decile_imputed": lambda x: x.mode().iloc[0] if not x.mode().empty else None,
        "credit_tier_imputed": lambda x: x.mode().iloc[0] if not x.mode().empty else None,
        "persona_text": lambda x: " ".join(x.sample(n=min(100, len(x))))
    })
    .reset_index()
)


**Generate clean AI summaries**

In [ ]:
from transformers import pipeline
from collections import Counter

# use flan-t5-large for richer summaries (if RAM allows)
summarizer = pipeline("text2text-generation", model="google/flan-t5-large")

def top_terms(series, n=3):
    vals = [v for v in series.dropna().astype(str).tolist() if v.lower() not in ["none", "unknown", "nan", ""]]
    return ", ".join([v for v, _ in Counter(vals).most_common(n)]) if vals else "None"

def structured_to_summary(row):
    desc = (
        f"Demographics: {row.get('gender_imputed', 'Unknown')}, aged {row.get('age_imputed', 'Unknown')}, "
        f"living in {row.get('urbanicity_imputed', 'Unknown')} area. "
        f"Financial: typically {row.get('credit_tier_imputed', 'Unknown')} credit tier, "
        f"in {row.get('family_income_decile_imputed', 'Unknown')} income decile. "
        f"Top interests include: {row.get('top_lifestyle', 'None')} and {row.get('top_media', 'None')}. "
        f"Technology habits: {row.get('top_tech', 'None')}. "
        f"Shopping style: {row.get('top_shopping', 'None')}."
    )

    prompt = (
        f"Summarize this persona cluster in 2-3 fluent sentences for a marketing audience:\n{desc}\n"
        f"Highlight their lifestyle, tech habits, and financial standing in natural language."
    )

    return summarizer(prompt, max_new_tokens=80, do_sample=False)[0]["generated_text"]



In [ ]:
# summarize key themes per cluster for lifestyle, media, tech, shopping
def top_category_terms(df, cluster_col, category_cols, name):
    return (
        df.groupby(cluster_col)[category_cols]
        .agg(lambda x: top_terms(x))
        .rename(columns={c: f"{name}_{c}" for c in category_cols})
    )

cluster_themes = df.groupby("predicted_cluster").agg({
    "lifestyle_imputed": top_terms,
    "media_imputed": top_terms,
    "technology_imputed": top_terms,
    "shopping_imputed": top_terms
}).reset_index()

# merge into cluster_summary
cluster_summary = cluster_summary.merge(cluster_themes, on="predicted_cluster", how="left")

# rename for readability
cluster_summary.rename(columns={
    "lifestyle_imputed": "top_lifestyle",
    "media_imputed": "top_media",
    "technology_imputed": "top_tech",
    "shopping_imputed": "top_shopping"
}, inplace=True)


**Step-8:Store in Chroma & query semantically**

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

embeddings = embedding_model.encode(cluster_summary["persona_text_clean"].tolist(), show_progress_bar=True)
embeddings_fixed = [list(map(float, np.array(e))) for e in embeddings]

collection = chroma_client.create_collection("cluster_personas_clean", get_or_create=True)
collection.add(
    ids=cluster_summary["predicted_cluster"].astype(str).tolist(),
    embeddings=embeddings_fixed,
    documents=cluster_summary["persona_text_clean"].tolist(),
    metadatas=cluster_summary[["predicted_cluster", "gender_imputed", "age_imputed", "urbanicity_imputed"]].to_dict(orient="records")
)


In [ ]:
query = "urban tech-savvy millennials who like digital banking and streaming services"

results = collection.query(query_texts=[query], n_results=4)

for i, meta in enumerate(results['metadatas'][0]):
    print(f"🧩 Cluster {meta['predicted_cluster']}")
    print(f"👥 Age: {meta.get('age_imputed', 'Unknown')} | Gender: {meta.get('gender_imputed', 'Unknown')} | Urbanicity: {meta.get('urbanicity_imputed', 'Unknown')}")
    print(f"📝 Summary: {results['documents'][0][i]}\n")


# **Full Revised**

In [ ]:
# ============================================================
# 🧠 RAG over BigQuery + Open-Source LLM Personas (Stable Version)
# ============================================================

from google.cloud import bigquery
import pandas as pd
import numpy as np
from collections import Counter
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import chromadb

# ============================================================
# 1️⃣ Load data from BigQuery
# ============================================================

client = bigquery.Client(project="stgw-shared-ai-dev")

query = """
SELECT *
FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles_imputed_only_cleaned`
WHERE RAND() < 0.001
LIMIT 5000
"""
df = client.query(query).to_dataframe()
print("✅ Sample loaded:", df.shape)

# ============================================================
# 2️⃣ Define column groups for theme-based persona creation
# ============================================================

demographics = [
    "gender_imputed", "age_imputed", "generations_imputed", "marital_status_imputed",
    "urbanicity_imputed", "education_individual_imputed", "ethnicity_imputed",
    "language_spoken_imputed"
]

financial = [
    "family_income_decile_imputed", "average_household_income_imputed", "net_worth_imputed",
    "credit_tier_imputed", "credit_card_spend_imputed", "credit_card_balance_imputed",
    "financial_beliefs_imputed", "discretionary_spend_imputed", "wealth_rating_imputed",
    "family_level_of_investable_assets_imputed", "home_equity_imputed",
    "loan_imputed", "mortgage_loan_imputed", "auto_loan_imputed",
    "investments_type_imputed", "financial_institutions_imputed"
]

property_home = [
    "home_ownership_imputed", "dwelling_unit_size_imputed", "building_type_imputed",
    "property_size_imputed", "property_realty_indicator_imputed", "current_home_value_imputed",
    "number_of_bedrooms_imputed", "number_of_bathrooms_imputed",
    "heating_type_imputed", "air_conditioning_imputed"
]

technology = [
    "mobile_phone_brand_imputed", "streaming_system_device_use_imputed",
    "apps_imputed", "i_app_taxonomies_imputed"
]

shopping = [
    "stores_imputed", "commerce_retail_shopping_imputed",
    "department_discount_warehouse_imputed", "apparel_imputed", "shoes_imputed"
]

food = [c for c in df.columns if c.startswith("grocery_")]

lifestyle = [
    "leisure_activities_hobbies_imputed", "sports_interests_imputed",
    "entertainment_interests_imputed", "travel_interests_imputed"
]

media = [
    "television_shows_imputed", "entertainment_sites_imputed",
    "sports_sites_imputed", "social_sites_imputed", "movie_genre_saw_in_theater_imputed"
]

political = [
    "voting_behavior_imputed", "political_affiliations_imputed",
    "political_outlook_imputed", "religion_imputed", "charitable_contributions_imputed"
]

# ============================================================
# 3️⃣ Helper: describe a section concisely
# ============================================================

def describe_section(df_row, columns, label):
    values = df_row[columns].dropna().astype(str).unique()
    if len(values) == 0:
        return f"{label}: None"
    return f"{label}: " + ", ".join(values[:10])  # cap for readability

# ============================================================
# 4️⃣ Build long-form persona text for each row
# ============================================================

persona_texts = []
for _, row in df.iterrows():
    sections = [
        f"Cluster: {row['predicted_cluster']}",
        describe_section(row, demographics, "Demographics"),
        describe_section(row, financial, "Financial Profile"),
        describe_section(row, property_home, "Home & Property"),
        describe_section(row, technology, "Technology & Devices"),
        describe_section(row, shopping, "Shopping Behavior"),
        describe_section(row, food, "Food & Grocery Preferences"),
        describe_section(row, lifestyle, "Lifestyle & Hobbies"),
        describe_section(row, media, "Media & Content Interests"),
        describe_section(row, political, "Political & Social Orientation")
    ]
    persona_texts.append(" | ".join(sections))

df["persona_text"] = persona_texts
print("✅ Persona text built for all rows")

# ============================================================
# 5️⃣ Cluster summaries (key demographic modes)
# ============================================================

cluster_summary = (
    df.groupby("predicted_cluster")
    .agg({
        "age_imputed": lambda x: x.mode().iloc[0] if not x.mode().empty else None,
        "gender_imputed": lambda x: x.mode().iloc[0] if not x.mode().empty else None,
        "urbanicity_imputed": lambda x: x.mode().iloc[0] if not x.mode().empty else None,
        "family_income_decile_imputed": lambda x: x.mode().iloc[0] if not x.mode().empty else None,
        "credit_tier_imputed": lambda x: x.mode().iloc[0] if not x.mode().empty else None,
        "persona_text": lambda x: " ".join(x.sample(n=min(100, len(x))))
    })
    .reset_index()
)

# Clean and trim persona_texts (prevent model overflow)
cluster_summary["persona_text"] = (
    cluster_summary["persona_text"]
    .astype(str)
    .str.replace(r"\s+", " ", regex=True)
    .str.slice(0, 1200)  # keep under safe token limit
)

# ============================================================
# 6️⃣ Extract top behavioral themes
# ============================================================

def top_terms(series, n=3):
    vals = [v for v in series.dropna().astype(str).tolist() if v.lower() not in ["none", "unknown", "nan", ""]]
    return ", ".join([v for v, _ in Counter(vals).most_common(n)]) if vals else "None"

behavioral_cols = {
    "top_tech": technology,
    "top_shopping": shopping,
    "top_lifestyle": lifestyle,
    "top_media": media
}

for name, cols in behavioral_cols.items():
    frame = (
        df.groupby("predicted_cluster")[cols]
        .agg(lambda x: top_terms(x))
        .reset_index()
    )
    frame[name] = frame[cols].apply(lambda r: ", ".join(r.values), axis=1)
    cluster_summary = cluster_summary.merge(frame[["predicted_cluster", name]], on="predicted_cluster", how="left")

print("✅ Cluster themes summarized")

# ============================================================
# 7️⃣ Summarization Helper (chunk-safe)
# ============================================================

summarizer = pipeline("text2text-generation", model="google/flan-t5-large", truncation=True)

def summarize_long_text(text, chunk_words=200, max_new_tokens=80):
    """Break long text into safe-sized chunks."""
    words = text.split()
    chunks = [" ".join(words[i:i + chunk_words]) for i in range(0, len(words), chunk_words)]
    summaries = []
    for chunk in chunks:
        try:
            out = summarizer(f"Summarize this persona: {chunk}",
                             max_new_tokens=max_new_tokens, truncation=True)
            summaries.append(out[0]["generated_text"])
        except Exception:
            continue
    return " ".join(summaries)[:500]

# ============================================================
# 8️⃣ Generate compact AI persona summaries
# ============================================================

def structured_to_summary(row):
    desc_fields = [
        f"Gender: {row.get('gender_imputed','Unknown')}",
        f"Age: {row.get('age_imputed','Unknown')}",
        f"Urbanicity: {row.get('urbanicity_imputed','Unknown')}",
        f"Credit tier: {row.get('credit_tier_imputed','Unknown')}",
        f"Income decile: {row.get('family_income_decile_imputed','Unknown')}"
    ]
    persona_snippet = row.get("persona_text", "")
    short_desc = ". ".join(desc_fields) + ". " + persona_snippet
    summary = summarize_long_text(short_desc)
    if not summary.strip():
        summary = (
            f"{row.get('gender_imputed','Unknown')}, {row.get('age_imputed','Unknown')}, "
            f"{row.get('urbanicity_imputed','Unknown')} | Credit: {row.get('credit_tier_imputed','Unknown')} | "
            f"Behaviors: {row.get('top_tech','None')}, {row.get('top_shopping','None')}."
        )
    return summary

cluster_summary["persona_text_clean"] = cluster_summary.apply(structured_to_summary, axis=1)
print("✅ Persona summaries generated")

# ============================================================
# 9️⃣ Store in ChromaDB
# ============================================================

chroma_client = chromadb.Client()
try:
    chroma_client.delete_collection("cluster_personas_clean")
except Exception:
    pass

collection = chroma_client.create_collection("cluster_personas_clean")

embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
embeddings = embedding_model.encode(cluster_summary["persona_text_clean"].tolist(), show_progress_bar=True)
embeddings_fixed = [list(map(float, np.array(e))) for e in embeddings]

collection.add(
    ids=cluster_summary["predicted_cluster"].astype(str).tolist(),
    embeddings=embeddings_fixed,
    documents=cluster_summary["persona_text_clean"].tolist(),
    metadatas=cluster_summary[["predicted_cluster", "gender_imputed", "age_imputed", "urbanicity_imputed"]].to_dict(orient="records")
)

print("✅ Stored cluster personas in ChromaDB:", collection.count())

# ============================================================
# 🔟 Query Example
# ============================================================

query = "urban tech-savvy millennials who like digital banking and streaming services"
results = collection.query(query_texts=[query], n_results=4)

print("🔎 Top matching clusters:\n")
for i, meta in enumerate(results["metadatas"][0]):
    print(f"🧩 Cluster {meta['predicted_cluster']}")
    print(f"👥 Age: {meta.get('age_imputed', 'Unknown')} | Gender: {meta.get('gender_imputed', 'Unknown')} | Urbanicity: {meta.get('urbanicity_imputed', 'Unknown')}")
    print(f"📝 Summary: {results['documents'][0][i]}\n")


# **Revised**

In [ ]:
!pip install sentence-transformers chromadb transformers scikit-learn

In [ ]:
# ============================================================
# Re-cluster + core-member selection + clean persona summaries
# ============================================================
# Requirements (install if needed in Colab):
# !pip install sentence-transformers chromadb transformers scikit-learn

from google.cloud import bigquery
import pandas as pd
import numpy as np
from collections import Counter
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples
from transformers import pipeline
import chromadb

# ---------------------------
# PARAMETERS (tweakable)
# ---------------------------
N_CLUSTERS = 10
MIN_CLUSTER_SIZE = 30        # drop clusters smaller than this
SILHOUETTE_Q = 0.30         # keep only samples with silhouette > q-quantile within cluster
SAMPLE_FRACTION = 0.001      # same sampling you used (adjust if needed)
MAX_PERSONA_SAMPLE = 100     # samples per cluster to build textual cues

# ---------------------------
# 1. Load data (sample)
# ---------------------------
client = bigquery.Client(project="stgw-shared-ai-dev")

query = f"""
SELECT *
FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles_imputed_only_cleaned`
WHERE RAND() < {SAMPLE_FRACTION}
LIMIT 5000
"""
df = client.query(query).to_dataframe()
print("Loaded sample rows:", df.shape)

# ---------------------------
# 2. Build row-level persona_text (concise)
# ---------------------------
# Keep this short per row to make embeddings clean
def describe_section(df_row, cols, cap=3):
    vals = df_row[cols].dropna().astype(str).tolist()
    vals = [v for v in vals if v and v.lower() not in ("none","nan","__missing__","__MISSING__")]
    # return up to cap unique tokens joined
    uniq = []
    for v in vals:
        if v not in uniq:
            uniq.append(v)
        if len(uniq) >= cap:
            break
    return ", ".join(uniq) if uniq else ""

# Choose a small set of high-signal columns for the row text
row_signal_cols = [
    "gender_imputed", "age_imputed", "urbanicity_imputed",
    "mobile_phone_brand_imputed", "apps_imputed", "streaming_system_device_use_imputed",
    "stores_imputed", "leisure_activities_hobbies_imputed"
]
# safe fallback if any missing in df
row_signal_cols = [c for c in row_signal_cols if c in df.columns]

def build_row_persona(r):
    parts = []
    for c in row_signal_cols:
        s = describe_section(r, [c], cap=2)
        if s:
            parts.append(f"{c.split('_imputed')[0]}: {s}")
    return " | ".join(parts)

df["row_persona_text"] = df.apply(build_row_persona, axis=1)
print("Built concise row_persona_text")

# ---------------------------
# 3. Embed row texts using MiniLM
# ---------------------------
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
row_texts = df["row_persona_text"].astype(str).tolist()
row_embeddings = embed_model.encode(row_texts, show_progress_bar=True)
print("Embeddings shape:", len(row_embeddings), len(row_embeddings[0]))

# ---------------------------
# 4. Re-cluster embeddings (KMeans)
# ---------------------------
kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init=10)
labels = kmeans.fit_predict(row_embeddings)
df["recluster_label"] = labels
print("Clusters assigned. Distribution:\n", df["recluster_label"].value_counts().sort_index())

# ---------------------------
# 5. Filter: drop tiny clusters
# ---------------------------
counts = df["recluster_label"].value_counts()
valid_clusters = counts[counts >= MIN_CLUSTER_SIZE].index.tolist()
df = df[df["recluster_label"].isin(valid_clusters)].copy()
print("After dropping small clusters, rows:", df.shape, "valid clusters:", valid_clusters)

# ---------------------------
# 6. Compute silhouette (within remaining clusters)
# ---------------------------
# Recompute embeddings array for filtered df
emb_array = np.array([row_embeddings[i] for i in df.index])
# If only one cluster remains, skip silhouette
if len(df["recluster_label"].unique()) > 1:
    sil_scores = silhouette_samples(emb_array, df["recluster_label"].to_numpy())
    df["silhouette"] = sil_scores
    # Keep core members per cluster: silhouette greater than cluster-quantile
    def keep_core(g):
        thr = g["silhouette"].quantile(SILHOUETTE_Q)
        core = g[g["silhouette"] > thr]
        # ensure at least some rows remain
        if core.shape[0] < max(3, int(0.2 * g.shape[0])):
            return g.sort_values("silhouette", ascending=False).head(max(3, int(0.2 * g.shape[0])))
        return core
    df_core = df.groupby("recluster_label", group_keys=False).apply(keep_core).reset_index(drop=True)
else:
    df["silhouette"] = 0.0
    df_core = df.copy()

print("Kept core members:", df_core.shape)

# ---------------------------
# 7. Build cluster-level structured summary using modes
# ---------------------------
def mode_safe(series):
    s = series.dropna().astype(str)
    return s.mode().iloc[0] if not s.mode().empty else None

cluster_cols_for_modes = [
    "age_imputed","gender_imputed","urbanicity_imputed",
    "family_income_decile_imputed","credit_tier_imputed"
]
# ensure columns exist
cluster_cols_for_modes = [c for c in cluster_cols_for_modes if c in df_core.columns]

# behavioral/top terms to capture brands/interests
behavioral_cols = [
    "mobile_phone_brand_imputed","apps_imputed","streaming_system_device_use_imputed",
    "stores_imputed","leisure_activities_hobbies_imputed","television_shows_imputed"
]
behavioral_cols = [c for c in behavioral_cols if c in df_core.columns]

def top_terms(series, n=3):
    vals = [v for v in series.dropna().astype(str).tolist() if v and v.lower() not in ("none","nan","__missing__","__MISSING__")]
    if not vals:
        return "None"
    cnt = Counter(vals)
    return ", ".join([t for t,_ in cnt.most_common(n)])

cluster_summary = []
for cid, grp in df_core.groupby("recluster_label"):
    row = {"cluster": int(cid), "size": int(len(grp))}
    # modes
    for c in cluster_cols_for_modes:
        row[c] = mode_safe(grp[c])
    # top behavior terms
    for c in behavioral_cols:
        row[f"top_{c}"] = top_terms(grp[c], n=3)
    # representative persona snippet (small)
    sample_texts = grp["row_persona_text"].dropna().astype(str).tolist()
    if sample_texts:
        row["persona_snippet"] = " | ".join(sample_texts[:min(MAX_PERSONA_SAMPLE, len(sample_texts))])[:1000]
    else:
        row["persona_snippet"] = ""
    cluster_summary.append(row)

cluster_summary = pd.DataFrame(cluster_summary).sort_values("cluster").reset_index(drop=True)
print("Built cluster_summary rows:", cluster_summary.shape)

# ---------------------------
# 8. Construct short structured prompt and summarize with a safe summarizer
# ---------------------------
# Use a CPU-friendly summarizer; distilbart is robust for short inputs
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=-1)

def make_prompt_and_summarize(r):
    parts = []
    parts.append(f"Gender: {r.get('gender_imputed','Unknown')}")
    parts.append(f"Age: {r.get('age_imputed','Unknown')}")
    parts.append(f"Urbanicity: {r.get('urbanicity_imputed','Unknown')}")
    parts.append(f"Income decile: {r.get('family_income_decile_imputed','Unknown')}")
    parts.append(f"Credit tier: {r.get('credit_tier_imputed','Unknown')}")
    # add top behaviors succinctly
    beh = []
    for c in behavioral_cols:
        v = r.get(f"top_{c}")
        if v and v != "None":
            shortname = c.replace("_imputed","").replace("top_","")
            beh.append(f"{shortname}: {v}")
    if beh:
        parts.append("Top behaviors: " + "; ".join(beh[:6]))
    if r.get("persona_snippet"):
        parts.append("Samples: " + r["persona_snippet"][:400])
    prompt = ". ".join(parts)
    # summarizer expects moderate length inputs — our prompt is short
    try:
        out = summarizer(prompt, max_length=80, min_length=20, truncation=True)
        summary = out[0].get("summary_text", "") if isinstance(out[0], dict) else str(out[0])
    except Exception as e:
        summary = prompt[:200]
    return summary.strip()

cluster_summary["persona_text_clean"] = cluster_summary.apply(make_prompt_and_summarize, axis=1)
print("Generated persona_text_clean for clusters.")

# ---------------------------
# 9. Embed cluster-level summaries & store in Chroma
# ---------------------------
chroma_client = chromadb.Client()
# delete & recreate to be safe
try:
    chroma_client.delete_collection("cluster_personas_clean_v2")
except Exception:
    pass
collection = chroma_client.create_collection("cluster_personas_clean_v2")

cluster_embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
cluster_embeddings = cluster_embed_model.encode(cluster_summary["persona_text_clean"].tolist(), show_progress_bar=True)
embeddings_fixed = [list(map(float, np.array(e))) for e in cluster_embeddings]

metadatas = cluster_summary[["cluster","size"]].to_dict(orient="records")
collection.add(
    ids=[str(int(c)) for c in cluster_summary["cluster"].tolist()],
    embeddings=embeddings_fixed,
    documents=cluster_summary["persona_text_clean"].tolist(),
    metadatas=metadatas
)
print("Stored", collection.count(), "clusters in Chroma.")

# ---------------------------
# 10. Provide a query function (returns top k clusters + small structured info)
# ---------------------------
def query_clusters_natural(user_query, top_k=4):
    q_emb = cluster_embed_model.encode([user_query])[0].tolist()
    res = collection.query(query_embeddings=[q_emb], n_results=top_k)
    out = []
    for i, cid in enumerate(res["ids"][0]):
        meta = res["metadatas"][0][i]
        doc = res["documents"][0][i]
        cid_int = int(cid)
        cs = cluster_summary[cluster_summary["cluster"]==cid_int].iloc[0].to_dict()
        out.append({
            "cluster": cid_int,
            "score": res["distances"][0][i] if "distances" in res else None,
            "size": meta.get("size"),
            "age": cs.get("age_imputed"),
            "gender": cs.get("gender_imputed"),
            "urbanicity": cs.get("urbanicity_imputed"),
            "summary": doc
        })
    return pd.DataFrame(out)

# Example:
print("\nExample query results:")
print(query_clusters_natural("Who uses mobile banking and streaming apps?", top_k=4).to_string(index=False))


In [ ]:
# ============================================================
# 11️⃣ Auto-Generate Short Persona Titles (Fixed)
# ============================================================

from transformers import pipeline

title_gen = pipeline("text2text-generation", model="google/flan-t5-small")

def make_title(summary):
    prompt = f"Give a short, catchy 3-5 word persona title for this cluster:\n{summary}"
    out = title_gen(prompt, max_length=15, num_return_sequences=1, do_sample=False)
    return out[0]["generated_text"].strip().replace('"', '')

print("🎯 Generating persona titles...")

# Ensure cluster id column exists
if "predicted_cluster" not in cluster_summary.columns:
    cluster_summary.reset_index(inplace=True)
    if "index" in cluster_summary.columns:
        cluster_summary.rename(columns={"index": "predicted_cluster"}, inplace=True)

cluster_summary["persona_title"] = cluster_summary["persona_text_clean"].apply(make_title)

print("✅ Titles generated successfully!\n")
print(cluster_summary[["predicted_cluster", "persona_title"]].head(10))


# ============================================================
# 12️⃣ Chat with a Persona (Interactive Simulation)
# ============================================================

chat_model = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2")

def chat_with_cluster(cluster_id, user_input, max_length=150):
    """Talk to a persona cluster."""
    if "predicted_cluster" not in cluster_summary.columns:
        return "❌ Cluster column missing."

    persona_row = cluster_summary.loc[cluster_summary["predicted_cluster"] == cluster_id]
    if persona_row.empty:
        return f"❌ Cluster {cluster_id} not found."

    persona_summary = persona_row["persona_text_clean"].values[0]
    persona_title = persona_row["persona_title"].values[0]

    prompt = (
        f"You are a consumer persona named '{persona_title}'. "
        f"Your profile: {persona_summary}. "
        f"Answer conversationally and stay in character.\n\n"
        f"User: {user_input}\n{persona_title}:"
    )

    response = chat_model(prompt, max_new_tokens=max_length, do_sample=True, temperature=0.7)
    return response[0]["generated_text"].split(":", 1)[-1].strip()

# Example usage
print("\n💬 Example Chat:")
example_cluster = int(cluster_summary.sample(1)["predicted_cluster"].iloc[0])
user_query = "What kind of phone or streaming service do you prefer?"
print(f"🧩 Talking to Cluster {example_cluster}...\n")
print(chat_with_cluster(example_cluster, user_query))


#

# **CLEANING DISK**

In [ ]:
!df -h


In [ ]:
#CLEANING DISK
!df -h
%%bash
echo "Before cleanup:"
df -h

echo -e "\nCleaning pip and apt caches..."
rm -rf ~/.cache/pip
apt-get clean
rm -rf /var/lib/apt/lists/*

echo -e "\nCleaning model and library caches..."
rm -rf ~/.cache/huggingface
rm -rf ~/.cache/torch
rm -rf ~/.nv
rm -rf /usr/local/share/.cache

echo -e "\nCleaning temporary directories..."
rm -rf /tmp/*
rm -rf /var/tmp/*

echo -e "\nAfter cleanup:"
df -h


# **Considering all**

In [ ]:
# ============================================================
# Re-cluster + core-member selection + clean persona summaries (SAFE VERSION)
# ============================================================

# !pip install sentence-transformers chromadb transformers scikit-learn

from google.cloud import bigquery
import pandas as pd
import numpy as np
from collections import Counter
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples
from transformers import pipeline
import chromadb

# ---------------------------
# PARAMETERS
# ---------------------------
N_CLUSTERS = 10
MIN_CLUSTER_SIZE = 30
SILHOUETTE_Q = 0.30
SAMPLE_FRACTION = 0.001
MAX_PERSONA_SAMPLE = 100

# ---------------------------
# 1. Load sample data
# ---------------------------
client = bigquery.Client(project="stgw-shared-ai-dev")

query = f"""
SELECT *
FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles_imputed_only_cleaned`
WHERE RAND() < {SAMPLE_FRACTION}
LIMIT 5000
"""
df = client.query(query).to_dataframe()
print("Loaded sample rows:", df.shape)

# ---------------------------
# 2. Column Groups (A–I)
# ---------------------------
groups = {
    "demographics": [
        "gender_imputed","age_imputed","generations_imputed","marital_status_imputed",
        "urbanicity_imputed","education_individual_imputed","ethnicity_imputed","language_spoken_imputed"
    ],
    "financial": [
        "family_income_decile_imputed","average_household_income_imputed","net_worth_imputed",
        "credit_tier_imputed","credit_card_spend_imputed","credit_card_balance_imputed","financial_beliefs_imputed",
        "discretionary_spend_imputed","wealth_rating_imputed","family_level_of_investable_assets_imputed",
        "home_equity_imputed","loan_imputed","mortgage_loan_imputed","auto_loan_imputed",
        "investments_type_imputed","financial_institutions_imputed"
    ],
    "property_home": [
        "home_ownership_imputed","dwelling_unit_size_imputed","building_type_imputed",
        "property_size_imputed","property_realty_indicator_imputed","current_home_value_imputed",
        "number_of_bedrooms_imputed","number_of_bathrooms_imputed","heating_type_imputed","air_conditioning_imputed"
    ],
    "technology": [
        "cell_smartphones_tablets_imputed","mobile_devices_imputed","mobile_phone_brand_imputed",
        "mobile_plan_type_imputed","i_device_types_imputed","i_device_models_imputed","i_device_oss_imputed",
        "streaming_system_device_use_imputed","technology_imputed","computer_purchase_imputed",
        "tablets_imputed","i_app_taxonomies_imputed","apps_imputed"
    ],
    "shopping": [
        "shopping_imputed","stores_imputed","commerce_retail_shopping_imputed","department_discount_warehouse_imputed",
        "apparel_imputed","shoes_imputed","coupons_used_imputed","mail_purchases_imputed"
    ],
    "food_grocery": [c for c in df.columns if c.startswith("grocery_")],
    "lifestyle": [
        "leisure_activities_hobbies_imputed","health_and_fitness_imputed","green_living_imputed",
        "auto_imputed","sports_interests_imputed","entertainment_interests_imputed",
        "travel_interests_imputed","children_family_imputed"
    ],
    "media": [
        "television_shows_imputed","entertainment_sites_imputed","sports_sites_imputed",
        "news_sites_imputed","social_sites_imputed","i_iab_categories_imputed","movie_genre_saw_in_theater_imputed"
    ],
    "political_social": [
        "voting_behavior_imputed","political_affiliations_imputed","political_outlook_imputed",
        "religion_imputed","charitable_contributions_imputed"
    ]
}

# Only keep existing columns in df
for g in groups:
    groups[g] = [c for c in groups[g] if c in df.columns]

# Flatten all selected columns
all_signal_cols = [c for cols in groups.values() for c in cols]
print(f"Total usable columns for clustering: {len(all_signal_cols)}")

# ---------------------------
# 3. Build concise row persona text
# ---------------------------
def describe_section(df_row, cols, cap=3):
    vals = df_row[cols].dropna().astype(str).tolist()
    vals = [v for v in vals if v.lower() not in ("none","nan","__missing__")]
    uniq = []
    for v in vals:
        if v not in uniq:
            uniq.append(v)
        if len(uniq) >= cap:
            break
    return ", ".join(uniq) if uniq else ""

def build_row_persona(r):
    parts = []
    for theme, cols in groups.items():
        text = describe_section(r, cols, cap=3)
        if text:
            parts.append(f"{theme}: {text}")
    return " | ".join(parts)

df["row_persona_text"] = df.apply(build_row_persona, axis=1)
print("✅ Built concise row_persona_text")

# ---------------------------
# 4. Embeddings + Clustering
# ---------------------------
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
row_embeddings = embed_model.encode(df["row_persona_text"].astype(str).tolist(), show_progress_bar=True)

kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init=10)
df["recluster_label"] = kmeans.fit_predict(row_embeddings)
print("✅ Clustering complete")

# ---------------------------
# 5. Drop small clusters
# ---------------------------
valid_clusters = df["recluster_label"].value_counts()[lambda x: x >= MIN_CLUSTER_SIZE].index
df = df[df["recluster_label"].isin(valid_clusters)].copy()
print("✅ Valid clusters after filtering:", list(valid_clusters))

# ---------------------------
# 6. Core Members via Silhouette
# ---------------------------
if len(valid_clusters) > 1:
    sil = silhouette_samples(row_embeddings, df["recluster_label"])
    df["silhouette"] = sil
    df_core = df.groupby("recluster_label", group_keys=False).apply(
        lambda g: g[g["silhouette"] > g["silhouette"].quantile(SILHOUETTE_Q)]
    ).reset_index(drop=True)
else:
    df_core = df.copy()
print("✅ Core members retained:", df_core.shape)

# ---------------------------
# 7. Cluster summary via modes + top terms
# ---------------------------
def mode_safe(series):
    s = series.dropna().astype(str)
    return s.mode().iloc[0] if not s.mode().empty else None

def top_terms(series, n=3):
    vals = [v for v in series.dropna().astype(str).tolist() if v and v.lower() not in ("none","nan","__missing__")]
    if not vals:
        return "None"
    cnt = Counter(vals)
    return ", ".join([t for t,_ in cnt.most_common(n)])

cluster_summary = []
for cid, grp in df_core.groupby("recluster_label"):
    row = {"cluster": int(cid), "size": len(grp)}
    for c in all_signal_cols:
        row[c] = mode_safe(grp[c])
    row["persona_snippet"] = " | ".join(grp["row_persona_text"].head(MAX_PERSONA_SAMPLE).tolist())[:1000]
    cluster_summary.append(row)

cluster_summary = pd.DataFrame(cluster_summary).sort_values("cluster").reset_index(drop=True)
print("✅ Built cluster summaries:", cluster_summary.shape)

# ---------------------------
# 8. Summarize cluster persona text
# ---------------------------
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=-1)

def make_prompt_and_summarize(r):
    prompt = f"""
    Gender: {r.get('gender_imputed','Unknown')}.
    Age: {r.get('age_imputed','Unknown')}.
    Urbanicity: {r.get('urbanicity_imputed','Unknown')}.
    Income: {r.get('family_income_decile_imputed','Unknown')}.
    Credit tier: {r.get('credit_tier_imputed','Unknown')}.
    Key traits: {r.get('persona_snippet','')[:400]}
    """
    try:
        out = summarizer(prompt, max_length=80, min_length=20, truncation=True)
        return out[0]['summary_text']
    except:
        return prompt[:200]

cluster_summary["persona_text_clean"] = cluster_summary.apply(make_prompt_and_summarize, axis=1)
print("✅ Summarization done for clusters")

# ---------------------------
# 9. Store in Chroma
# ---------------------------
chroma_client = chromadb.Client()
try:
    chroma_client.delete_collection("cluster_personas_clean_v2")
except:
    pass
collection = chroma_client.create_collection("cluster_personas_clean_v2")

cluster_embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
cluster_embeddings = cluster_embed_model.encode(cluster_summary["persona_text_clean"].tolist(), show_progress_bar=True)

collection.add(
    ids=[str(i) for i in cluster_summary["cluster"].tolist()],
    embeddings=[list(map(float, e)) for e in cluster_embeddings],
    documents=cluster_summary["persona_text_clean"].tolist(),
    metadatas=cluster_summary[["cluster","size"]].to_dict(orient="records")
)
print("✅ Stored", collection.count(), "clusters in Chroma.")

# ---------------------------
# 10. Query example
# ---------------------------
def query_clusters_natural(user_query, top_k=4):
    q_emb = cluster_embed_model.encode([user_query])[0].tolist()
    res = collection.query(query_embeddings=[q_emb], n_results=top_k)
    output = []
    for i, cid in enumerate(res["ids"][0]):
        meta = res["metadatas"][0][i]
        doc = res["documents"][0][i]
        cs = cluster_summary[cluster_summary["cluster"]==int(cid)].iloc[0].to_dict()
        output.append({
            "cluster": int(cid),
            "score": res["distances"][0][i],
            "size": meta.get("size"),
            "age": cs.get("age_imputed"),
            "gender": cs.get("gender_imputed"),
            "urbanicity": cs.get("urbanicity_imputed"),
            "summary": doc
        })
    return pd.DataFrame(output)

print("\n✅ Example Query:")
print(query_clusters_natural("Who uses mobile banking and streaming apps?", top_k=4).to_string(index=False))


# **V2**

In [ ]:
# ============================================================
# Re-cluster + core-member selection + clean persona summaries (SAFE VERSION)
# ============================================================

# !pip install sentence-transformers chromadb transformers scikit-learn

from google.cloud import bigquery
import pandas as pd
import numpy as np
from collections import Counter
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples
from transformers import pipeline
import chromadb

# ---------------------------
# PARAMETERS
# ---------------------------
N_CLUSTERS = 10
MIN_CLUSTER_SIZE = 30
SILHOUETTE_Q = 0.30
SAMPLE_FRACTION = 0.001
MAX_PERSONA_SAMPLE = 100

# ---------------------------
# 1. Load sample data
# ---------------------------
client = bigquery.Client(project="stgw-shared-ai-dev")

query = f"""
SELECT *
FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles_imputed_only_cleaned`
WHERE RAND() < {SAMPLE_FRACTION}
LIMIT 5000
"""
df = client.query(query).to_dataframe()
print("Loaded sample rows:", df.shape)

# ---------------------------
# 2. Column Groups (A–I)
# ---------------------------
groups = {
    "demographics": [
        "gender_imputed","age_imputed","generations_imputed","marital_status_imputed",
        "urbanicity_imputed","education_individual_imputed","ethnicity_imputed","language_spoken_imputed"
    ],
    "financial": [
        "family_income_decile_imputed","average_household_income_imputed","net_worth_imputed",
        "credit_tier_imputed","credit_card_spend_imputed","credit_card_balance_imputed","financial_beliefs_imputed",
        "discretionary_spend_imputed","wealth_rating_imputed","family_level_of_investable_assets_imputed",
        "home_equity_imputed","loan_imputed","mortgage_loan_imputed","auto_loan_imputed",
        "investments_type_imputed","financial_institutions_imputed"
    ],
    "property_home": [
        "home_ownership_imputed","dwelling_unit_size_imputed","building_type_imputed",
        "property_size_imputed","property_realty_indicator_imputed","current_home_value_imputed",
        "number_of_bedrooms_imputed","number_of_bathrooms_imputed","heating_type_imputed","air_conditioning_imputed"
    ],
    "technology": [
        "cell_smartphones_tablets_imputed","mobile_devices_imputed","mobile_phone_brand_imputed",
        "mobile_plan_type_imputed","i_device_types_imputed","i_device_models_imputed","i_device_oss_imputed",
        "streaming_system_device_use_imputed","technology_imputed","computer_purchase_imputed",
        "tablets_imputed","i_app_taxonomies_imputed","apps_imputed"
    ],
    "shopping": [
        "shopping_imputed","stores_imputed","commerce_retail_shopping_imputed","department_discount_warehouse_imputed",
        "apparel_imputed","shoes_imputed","coupons_used_imputed","mail_purchases_imputed"
    ],
    "food_grocery": [c for c in df.columns if c.startswith("grocery_")],
    "lifestyle": [
        "leisure_activities_hobbies_imputed","health_and_fitness_imputed","green_living_imputed",
        "auto_imputed","sports_interests_imputed","entertainment_interests_imputed",
        "travel_interests_imputed","children_family_imputed"
    ],
    "media": [
        "television_shows_imputed","entertainment_sites_imputed","sports_sites_imputed",
        "news_sites_imputed","social_sites_imputed","i_iab_categories_imputed","movie_genre_saw_in_theater_imputed"
    ],
    "political_social": [
        "voting_behavior_imputed","political_affiliations_imputed","political_outlook_imputed",
        "religion_imputed","charitable_contributions_imputed"
    ]
}

# Only keep existing columns in df
for g in groups:
    groups[g] = [c for c in groups[g] if c in df.columns]

# Flatten all selected columns
all_signal_cols = [c for cols in groups.values() for c in cols]
print(f"Total usable columns for clustering: {len(all_signal_cols)}")

# ---------------------------
# 3. Build concise row persona text
# ---------------------------
def describe_section(df_row, cols, cap=3):
    vals = df_row[cols].dropna().astype(str).tolist()
    vals = [v for v in vals if v.lower() not in ("none","nan","__missing__")]
    uniq = []
    for v in vals:
        if v not in uniq:
            uniq.append(v)
        if len(uniq) >= cap:
            break
    return ", ".join(uniq) if uniq else ""

def build_row_persona(r):
    parts = []
    for theme, cols in groups.items():
        text = describe_section(r, cols, cap=3)
        if text:
            parts.append(f"{theme}: {text}")
    return " | ".join(parts)

df["row_persona_text"] = df.apply(build_row_persona, axis=1)
print("✅ Built concise row_persona_text")

# ---------------------------
# 4. Embeddings + Clustering
# ---------------------------
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
row_embeddings = embed_model.encode(df["row_persona_text"].astype(str).tolist(), show_progress_bar=True)

kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init=10)
df["recluster_label"] = kmeans.fit_predict(row_embeddings)
print("✅ Clustering complete")

# ---------------------------
# 5. Drop small clusters
# ---------------------------
valid_clusters = df["recluster_label"].value_counts()[lambda x: x >= MIN_CLUSTER_SIZE].index
df = df[df["recluster_label"].isin(valid_clusters)].copy()
print("✅ Valid clusters after filtering:", list(valid_clusters))

# ---------------------------
# 6. Core Members via Silhouette
# ---------------------------
if len(valid_clusters) > 1:
    sil = silhouette_samples(row_embeddings, df["recluster_label"])
    df["silhouette"] = sil
    df_core = (
        df.groupby("recluster_label", group_keys=False)
        .apply(lambda g: g[g["silhouette"] > g["silhouette"].quantile(SILHOUETTE_Q)])
        .reset_index(drop=True)
    )
else:
    df_core = df.copy()

# Ensure recluster_label is present
if "recluster_label" not in df_core.columns:
    df_core = df_core.merge(df[["recluster_label"]], left_index=True, right_index=True, how="left")

print("✅ Core members retained:", df_core.shape)
print("✅ recluster_label column restored:", "recluster_label" in df_core.columns)


# ---------------------------
# 7. Cluster summary via modes + top terms
# ---------------------------
def mode_safe(series):
    s = series.dropna().astype(str)
    return s.mode().iloc[0] if not s.mode().empty else None

def top_terms(series, n=3):
    vals = [v for v in series.dropna().astype(str).tolist() if v and v.lower() not in ("none","nan","__missing__")]
    if not vals:
        return "None"
    cnt = Counter(vals)
    return ", ".join([t for t,_ in cnt.most_common(n)])

cluster_summary = []
for cid, grp in df_core.groupby("recluster_label"):
    row = {"cluster": int(cid), "size": len(grp)}
    for c in all_signal_cols:
        row[c] = mode_safe(grp[c])
    row["persona_snippet"] = " | ".join(grp["row_persona_text"].head(MAX_PERSONA_SAMPLE).tolist())[:1000]
    cluster_summary.append(row)

cluster_summary = pd.DataFrame(cluster_summary).sort_values("cluster").reset_index(drop=True)
print("✅ Built cluster summaries:", cluster_summary.shape)

# ---------------------------
# 8. Behavioral enrichment summary (SAFE FIX)
# ---------------------------
print("🧩 Adding behavioral summaries...")

def summarize_behavior_group(g):
    tags = []
    for col in g.columns:
        vals = g[col].dropna().astype(str).unique().tolist()[:3]
        for v in vals:
            if v.lower() not in ("nan", "none", "__missing__", "na"):
                tags.append(f"{col}: {v}")
    return ", ".join(tags[:10])

behavior_summary = df_core[[c for c in all_signal_cols if "imputed" in c] + ["recluster_label"]]
behavior_tags = (
    behavior_summary.groupby("recluster_label")
    .apply(summarize_behavior_group)
    .reset_index(name="top_behaviors")
)

cluster_summary = cluster_summary.merge(behavior_tags, left_on="cluster", right_on="recluster_label", how="left").drop(columns=["recluster_label"])
print("✅ Identified behavioral columns and added summaries.")

# ---------------------------
# 9. Summarize cluster persona text
# ---------------------------
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=-1)

def make_prompt_and_summarize(r):
    prompt = f"""
    Gender: {r.get('gender_imputed','Unknown')}.
    Age: {r.get('age_imputed','Unknown')}.
    Urbanicity: {r.get('urbanicity_imputed','Unknown')}.
    Income: {r.get('family_income_decile_imputed','Unknown')}.
    Credit tier: {r.get('credit_tier_imputed','Unknown')}.
    Key traits: {r.get('persona_snippet','')[:400]}
    """
    try:
        out = summarizer(prompt, max_length=80, min_length=20, truncation=True)
        return out[0]['summary_text']
    except:
        return prompt[:200]

cluster_summary["persona_text_clean"] = cluster_summary.apply(make_prompt_and_summarize, axis=1)
print("✅ Summarization done for clusters")

# ---------------------------
# 10. Store in Chroma
# ---------------------------
chroma_client = chromadb.Client()
try:
    chroma_client.delete_collection("cluster_personas_clean_v2")
except:
    pass
collection = chroma_client.create_collection("cluster_personas_clean_v2")

cluster_embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
cluster_embeddings = cluster_embed_model.encode(cluster_summary["persona_text_clean"].tolist(), show_progress_bar=True)

collection.add(
    ids=[str(i) for i in cluster_summary["cluster"].tolist()],
    embeddings=[list(map(float, e)) for e in cluster_embeddings],
    documents=cluster_summary["persona_text_clean"].tolist(),
    metadatas=cluster_summary[["cluster","size"]].to_dict(orient="records")
)
print("✅ Stored", collection.count(), "clusters in Chroma.")

# ---------------------------
# 11. Query example
# ---------------------------
def query_clusters_natural(user_query, top_k=4):
    q_emb = cluster_embed_model.encode([user_query])[0].tolist()
    res = collection.query(query_embeddings=[q_emb], n_results=top_k)
    output = []
    for i, cid in enumerate(res["ids"][0]):
        meta = res["metadatas"][0][i]
        doc = res["documents"][0][i]
        cs = cluster_summary[cluster_summary["cluster"]==int(cid)].iloc[0].to_dict()
        output.append({
            "cluster": int(cid),
            "score": res["distances"][0][i],
            "size": meta.get("size"),
            "age": cs.get("age_imputed"),
            "gender": cs.get("gender_imputed"),
            "urbanicity": cs.get("urbanicity_imputed"),
            "summary": doc
        })
    return pd.DataFrame(output)

print("\n✅ Example Query:")
print(query_clusters_natural("Who uses mobile banking and streaming apps?", top_k=4).to_string(index=False))


#**ONGOING PART**

In [ ]:
from google.cloud import bigquery

# --- CONFIGURATION ---
PROJECT = "stgw-shared-ai-dev"
DATASET = "parquet_data_set"
TABLE_NAME = "2501_new_parquet_table" # Note: Using the original table name as requested by the user

# Full table references
FULL_TABLE_REF = f"{PROJECT}.{DATASET}.{TABLE_NAME}"

client = bigquery.Client(project=PROJECT)

def get_column_count():
    """Fetches the total number of columns from the INFORMATION_SCHEMA."""
    q_col = f"""
    SELECT
        COUNT(column_name)
    FROM `{PROJECT}.{DATASET}`.INFORMATION_SCHEMA.COLUMNS
    WHERE table_name = '{TABLE_NAME}';
    """
    print(f"Querying schema for column count of table: {FULL_TABLE_REF}...")

    try:
        rows = client.query(q_col).result()
        for r in rows:
            return r[0]
    except Exception as e:
        print(f"An error occurred while fetching column count: {e}")
        return None

def get_row_count():
    """Fetches the total number of rows using COUNT(*)."""
    q_row = f"""
    SELECT
        COUNT(*)
    FROM `{FULL_TABLE_REF}`;
    """
    print(f"Querying table for total row count of table: {FULL_TABLE_REF}...")

    try:
        rows = client.query(q_row).result()
        for r in rows:
            return r[0]
    except Exception as e:
        print(f"An error occurred while fetching row count: {e}")
        return None

def main():
    """Main execution logic to print the table counts."""
    col_count = get_column_count()
    row_count = get_row_count()

    print("\n--- RESULTS ---")
    if col_count is not None:
        print(f"Column Count: {col_count}")

    if row_count is not None:
        print(f"Row Count: {row_count}")

    if col_count is None or row_count is None:
        print("Could not retrieve all necessary information. Please check the error messages above.")
    print("---------------")

if __name__ == "__main__":
    main()

In [ ]:
from google.cloud import bigquery
import json
import math

# --- CONFIGURATION (Use the same values as your imputation script) ---
PROJECT = "stgw-shared-ai-dev"
DATASET = "parquet_data_set"
SAMPLE_TABLE_NAME = "2501_new_parquet_table"
IMPUTED_TABLE_NAME = f"{SAMPLE_TABLE_NAME}_imputed"

IMPUTED_TABLE = f"{PROJECT}.{DATASET}.{IMPUTED_TABLE_NAME}"

client = bigquery.Client(project=PROJECT)

def get_reportable_columns():
    """
    Fetches the column names and types from the imputed table's schema.
    It returns only the columns that were processed and had a missing flag created
    (i.e., non-flag columns that are NOT TIMESTAMP).
    """
    q = f"""
    SELECT column_name, data_type
    FROM `{PROJECT}.{DATASET}`.INFORMATION_SCHEMA.COLUMNS
    WHERE table_name = '{IMPUTED_TABLE_NAME}'
    AND NOT STARTS_WITH(column_name, 'is_missing_')
    ORDER BY ordinal_position
    """
    print(f"Querying imputed table schema to find reportable columns...")
    rows = client.query(q).result()

    reportable_cols = []
    skipped_cols = []

    # Filter out the TIMESTAMP columns, as they do not have an is_missing_ flag.
    for r in rows:
        col_name = r.column_name
        data_type = r.data_type.upper()

        if data_type == 'TIMESTAMP':
            skipped_cols.append(col_name)
        else:
            reportable_cols.append(col_name)

    print(f"Skipped {len(skipped_cols)} TIMESTAMP columns for reporting: {', '.join(skipped_cols)}")
    return reportable_cols

def generate_missing_percentage_query(reportable_cols):
    """
    Generates the massive BigQuery SQL statement to calculate the percentage
    of missing values for all reportable columns using their indicator flags.
    """
    select_expressions = []

    # 1. Create expressions for each column
    for col_name in reportable_cols:
        flag_col = f"is_missing_{col_name}"

        # Count of missing values (where the flag is 1)
        count_expr = f"SUM(t.{flag_col}) AS {col_name}_missing_count"

        # Percentage of missing values: (Count * 100.0) / Total Rows
        percent_expr = f"(SUM(t.{flag_col}) * 100.0 / COUNT(*)) AS {col_name}_missing_percent"

        select_expressions.append(count_expr)
        select_expressions.append(percent_expr)

    # Combine all expressions into the final SELECT statement
    select_sql = ",\n  ".join(select_expressions)

    # The final query counts rows and uses the SUM(is_missing_flag) to get the null count
    report_sql = f"""
    SELECT
      COUNT(*) AS total_rows,
      {select_sql}
    FROM `{IMPUTED_TABLE}` AS t
    LIMIT 1;
    """
    return report_sql

def run_missing_percentage_report():
    """Main function to run the report."""
    # Now calls the updated function to get only columns with flags
    reportable_cols = get_reportable_columns()
    print(f"Found {len(reportable_cols)} reportable columns for missing value analysis.")

    # Due to the large number of columns, this query may still be very long.
    # BigQuery usually handles these "wide" reporting queries well.
    report_query = generate_missing_percentage_query(reportable_cols)

    print("\nRunning Missing Percentage Report Query...")

    # Execute the query
    job = client.query(report_query)

    # Fetch the single resulting row
    result_row = list(job.result())[0]

    # Process and print the results
    report = {}
    total_rows = result_row['total_rows']

    print(f"\n--- Missing Percentage Report (Total Rows: {total_rows:,}) ---")

    # Iterate through column names to group count and percentage
    for col_name in reportable_cols:
        count_key = f"{col_name}_missing_count"
        percent_key = f"{col_name}_missing_percent"

        missing_count = result_row.get(count_key)
        missing_percent = result_row.get(percent_key)

        # Check if the column was successfully found in the report (should always be true now)
        if missing_count is not None:
            # Use f-strings for formatted output
            print(f"**{col_name}**: Missing Count = {missing_count:,.0f}, Percentage = {missing_percent:.2f}%")

            report[col_name] = {
                'count': missing_count,
                'percent': missing_percent
            }

    print("---------------------------------------------------------------")
    return report

# Execute the main function
if __name__ == "__main__":
    missing_data_report = run_missing_percentage_report()
    # You can now use the 'missing_data_report' dictionary for further analysis
    # e.g., print(json.dumps(missing_data_report, indent=2))

In [ ]:
from google.cloud import bigquery
import pandas as pd
import math
import json # Added import for completeness if user decides to use it later

# --- CONFIGURATION (Use the same values as your imputation script) ---
PROJECT = "stgw-shared-ai-dev"
DATASET = "parquet_data_set"
SAMPLE_TABLE_NAME = "2501_new_parquet_table"
IMPUTED_TABLE_NAME = f"{SAMPLE_TABLE_NAME}_imputed"

IMPUTED_TABLE = f"{PROJECT}.{DATASET}.{IMPUTED_TABLE_NAME}"

client = bigquery.Client(project=PROJECT)

def get_reportable_columns():
    """
    Fetches the column names and types from the imputed table's schema.
    It returns only the columns that were processed and had a missing flag created
    (i.e., non-flag columns that are NOT TIMESTAMP).
    """
    q = f"""
    SELECT column_name, data_type
    FROM `{PROJECT}.{DATASET}`.INFORMATION_SCHEMA.COLUMNS
    WHERE table_name = '{IMPUTED_TABLE_NAME}'
    AND NOT STARTS_WITH(column_name, 'is_missing_')
    ORDER BY ordinal_position
    """
    print(f"Querying imputed table schema to find reportable columns...")
    rows = client.query(q).result()

    reportable_cols = []
    skipped_cols = []

    # Filter out the TIMESTAMP columns, as they do not have an is_missing_ flag.
    for r in rows:
        col_name = r.column_name
        data_type = r.data_type.upper()

        if data_type == 'TIMESTAMP':
            skipped_cols.append(col_name)
        else:
            reportable_cols.append(col_name)

    if skipped_cols:
        print(f"Skipped {len(skipped_cols)} TIMESTAMP columns for reporting: {', '.join(skipped_cols)}")

    return reportable_cols

def generate_missing_percentage_query(reportable_cols):
    """
    Generates the BigQuery SQL statement to calculate the count and percentage
    of missing values for all reportable columns.
    """
    select_expressions = []

    # Generate SUM and Percentage expressions for all reportable columns
    for col_name in reportable_cols:
        flag_col = f"is_missing_{col_name}"

        count_expr = f"SUM(t.{flag_col}) AS {col_name}_missing_count"
        percent_expr = f"(SUM(t.{flag_col}) * 100.0 / COUNT(*)) AS {col_name}_missing_percent"

        select_expressions.append(count_expr)
        select_expressions.append(percent_expr)

    select_sql = ",\n  ".join(select_expressions)

    report_sql = f"""
    SELECT
      COUNT(*) AS total_rows,
      {select_sql}
    FROM `{IMPUTED_TABLE}` AS t
    LIMIT 1;
    """
    return report_sql

def run_missing_percentage_report():
    """
    Main function to run the report and return a pandas DataFrame.
    """
    # Use the updated function name
    reportable_cols = get_reportable_columns()
    print(f"Found {len(reportable_cols)} reportable columns for missing value analysis.")

    report_query = generate_missing_percentage_query(reportable_cols)

    print("\nRunning Missing Percentage Report Query...")

    # Execute the query and load results directly into a DataFrame
    # Note: This is efficient for the single row result.
    df_raw = client.query(report_query).to_dataframe()

    # --- Process the wide DataFrame into a long, readable report ---

    total_rows = df_raw.iloc[0]['total_rows']
    report_data = []

    for col_name in reportable_cols:
        count_key = f"{col_name}_missing_count"
        percent_key = f"{col_name}_missing_percent"

        missing_count = df_raw.iloc[0][count_key]
        missing_percent = df_raw.iloc[0][percent_key]

        report_data.append({
            'Column Name': col_name,
            'Missing Count': missing_count,
            'Missing Percentage': missing_percent
        })

    # Create the final clean DataFrame
    df_report = pd.DataFrame(report_data)

    print(f"\nReport generated for {len(df_report)} columns (Total Rows: {total_rows:,})")
    print("--------------------------------------------------------------------------------")

    # Print the DataFrame header and first few rows for immediate feedback
    print(df_report.head())
    print("...")

    return df_report

# --- Example Notebook Execution ---
if __name__ == "__main__":
    df_missing_report = run_missing_percentage_report()

In [ ]:
import pandas as pd
import json

# WARNING: This script assumes 'df_missing_report' DataFrame exists in the environment
# from the execution of the previous reporting step.

if 'df_missing_report' not in locals():
    print("Error: The DataFrame 'df_missing_report' was not found.")
    print("Please ensure you have run the 'run_missing_percentage_report()' function first.")
else:
    df = df_missing_report.copy()

    # 1. Calculate the Completeness Percentage
    df['Completeness Percentage'] = 100.0 - df['Missing Percentage']

    # --- Criteria 1: 85% and Above Completeness ---
    # Equivalent to: Missing Percentage <= 15%
    high_completeness_df = df[df['Completeness Percentage'] >= 85]
    high_completeness_cols = high_completeness_df['Column Name'].tolist()

    # --- Criteria 2: 40% to 84% Completeness ---
    # Equivalent to: 16% <= Missing Percentage <= 60%
    medium_completeness_df = df[
        (df['Completeness Percentage'] >= 40) &
        (df['Completeness Percentage'] < 85)
    ]
    medium_completeness_cols = medium_completeness_df['Column Name'].tolist()

    # --- Output Results ---

    print("\n=======================================================")
    print(f"| High Completeness Columns (85%+): {len(high_completeness_cols):<4} columns |")
    print("=======================================================")
    print(json.dumps(high_completeness_cols, indent=2))

    print("\n=======================================================")
    print(f"| Medium Completeness Columns (40% - 84%): {len(medium_completeness_cols):<4} columns |")
    print("=======================================================")
    print(json.dumps(medium_completeness_cols, indent=2))
    print("\n-------------------------------------------------------")

    # Also save the filtered DataFrames for review
    high_completeness_df.to_csv('high_completeness_report(>85%).csv', index=False)
    medium_completeness_df.to_csv('medium_completeness_report(40-84%).csv', index=False)
    print(f"✅ Filtered results saved to 'high_completeness_report(>85%).csv' and 'medium_completeness_report(40-60%).csv'")

In [ ]:
!pip install google-cloud-bigquery pandas --quiet
from google.cloud import bigquery
import pandas as pd
import json

# --- CONFIGURATION ---
PROJECT = "stgw-shared-ai-dev"
DATASET = "parquet_data_set"
SAMPLE_TABLE_NAME = "2501_new_parquet_table"

# Tables involved
IMPUTED_TABLE_NAME = f"{SAMPLE_TABLE_NAME}_imputed"
IMPUTED_TABLE = f"{PROJECT}.{DATASET}.{IMPUTED_TABLE_NAME}"

# New destination table for the filtered data (columns with >= 40% completeness)
FILTERED_TABLE_NAME = f"{SAMPLE_TABLE_NAME}_filtered_complete"
FILTERED_TABLE = f"{PROJECT}.{DATASET}.{FILTERED_TABLE_NAME}"

# Filtering Threshold: Keep columns if Missing Percentage is <= 60.0
MAX_MISSING_PERCENT = 60.0

# --- 1. Initialize BigQuery Client ---
try:
    client = bigquery.Client(project=PROJECT)
    print(f"✅ BigQuery client initialized for project: {PROJECT}")
except Exception as e:
    print(f"❌ Failed to initialize BigQuery client. Error: {e}")
    exit()

def get_reportable_columns():
    """
    Fetches non-TIMESTAMP, non-flag column names from the imputed table's schema.
    """
    q = f"""
    SELECT column_name, data_type
    FROM `{PROJECT}.{DATASET}`.INFORMATION_SCHEMA.COLUMNS
    WHERE table_name = '{IMPUTED_TABLE_NAME}'
    AND NOT STARTS_WITH(column_name, 'is_missing_')
    ORDER BY ordinal_position
    """
    rows = client.query(q).result()

    reportable_cols = []
    # Filter out the TIMESTAMP columns, as they do not have an is_missing_ flag.
    for r in rows:
        if r.data_type.upper() != 'TIMESTAMP':
            reportable_cols.append(r.column_name)

    return reportable_cols

def generate_missing_percentage_query(reportable_cols):
    """
    Generates the massive BigQuery SQL statement to calculate the count and percentage
    of missing values for all reportable columns using their indicator flags.
    """
    select_expressions = []
    for col_name in reportable_cols:
        flag_col = f"is_missing_{col_name}"
        # We only need the percentage for filtering in the report, but calculate both for completeness
        count_expr = f"SUM(t.{flag_col}) AS {col_name}_missing_count"
        percent_expr = f"(SUM(t.{flag_col}) * 100.0 / COUNT(*)) AS {col_name}_missing_percent"
        select_expressions.append(count_expr)
        select_expressions.append(percent_expr)

    select_sql = ",\n  ".join(select_expressions)

    report_sql = f"""
    SELECT
      COUNT(*) AS total_rows,
      {select_sql}
    FROM `{IMPUTED_TABLE}` AS t
    LIMIT 1;
    """
    return report_sql

def run_and_filter_columns():
    """Main function to run the report, apply the filter, and create the new BigQuery table."""
    reportable_cols = get_reportable_columns()
    total_initial_cols = len(reportable_cols)
    print(f"--- 2. Initializing Filter ---")
    print(f"Total reportable columns found in {IMPUTED_TABLE_NAME}: {total_initial_cols}")
    print(f"Applying filter: Missing Percentage <= {MAX_MISSING_PERCENT:.1f}% (Completeness >= {100.0 - MAX_MISSING_PERCENT:.1f}%)")

    # 3. Get the Missing Percentage Report
    report_query = generate_missing_percentage_query(reportable_cols)
    df_raw = client.query(report_query).to_dataframe()

    # --- 4. Process Results and Apply Filter ---

    cols_to_keep = []

    for col_name in reportable_cols:
        percent_key = f"{col_name}_missing_percent"

        # Get the missing percentage for the column
        missing_percent = df_raw.iloc[0][percent_key]

        if missing_percent <= MAX_MISSING_PERCENT:
            cols_to_keep.append(col_name)
            # Add the missing flag associated with the column as well
            cols_to_keep.append(f"is_missing_{col_name}")

    # Always keep the identifier column if present (assuming one named 'id')
    # If a specific ID column is known, it should be hardcoded. Since it's not, we'll try to keep common ones.
    if 'id' in df_raw.columns and 'id' not in cols_to_keep:
        cols_to_keep.insert(0, 'id')

    # Check for TIMESTAMP columns in the imputed table and keep them (they were filtered out of reportable_cols)
    # We must explicitly query the schema for them again
    timestamp_cols = [
        r.column_name for r in client.query(f"""
        SELECT column_name
        FROM `{PROJECT}.{DATASET}`.INFORMATION_SCHEMA.COLUMNS
        WHERE table_name = '{IMPUTED_TABLE_NAME}' AND data_type = 'TIMESTAMP'
        """).result()
    ]

    for col in timestamp_cols:
        if col not in cols_to_keep:
            cols_to_keep.append(col)

    # Remove duplicates and ensure the ID/TIMESTAMP columns are at the start
    cols_to_keep = list(dict.fromkeys(cols_to_keep))

    # --- 5. Generate and Execute CREATE TABLE AS SELECT Query ---

    # Build the select clause
    select_clause = ",\n  ".join([f't.`{col}`' for col in cols_to_keep])

    create_table_sql = f"""
    CREATE OR REPLACE TABLE
      `{FILTERED_TABLE}`
    AS
    SELECT
      {select_clause}
    FROM
      `{IMPUTED_TABLE}` AS t;
    """

    # 6. Summary and Execution
    included_feature_count = int(len(cols_to_keep) / 2) # Divide by 2 because we kept feature + flag
    discarded_feature_count = total_initial_cols - included_feature_count

    print(f"\n--- 6. Execution Summary ---")
    print(f"✅ Columns Kept (Completeness >= {100.0 - MAX_MISSING_PERCENT:.1f}%): {included_feature_count} features (+ flags/timestamps)")
    print(f"❌ Columns Discarded (Completeness < {100.0 - MAX_MISSING_PERCENT:.1f}%): {discarded_feature_count} features")
    print(f"Total Columns Selected for new table: {len(cols_to_keep)}")

    print(f"\nSubmitting job to create filtered table: {FILTERED_TABLE}...")

    # Execute the query
    job = client.query(create_table_sql)
    job.result()

    print("\n========================================================")
    print("🎉 SUCCESS! The filtered table has been created.")
    print(f"Destination Table: {FILTERED_TABLE}")
    print("========================================================")

    # 7. Verification
    print(f"\nVerifying the number of columns in the new table...")
    new_col_count = client.query(f"""
        SELECT count(*)
        FROM `{PROJECT}.{DATASET}`.INFORMATION_SCHEMA.COLUMNS
        WHERE table_name = '{FILTERED_TABLE_NAME}'
    """).to_dataframe().iloc[0][0]

    print(f"Final column count in {FILTERED_TABLE_NAME}: {new_col_count}")

    return {
        'total_initial': total_initial_cols,
        'included': included_feature_count,
        'discarded': discarded_feature_count,
        'new_table': FILTERED_TABLE
    }


# Execute the main function
if __name__ == "__main__":
    report = run_and_filter_columns()

    print("\n--- Final Counts ---")
    print(f"Initial Candidate Features: {report['total_initial']}")
    print(f"Features Included (Completeness >= 40%): {report['included']}")
    print(f"Features Discarded (Completeness < 40%): {report['discarded']}")
    print(f"New Filtered Table: {report['new_table']}")

In [ ]:
!pip install google-cloud-bigquery pandas-gbq --quiet
from google.cloud import bigquery
import pandas as pd
import pandas_gbq

# --- CONFIGURATION ---
PROJECT = "stgw-shared-ai-dev"
DATASET = "parquet_data_set"

# --- UPDATED SOURCE TABLE NAME (The table with < 60% missing data) ---
SOURCE_TABLE = "2501_new_parquet_table_filtered_complete"

# --- NEW CONFIGURATION: Destination table for the stats ---
DEST_TABLE = f"{SOURCE_TABLE}_profiling_stats_core" # Changed destination table name to avoid conflict
DESTINATION = f"{DATASET}.{DEST_TABLE}"
# -----------------------------------------------------------

# --- 1️⃣ Connect to BigQuery ---
client = bigquery.Client(project=PROJECT)

# --- 2️⃣ Table reference ---
dataset_ref = f"{client.project}.{DATASET}"
SOURCE_TABLE_REF = f"{dataset_ref}.{SOURCE_TABLE}"

# --- 3️⃣ Get column names & datatypes (MODIFIED TO EXCLUDE FLAGS) ---
query_columns = f"""
SELECT column_name, data_type
FROM `{dataset_ref}.INFORMATION_SCHEMA.COLUMNS`
WHERE table_name = '{SOURCE_TABLE}'
  AND NOT STARTS_WITH(column_name, 'is_missing_') -- <<< KEY CHANGE HERE
ORDER BY ordinal_position
"""
cols = client.query(query_columns).to_dataframe()

print(f"✅ Found {len(cols)} columns to profile in {SOURCE_TABLE} (Expecting ~850 columns).")

# --- 4️⃣ Safe quoting function ---
def safe(col):
    return f"`{col.replace('`', '')}`"

# --- 5️⃣ Loop over columns and compute stats ---
profiles = []
# This process is necessary to profile all columns individually.
print("Starting feature profiling for core features only...")
for index, row in cols.iterrows():
    col, dtype = row["column_name"], row["data_type"]
    cq = safe(col)

    # Profiling query structure
    q = f"""
    SELECT
      '{col}' AS column_name,
      '{dtype}' AS data_type,
      COUNT(*) AS total_rows,
      COUNT(DISTINCT {cq}) AS unique_values,
      ROUND(COUNT(DISTINCT {cq}) * 100.0 / COUNT(*), 2) AS uniqueness_pct,
      -- Convert the top 3 values array to a JSON string
      TO_JSON_STRING((
        SELECT ARRAY_AGG(STRUCT(val, freq) ORDER BY freq DESC LIMIT 3)
        FROM (
          SELECT CAST({cq} AS STRING) AS val, COUNT(*) AS freq
          FROM `{SOURCE_TABLE_REF}`
          WHERE {cq} IS NOT NULL
          GROUP BY 1
          ORDER BY freq DESC
          LIMIT 3
        )
      )) AS top_values_json,
      -- Calculate Imbalance Ratio (frequency of the most common value divided by total non-null count)
      (
        SELECT SAFE_DIVIDE(MAX(freq), SUM(freq))
        FROM (
          SELECT {cq} AS val, COUNT(*) AS freq
          FROM `{SOURCE_TABLE_REF}`
          WHERE {cq} IS NOT NULL
          GROUP BY 1
        )
      ) AS imbalance_ratio
    FROM `{SOURCE_TABLE_REF}`
    LIMIT 1
    """

    try:
        subdf = client.query(q).to_dataframe()
        profiles.append(subdf)
    except Exception as e:
        print(f"⚠️ Skipping '{col}' due to query error: {e}")
        continue

# --- 6️⃣ Combine all column profiles ---
if profiles:
    df_profile = pd.concat(profiles, ignore_index=True)
else:
    raise RuntimeError("No valid column profiles generated.")

# --- 7️⃣ Recommendations based on value diversity ---
def rec(row):
    # This logic applies well to categorical and even continuous data for quick assessment
    if row.unique_values <= 1:
        return "⚠️ Constant — drop"
    if row.unique_values < 10:
        return "✅ Good categorical"
    if row.unique_values < 50:
        return "ℹ️ Moderate — check imbalance"
    if row.unique_values < 200:
        return "🧩 Consider binning/grouping"
    return "📉 Continuous / high-cardinality"

df_profile["recommendation"] = df_profile.apply(rec, axis=1)

# --- 8️⃣ Save the summarized profile to BigQuery ---

print(f"\nSubmitting DataFrame to BigQuery table: **{DESTINATION}**")

pandas_gbq.to_gbq(
    df_profile,
    destination_table=DESTINATION,
    project_id=PROJECT,
    if_exists='replace' # Overwrite the table if it already exists
)

print(f"🎉 **Successfully saved the stats table to BigQuery:** {client.project}.{DESTINATION}")

In [ ]:
!pip install google-cloud-bigquery --quiet
from google.cloud import bigquery
import time

# --- CONFIGURATION ---
PROJECT = "stgw-shared-ai-dev"
DATASET = "parquet_data_set"

# Source table for the statistics (the output of the previous profiling script)
STATS_SOURCE_TABLE = f"{PROJECT}.{DATASET}.2501_new_parquet_table_filtered_complete_profiling_stats_core"

# Destination table for the summary counts
SUMMARY_DEST_TABLE = f"{PROJECT}.{DATASET}.recommendation_summary_table"

# --- 1. Initialize BigQuery Client ---
try:
    client = bigquery.Client(project=PROJECT)
    print(f"✅ BigQuery client initialized for project: {PROJECT}")
except Exception as e:
    print(f"❌ Failed to initialize BigQuery client. Ensure authentication is set up. Error: {e}")
    exit()

# --- 2. Define the SQL Query (identical to save_recommendation_summary.sql) ---
# This query counts each recommendation type and saves the results to the destination table.
SQL_QUERY = f"""
CREATE OR REPLACE TABLE
  `{SUMMARY_DEST_TABLE}`
AS
SELECT
  recommendation,
  COUNT(column_name) AS column_count,
  ROUND(COUNT(column_name) * 100.0 / SUM(COUNT(column_name)) OVER (), 2) AS percentage_of_total_columns
FROM
  `{STATS_SOURCE_TABLE}`
GROUP BY
  recommendation
ORDER BY
  column_count DESC;
"""

# --- 3. Execute the Query ---
print(f"\nSubmitting job to create summary table: {SUMMARY_DEST_TABLE}...")

try:
    # client.query() executes the DDL statement
    job = client.query(SQL_QUERY)

    # Wait for the job to complete
    job.result()

    print("\n========================================================")
    print(f"🎉 SUCCESS! The recommendation summary has been saved.")
    print(f"Destination Table: {SUMMARY_DEST_TABLE}")
    print("========================================================")

except Exception as e:
    print(f"❌ An error occurred during query execution: {e}")

# --- 4. Optional: Print the first 5 rows of the new table for confirmation ---
time.sleep(2) # Give BigQuery a moment to register the new table
try:
    df_summary = client.query(f"SELECT * FROM `{SUMMARY_DEST_TABLE}` LIMIT 5").to_dataframe()
    print("\n--- First 5 Rows of New Summary Table ---")
    print(df_summary)
except Exception as e:
    print(f"\nCould not read the newly created table: {e}")

In [ ]:
#!pip install google-cloud-bigquery scipy --quiet
from google.cloud import bigquery
import pandas as pd
from itertools import combinations
from scipy.stats import chi2_contingency
import numpy as np
import pandas_gbq # Required for saving the final DataFrame

# --- CONFIGURATION ---
PROJECT = "stgw-shared-ai-dev"
DATASET = "parquet_data_set"

# Source tables
SOURCE_DATA_TABLE = "2501_new_parquet_table"
STATS_TABLE = f"{PROJECT}.{DATASET}.2501_new_parquet_table_filtered_complete_profiling_stats_core"

# Destination table for the correlation matrix (now Cramer's V matrix)
CORRELATION_DEST_TABLE = f"{PROJECT}.{DATASET}.categorical_association_matrix"

# Sampling size for analysis. Adjust based on your Colab/memory limits.
SAMPLE_SIZE = 500000

# --- 1. Initialize BigQuery Client ---
try:
    client = bigquery.Client(project=PROJECT)
    print(f"✅ BigQuery client initialized for project: {PROJECT}")
except Exception as e:
    print(f"❌ Failed to initialize BigQuery client. Error: {e}")
    exit()

def get_table_row_count(client, table_ref):
    """Queries the exact total row count of the specified BigQuery table."""
    print(f"\n--- 1.5. Querying actual row count for {table_ref} ---")
    query = f"SELECT count(*) as total_rows FROM `{table_ref}`"
    try:
        job = client.query(query)
        result = list(job.result())[0]
        total_rows = result['total_rows']
        print(f"Found total rows: {total_rows:,}")
        return total_rows
    except Exception as e:
        print(f"❌ Failed to get row count. Assuming a large number (26M) for sampling. Error: {e}")
        return 26000000 # Fallback to the known approximate size

def get_non_constant_columns(client, stats_table):
    """
    Fetches columns specifically classified as good or moderate categorical features
    from the profiling stats table for association testing.
    """
    print("\n--- 2. Fetching Good/Moderate Categorical Columns for Association Test ---")

    # Filtering for the explicit recommendation strings defined in the profiling script
    query = f"""
    SELECT
        column_name
    FROM
        `{stats_table}`
    WHERE
        recommendation IN (
            '✅ Good categorical',
            'ℹ️ Moderate — check imbalance',
            '🧩 Consider binning/grouping'
        )
    ORDER BY
        unique_values ASC
    """

    df = client.query(query).to_dataframe()
    column_list = df['column_name'].tolist()

    print(f"Found {len(column_list)} categorical columns selected for analysis.")
    return column_list

def calculate_cramers_v(x, y):
    """
    Calculates Cramer's V statistic for two categorical series.
    Cramer's V ranges from 0 (no association) to 1 (perfect association).
    """
    # Create the contingency table
    conf_matrix = pd.crosstab(x, y)

    # Calculate Chi-Squared statistic
    chi2, p, dof, expected = chi2_contingency(conf_matrix)

    # Calculate N (total observations)
    n = conf_matrix.sum().sum()

    # Calculate phi squared (V^2)
    # The term min(k-1, r-1) is necessary where k and r are the dimensions of the table.
    min_dim = min(conf_matrix.shape) - 1

    # Handle the case where min_dim is 0 (i.e., a 1x1 table, though excluded by non-constant filter)
    if min_dim == 0 or n == 0:
        return 0.0

    phi2 = max(0, chi2 / n)

    # Calculate Cramer's V
    cramers_v = np.sqrt(phi2 / min_dim)

    return cramers_v

def analyze_and_save_association(client, data_table, column_list):
    """
    Pulls a sample of the data, calculates the Cramer's V matrix,
    and saves the result to BigQuery.
    """
    # Get the actual total rows for accurate sampling probability
    full_table_ref = f"{PROJECT}.{DATASET}.{data_table}"
    total_rows = get_table_row_count(client, full_table_ref)

    # Calculate the probability needed to get the target SAMPLE_SIZE
    sampling_probability = SAMPLE_SIZE / total_rows

    print(f"\n--- 3. Sampling {SAMPLE_SIZE:,} Rows from BigQuery ---")

    # 1. Pull a random sample from the large table using the dynamic probability
    sample_query = f"""
    SELECT
      {', '.join([f'`{col}`' for col in column_list])}
    FROM
      `{full_table_ref}`
    WHERE
      rand() < {sampling_probability}
    LIMIT {SAMPLE_SIZE}
    """

    try:
        df_sample = client.query(sample_query).to_dataframe()
        print(f"Successfully pulled {len(df_sample)} sample rows.")

        SAMPLE_CSV_PATH = "/content/bigquery_sample_data.csv"
        df_sample.to_csv(SAMPLE_CSV_PATH, index=False)
        print(f"✅ Sample data saved to local file: {SAMPLE_CSV_PATH}")

    except Exception as e:
        print(f"❌ Failed to sample data from BigQuery. Error: {e}")
        return

    # 2. Calculate the Cramer's V Matrix
    num_cols = len(column_list)
    v_matrix = pd.DataFrame(index=column_list, columns=column_list, dtype=float)

    print("\n--- 4. Calculating Pairwise Cramer's V Matrix ---")

    for i, col1 in enumerate(column_list):
        for j, col2 in enumerate(column_list):
            if i >= j: # Calculate only the upper or lower triangle (and diagonal)
                # Association of a variable with itself is always 1
                if i == j:
                    v_matrix.loc[col1, col2] = 1.0
                else:
                    # Calculate V and populate both sides of the symmetrical matrix
                    v = calculate_cramers_v(df_sample[col1], df_sample[col2])
                    v_matrix.loc[col1, col2] = v
                    v_matrix.loc[col2, col1] = v

    print("✅ Cramer's V matrix calculation complete.")

    # 3. Save the Matrix to BigQuery
    print(f"\n--- 5. Saving Association Matrix to BigQuery: {CORRELATION_DEST_TABLE} ---")

    # pandas_gbq requires the index (column names) to be part of the table.
    v_matrix_flat = v_matrix.reset_index().rename(columns={'index': 'feature'})

    try:
        pandas_gbq.to_gbq(
            v_matrix_flat,
            destination_table=CORRELATION_DEST_TABLE.replace(f'{PROJECT}.', ''), # needs dataset.table
            project_id=PROJECT,
            if_exists='replace'
        )
        print("🎉 SUCCESS! Cramer's V association matrix saved to BigQuery.")

    except Exception as e:
        print(f"❌ Failed to save Cramer's V matrix to BigQuery. Error: {e}")
        return

    print("\n--- Top Multicollinearity Findings (V > 0.8) ---")
    # Identify highly associated pairs for display
    high_v = []
    for col1, col2 in combinations(column_list, 2):
        v = v_matrix.loc[col1, col2]
        if v > 0.8:
            high_v.append((col1, col2, v))

    if high_v:
        high_v_df = pd.DataFrame(high_v, columns=['Feature_A', 'Feature_B', "Cramer's_V"])
        print(high_v_df.sort_values("Cramer's_V", ascending=False).head())
    else:
        print("No highly associated pairs (Cramer's V > 0.8) found in the sample.")


# --- Main Execution ---
if __name__ == "__main__":
    non_constant_cols = get_non_constant_columns(client, STATS_TABLE)

    # NOTE: If non_constant_cols is too large (e.g., > 500 columns),
    # the N*N calculation and memory load might still be too high in Colab.
    # You might need to further subset the columns here if memory errors occur.

    analyze_and_save_association(client, SOURCE_DATA_TABLE, non_constant_cols)

In [ ]:
import pandas as pd
from google.cloud import bigquery

# --- CONFIGURATION ---
PROJECT = "stgw-shared-ai-dev"
DATASET = "parquet_data_set"

# The destination table created by correlation_analysis.py
ASSOCIATION_MATRIX_TABLE = f"{PROJECT}.{DATASET}.categorical_association_matrix"
LOCAL_CSV_PATH = "/content/association_matrix.csv" # New path for the local CSV

# --- 1. Initialize BigQuery Client ---
try:
    client = bigquery.Client(project=PROJECT)
    print(f"✅ BigQuery client initialized.")
except Exception as e:
    print(f"❌ Failed to initialize BigQuery client. Error: {e}")
    exit()

# --- 2. Define Query to Read the Table ---
query = f"SELECT * FROM `{ASSOCIATION_MATRIX_TABLE}`"

print(f"\nReading association matrix from BigQuery table: {ASSOCIATION_MATRIX_TABLE}")

# --- 3. Execute Query and Load into DataFrame ---
try:
    df_matrix = client.query(query).to_dataframe()

    print("🎉 Successfully loaded the association matrix into df_matrix.")
    print(f"DataFrame shape: {df_matrix.shape}")

    # --- 4. Save DataFrame to Local CSV ---
    df_matrix.to_csv(LOCAL_CSV_PATH, index=False)
    print(f"✅ Association matrix saved to local CSV: {LOCAL_CSV_PATH}")

    # Display the first few rows for confirmation
    print("\n--- First 5 Rows of Cramer's V Matrix ---")
    print(df_matrix.head())

except Exception as e:
    print(f"❌ Failed to read data from BigQuery. Error: {e}")

In [ ]:
import pandas as pd
from google.cloud import bigquery
from itertools import combinations
import numpy as np

# --- CONFIGURATION ---
PROJECT = "stgw-shared-ai-dev"
DATASET = "parquet_data_set"

# The destination table created by correlation_analysis.py
ASSOCIATION_MATRIX_TABLE = f"{PROJECT}.{DATASET}.categorical_association_matrix"
LOCAL_CSV_PATH = "/content/association_matrix.csv"
HIGH_ASSOCIATION_CSV_PATH = "/content/highly_associated_pairs.csv"
# NEW OUTPUT PATH for weakly associated pairs
WEAK_ASSOCIATION_CSV_PATH = "/content/weakly_associated_pairs.csv"

# --- 1. Initialize BigQuery Client ---
try:
    client = bigquery.Client(project=PROJECT)
    print(f"✅ BigQuery client initialized.")
except Exception as e:
    print(f"❌ Failed to initialize BigQuery client. Error: {e}")
    exit()

# --- 2. Define Query to Read the Table ---
query = f"SELECT * FROM `{ASSOCIATION_MATRIX_TABLE}`"

print(f"\nReading association matrix from BigQuery table: {ASSOCIATION_MATRIX_TABLE}")

# --- 3. Execute Query and Load into DataFrame ---
try:
    df_matrix_flat = client.query(query).to_dataframe()

    print("🎉 Successfully loaded the association matrix into df_matrix_flat.")
    print(f"DataFrame shape: {df_matrix_flat.shape}")

    # Set the 'feature' column as the index to return to the matrix format
    df_matrix = df_matrix_flat.set_index('feature')

    # --- 4. Save the full DataFrame to Local CSV ---
    df_matrix_flat.to_csv(LOCAL_CSV_PATH, index=False)
    print(f"✅ Full association matrix saved to local CSV: {LOCAL_CSV_PATH}")

    # --- 5. Find and Save Highly and Weakly Associated Pairs (V > 0.75) ---
    print("\n--- 5. Categorizing Associated Pairs (Threshold V=0.75) ---")

    column_list = df_matrix.columns.tolist()
    high_v_pairs = []
    weak_v_pairs = []

    # Iterate over unique pairs of columns
    for col1, col2 in combinations(column_list, 2):
        # Retrieve the Cramer's V value from the matrix
        v = df_matrix.loc[col1, col2]

        pair_data = {
            'Feature_A': col1,
            'Feature_B': col2,
            "Cramer's_V": v
        }

        # Check association strength and append to the corresponding list
        if v >= 0.75:
            high_v_pairs.append(pair_data)
        else:
            weak_v_pairs.append(pair_data)

    # --- Save High Association Pairs ---
    if high_v_pairs:
        df_high_v = pd.DataFrame(high_v_pairs)
        # Sort by V score descending and save
        df_high_v = df_high_v.sort_values("Cramer's_V", ascending=False)
        df_high_v.to_csv(HIGH_ASSOCIATION_CSV_PATH, index=False)

        print(f"✅ Found {len(df_high_v)} highly associated pairs (V >= 0.75).")
        print(f"Saved results to local CSV: {HIGH_ASSOCIATION_CSV_PATH}")

        print("\n--- Top 5 Highly Associated Pairs ---")
        print(df_high_v.head())
    else:
        print("No highly associated pairs (Cramer's V >= 0.75) found.")

    # --- Save Weak Association Pairs ---
    if weak_v_pairs:
        df_weak_v = pd.DataFrame(weak_v_pairs)
        # Sort by V score ascending (weakest first) and save
        df_weak_v = df_weak_v.sort_values("Cramer's_V", ascending=True)
        df_weak_v.to_csv(WEAK_ASSOCIATION_CSV_PATH, index=False)

        print(f"\n✅ Found {len(df_weak_v)} weakly associated pairs (V < 0.75).")
        print(f"Saved results to local CSV: {WEAK_ASSOCIATION_CSV_PATH}")

        print("\n--- Top 5 Weakly Associated Pairs (V closest to 0) ---")
        print(df_weak_v.head())
    else:
        print("All pairs found were highly associated.")

except Exception as e:
    print(f"❌ Failed during data processing or CSV saving. Error: {e}")

In [ ]:
import pandas as pd
from google.cloud import bigquery
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# --- CONFIGURATION ---
PROJECT = "stgw-shared-ai-dev"
DATASET = "parquet_data_set"

# The destination table created by correlation_analysis.py
ASSOCIATION_MATRIX_TABLE = f"{PROJECT}.{DATASET}.categorical_association_matrix"

# --- 1. Initialize BigQuery Client ---
try:
    client = bigquery.Client(project=PROJECT)
    print(f"✅ BigQuery client initialized.")
except Exception as e:
    print(f"❌ Failed to initialize BigQuery client. Error: {e}")
    exit()

# --- 2. Define Query to Read the Table ---
query = f"SELECT * FROM `{ASSOCIATION_MATRIX_TABLE}`"

print(f"\nReading association matrix from BigQuery table: {ASSOCIATION_MATRIX_TABLE}")

# --- 3. Execute Query and Load into DataFrame ---
try:
    df_matrix_flat = client.query(query).to_dataframe()

    print("🎉 Successfully loaded the association matrix into df_matrix_flat.")

    # Set the 'feature' column as the index to return to the matrix format
    df_matrix = df_matrix_flat.set_index('feature')

    # --- 4. Identify the Weakly Associated Features ---
    # Goal: Filter the matrix to show only features that, on average, have weak association (low Cramer's V)
    # with ALL other features. We use the average association score across the row/column.

    # Calculate the average Cramer's V score for each feature across all other features
    average_association = df_matrix.mean(axis=1).sort_values(ascending=True)

    # We select the top N features with the lowest average association.
    # We choose N=20 here, but this number can be adjusted (e.g., 50 if memory allows).
    N_WEAKEST_FEATURES = 20

    weakest_features = average_association.head(N_WEAKEST_FEATURES).index.tolist()

    print(f"\nIdentifying the {N_WEAKEST_FEATURES} features with the lowest average association (best candidates).")
    print("Selected features:", weakest_features)

    # Filter the matrix to include only the weakest features
    df_weak_matrix = df_matrix.loc[weakest_features, weakest_features]

    # --- 5. Generate Heatmap for Weakly Associated Features ---
    plt.figure(figsize=(12, 10))

    # Define the custom colormap: Light colors for low V (weak association), Dark for high V
    cmap = sns.light_palette("seagreen", as_cmap=True, reverse=True)

    sns.heatmap(
        df_weak_matrix,
        annot=True,              # Show the Cramer's V values on the map
        fmt=".2f",               # Format to 2 decimal places
        cmap=cmap,               # Use the reversed light palette
        linewidths=0.5,
        linecolor='gray',
        square=True,
        cbar_kws={'label': "Cramer's V (Association Strength)"}
    )

    plt.title(f"Heatmap of {N_WEAKEST_FEATURES} Least Associated Categorical Features (Cramer's V)", fontsize=14)
    plt.show()

    print("\n✅ Heatmap displayed successfully.")

except Exception as e:
    print(f"❌ Failed to read data or generate heatmap. Ensure the table exists and data is valid. Error: {e}")

In [ ]:

from google.cloud import storage
import zipfile
import os

# --- Configuration ---
BUCKET_NAME = 'hqs-bucket' # Your GCS bucket name
GCS_ZIP_PATH = 'bigquery_sample_data.csv.zip'
LOCAL_TEMP_DIR = '/home/jupyter/data_temp/' # Standard local folder in Vertex Notebooks

# 1. Setup client and paths
client = storage.Client()
bucket = client.bucket(BUCKET_NAME)
blob = bucket.blob(GCS_ZIP_PATH)

os.makedirs(LOCAL_TEMP_DIR, exist_ok=True)
local_zip_path = os.path.join(LOCAL_TEMP_DIR, GCS_ZIP_PATH)

# 2. Download the ZIP file from GCS
print(f"Downloading {GCS_ZIP_PATH} from GCS to {local_zip_path}...")
blob.download_to_filename(local_zip_path)
print("Download complete.")

# 3. Unzip the file locally
try:
    with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
        print("Unzipping the file...")
        zip_ref.extractall(LOCAL_TEMP_DIR)

        # Determine the name of the extracted CSV
        extracted_files = zip_ref.namelist()
        # Find the first file that ends with .csv
        csv_file_name = next(f for f in extracted_files if f.endswith('.csv'))

        print(f"Successfully unzipped CSV file: {csv_file_name}")

except Exception as e:
    print(f"Error during unzip. Please verify the ZIP file integrity: {e}")

import pandas as pd

# Path to the unzipped CSV
local_csv_path = os.path.join(LOCAL_TEMP_DIR, csv_file_name)

# 4. Load the extracted CSV
print(f"Loading {local_csv_path} into pandas DataFrame...")
# Use low_memory=False for large files to avoid mixed-type warnings/errors
try:
    df = pd.read_csv(local_csv_path, low_memory=False)
    print("\nDataFrame loaded successfully.")
    print(df.head())
    print(f"DataFrame Shape: {df.shape}")
except Exception as e:
    print(f"Error loading CSV file: {e}")


In [ ]:

# --- IMPORTANT: Ensure 'df' (the 5M row DataFrame) is loaded before running this block. ---

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import hdbscan
import gc
import time

# --- COLUMN LISTS (Same as before) ---
FULL_MULTI_VALUE_COLS = ['own_smarthome_controller_hub', 'women_contraception', 'debit_cards_used_30days', 'first_aid', 'head_of_household', 'head_of_household_gender', 'hispanic_place_of_birth', 'household_gender', 'international_travel', 'loans', 'manicure', 'massage', 'mortgage_loan_lien', 'movie_tickets', 'orthodontia_devices', 'age_of_children_present_0_18', 'spanish_speaking_household', 'tax_preparation_professional', 'travel_loyalty_membership', 'vehicle_lease_purchase_transmission', 'vehicle_lease_purchase_year', 'waxing', 'camping_equipment_spent', 'marital_status', 'mobile_phones_agreement_type', 'auto_loan', 'portable_gps', 'mobile_phone_plan_type', 'shoppers_electronics', 'camcorder_video_camera_brand', 'casino_gambling', 'casino_gambling_place', 'checking_savings_account', 'computer_software_program_personal_income_taxes', 'cooking_sites', 'cosmetic_surgery', 'credit_card_balance', 'credit_card_payment_method', 'credit_card_quantiles', 'cruise_ships', 'dentist_visits', 'diabetic', 'doctor_visits', 'financial_risk_tolerance', 'financial_stability', 'fishing', 'gym_membership', 'health_supplements', 'home_improvement', 'household_financial_assets', 'household_investment_assets', 'household_liability_assets', 'household_net_worth', 'insurance_life_insurance', 'investment_platform_online', 'investment_platform_traditional', 'investment_products', 'investments_options', 'investments_quantiles', 'joint_credit_card', 'life_insurance', 'luxury_purchases', 'magazine_subscriptions', 'mail_order_purchases', 'marine_recreational_vehicles', 'media_consumption', 'mortgage_loan', 'online_banking', 'online_brokerage', 'online_payments', 'online_shopping', 'online_trading', 'pain_relief', 'passengers_on_vehicle', 'retirement_investment_assets', 'retirement_savings_plan', 'retirement_vehicle', 'social_media_media_quantiles', 'stocks_bonds_options', 'travel_club_membership', 'vehicle_lease_purchase_model_year']
FULL_SINGLE_VALUE_COLS = ['equity_balance', 'gender', 'credit_behavior', 'urbanicity', 'credit_tier', 'education_individual', 'person_role', 'age', 'discretionary_spend', 'net_worth', 'family_income_decile', 'wealth_rating', 'home_equity', 'religion', 'age_gender_combination', 'internet_media_quantiles', 'morning_radio_media_quantiles', 'online_video_media_quantiles', 'primetime_radio_media_quantiles', 'streaming_video_media_quantiles', 'tv_media_quantiles', 'air_conditioning', 'automotive_repair_spent', 'automotive_repair_spent_quantiles', 'cable_satellite_provider', 'car_ownership_cost_quantiles', 'car_purchase_price_quantiles', 'cash_in_bank_quantiles', 'checking_account', 'college_savings_plan', 'credit_card_issuer', 'credit_card_rewards', 'credit_card_tier', 'credit_score_quantiles', 'dinner_club', 'distance_to_nearest_airport', 'distance_to_nearest_college', 'estimated_home_value_quantiles', 'home_built_year', 'home_loan_interest_rate_quantiles', 'home_loan_term', 'home_purchase_price_quantiles', 'home_rental_value_quantiles', 'home_type', 'household_size', 'household_type', 'insurance_homeowners', 'insurance_property_quantiles', 'insurance_vehicle_quantiles', 'internet_provider', 'investment_account_quantiles', 'length_of_residence', 'loan_to_value_ratio_quantiles', 'loan_type', 'loan_type_quantiles', 'mail_order_purchases_quantiles', 'mobile_phones_number_in_household', 'mortgage_amount_quantiles', 'online_payments_quantiles', 'person_count', 'political_affiliation', 'primary_language', 'retirement_plan_quantiles', 'social_media_platform', 'stock_market_investments', 'tax_preparation_method', 'time_at_current_address_quantiles', 'vehicle_lease_purchase_make', 'vehicle_lease_purchase_make_model', 'vehicle_lease_purchase_type', 'vehicle_lease_purchase_vehicle_count', 'vehicle_owned', 'vintage_of_credit_file_quantiles', 'wireless_carrier', 'wireless_spend_quantiles']


# --- STEP 1: PREPROCESSING AND MEMORY OPTIMIZATION (FIXED) ---

print(f"Starting Preprocessing. Initial Shape: {df.shape}")

# 1. Standardize Missing Values
df.fillna('MISSING', inplace=True)
for col in df.columns:
    df[col] = df[col].astype(str).str.strip().replace(['NaN', 'nan', ''], 'MISSING')

df_temp = df.copy()

# 2. Multi-Hot Encoding
for col in FULL_MULTI_VALUE_COLS:
    if col in df_temp.columns:
        dummies = df_temp[col].str.get_dummies(sep='|')
        dummies.columns = [f'{col}_{c.strip().replace(" ", "_")}' for c in dummies.columns]
        df_temp = pd.concat([df_temp, dummies], axis=1)
        df_temp.drop(columns=[col], inplace=True)

# 3. Robust Final One-Hot Encoding (FIX: Find ALL remaining object columns)
# Ensure ALL string/object columns are converted to 0/1 before final type conversion
remaining_object_cols = df_temp.select_dtypes(include=['object']).columns.tolist()

print(f"Final OHE target columns found: {len(remaining_object_cols)}")

if len(remaining_object_cols) > 0:
    df_encoded = pd.get_dummies(df_temp, columns=remaining_object_cols, drop_first=True)
else:
    df_encoded = df_temp # No more encoding necessary

# Memory Optimization: Convert to low memory integer type (0s and 1s)
X = df_encoded.values.astype(np.uint8)

# Clean up memory
del df_temp, df_encoded
gc.collect()

print(f"✅ Encoding Complete. Final Input Shape for AE: {X.shape} (Type: {X.dtype})")


In [ ]:

import numpy as np
import time
import gc
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# --- Configuration ---
INPUT_MATRIX_FILENAME = 'encoded_input_matrix.npz'

print(f"Starting to load the compressed input matrix from {INPUT_MATRIX_FILENAME}...")

start_time = time.time()
try:
    # Use np.load to load the compressed file
    # The data was saved with the key 'X', so we access it as ['X']
    with np.load(INPUT_MATRIX_FILENAME) as data:
        X = data['X']

    end_load_time = time.time()

    print(f"\n✅ Data loaded successfully in {end_load_time - start_time:.2f} seconds.")
    print("-" * 50)

    # 1. Print Shape and Data Type
    print(f"Array Name: X")
    print(f"Total rows (Customers): {X.shape[0]:,}")
    print(f"Total columns (Encoded Features): {X.shape[1]:,}")
    print(f"Data Type (Dtype): {X.dtype}")
    print("-" * 50)

    # 2. Print a Small Sample
    # Showing the first 5 rows and 10 columns to verify the binary (0/1) encoding
    sample_rows = 5
    sample_cols = 10

    print(f"Displaying a sample (first {sample_rows} rows, first {sample_cols} columns):")

    # Use array slicing to get a readable sample
    print(X[:sample_rows, :sample_cols])
    print("-" * 50)


    # STEP 4: AUTOENCODER (AE) FOR LATENT FEATURE EXTRACTION

    # X is now loaded and available.
    input_dim = X.shape[1]
    latent_dim = 25
    BATCH_SIZE = 16384 # CPU optimization

    print(f"\n--- Starting Autoencoder (AE) Processing ---")
    print(f"Input Dimension: {input_dim}")
    print(f"Latent Dimension (Bottleneck): {latent_dim}")

    # Callbacks for robust training on a slow CPU
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True),
        ModelCheckpoint('autoencoder_weights.keras', monitor='val_loss', save_best_only=True, verbose=0)
    ]

    # 1. Build the Autoencoder Model (Shallower and Narrower for CPU)
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(256, activation='relu')(input_layer)
    encoded = Dense(64, activation='relu')(encoded)
    latent_vector = Dense(latent_dim, activation='relu', name='latent_layer')(encoded) # Bottleneck
    decoded = Dense(64, activation='relu')(latent_vector)
    decoded = Dense(256, activation='relu')(decoded)
    output_layer = Dense(input_dim, activation='sigmoid')(decoded)

    autoencoder = Model(input_layer, output_layer)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

    print(f"Starting AE Training with BATCH_SIZE={BATCH_SIZE}...")


    # 2. Train the AE
    start_train_time = time.time()
    autoencoder.fit(X, X,
                    epochs=10,
                    batch_size=BATCH_SIZE,
                    shuffle=True,
                    validation_split=0.1,
                    callbacks=callbacks,
                    verbose=2) # Use verbose=2 to show one line per epoch

    print(f"AE Training Complete in {(time.time() - start_train_time):.2f} seconds.")

    # 3. Extract Latent Features
    encoder_model = Model(input_layer, latent_vector)
    # Use an even larger batch size for faster prediction
    latent_features = encoder_model.predict(X, batch_size=BATCH_SIZE * 4)
    print(f"✅ Latent Features Extracted. Shape: {latent_features.shape}")

    # Clean up input data array to free memory before clustering
    del X
    gc.collect()
    print("Memory cleaned up: Original high-dimensional matrix X deleted.")

except FileNotFoundError:
    print(f"❌ ERROR: The file '{INPUT_MATRIX_FILENAME}' was not found. Please ensure the preceding step (Step 4) was executed successfully to create it.")
except Exception as e:
    print(f"❌ An unexpected error occurred during loading or training: {e}")

In [ ]:

import numpy as np
import hdbscan
import pandas as pd
import time
import gc

# 1. Sample the Latent Space for Clustering
SAMPLE_SIZE = 450000
np.random.seed(42)
sample_indices = np.random.choice(latent_features.shape[0], size=SAMPLE_SIZE, replace=False)
latent_sample = latent_features[sample_indices]
print(f"\n--- Re-running HDBSCAN fit on sample of {SAMPLE_SIZE:,} points. ---")

# 2. HDBSCAN (prediction_data=True is mandatory for approximate_predict)
clusterer = hdbscan.HDBSCAN(min_cluster_size=1000,
                            metric='euclidean',
                            cluster_selection_method='eom',
                            prediction_data=True)

start_time_fit = time.time()
clusterer.fit(latent_sample)
print(f"HDBSCAN Fit Complete in {(time.time() - start_time_fit):.2f} seconds.")

# 3. Apply Clusters to Full Dataset using approximate_predict
latent_features_np = np.asarray(latent_features)

# UNPACKING FIX: Only take the first element (the labels array)
start_time_predict = time.time()
cluster_labels_full, cluster_probabilities = hdbscan.approximate_predict(clusterer, latent_features_np)
print(f"Prediction on full dataset complete in {(time.time() - start_time_predict):.2f} seconds.")


# --- CRITICAL DIAGNOSTIC CHECKS ---
print("\n--- CRITICAL SIZE CHECK ---")
print(f"1. Full DataFrame 'df' rows before cluster assignment: {df.shape[0]:,} rows")
print(f"2. Full Cluster Label array size: {cluster_labels_full.shape[0]:,} labels")

if df.shape[0] != cluster_labels_full.shape[0]:
    # This block will execute if df has shrunk before this point!
    print("🚨 WARNING: DataFrame size does not match cluster label array size. DATA LOSS DETECTED.")
    print("You must re-run STEP 1 to load the full DataFrame before this step.")
    # Attempting to proceed, but result will be corrupted if sizes don't match.
# -----------------------------------


# 4. Attach results back to the original DataFrame 'df'
df['Cluster'] = cluster_labels_full
print("✅ Clustering Complete. Cluster labels added to df['Cluster'].")

print("\n--- Final Top 10 Cluster Sizes (using the full 5M rows) ---")
print(df['Cluster'].value_counts().head(10).to_markdown(numalign="left", stralign="left"))


# --- SAVE OUTPUT ---
output_filename = "clustered_customer_data.csv"

# FINAL DIAGNOSTIC CHECK BEFORE SAVING
print(f"\n3. Final DataFrame 'df' rows before saving: {df.shape[0]:,} rows")

# The only line that can shrink your output file is if df itself is already small here.
print(f"Saving {df.shape[0]:,} rows to {output_filename}...")
df.to_csv(output_filename, index=False)
print(f"✅ Data successfully saved to {output_filename}")

# Clean up memory
del latent_features_np
gc.collect()

In [ ]:

import numpy as np
import time
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
# NOTE: This code assumes the HDBSCAN model ('clusterer') has been fitted
# on the sample data ('latent_sample') from the previous step.

# --- Cluster Evaluation (Metrics on the Fitted Sample) ---
# Evaluate the clustering quality on the fitted sample's non-noise points
labels_sample = clusterer.labels_
# Filter out noise points (-1) for a meaningful evaluation
valid_indices = labels_sample != -1
X_valid = latent_sample[valid_indices]
labels_valid = labels_sample[valid_indices]
n_clusters = len(np.unique(labels_valid))

print(f"\n--- Quantitative Cluster Evaluation ({valid_indices.sum():,} Non-Noise Samples, {n_clusters} Clusters) ---")

if n_clusters > 1 and X_valid.shape[0] > 10:
    start_eval_time = time.time()

    # 1. Silhouette Score
    # Range: -1 (worst) to +1 (best). Measures separation and cohesion.
    silhouette_avg = silhouette_score(X_valid, labels_valid)
    print(f"Silhouette Score (Average): {silhouette_avg:.4f}")

    # 2. Calinski-Harabasz Index (Variance Ratio Criterion)
    # Higher is better. Measures ratio of between-cluster variance to within-cluster variance.
    ch_score = calinski_harabasz_score(X_valid, labels_valid)
    print(f"Calinski-Harabasz Index: {ch_score:.2f}")

    # 3. Davies-Bouldin Index
    # Lower is better (closer to 0). Measures average similarity between clusters.
    db_score = davies_bouldin_score(X_valid, labels_valid)
    print(f"Davies-Bouldin Index: {db_score:.4f}")

    print(f"Evaluation completed in {(time.time() - start_eval_time):.2f} seconds.")
    print("\nInterpretation Guidance:")
    print(" - Silhouette: Aim for positive scores (0.5+ is generally good).")
    print(" - Calinski-Harabasz: Aim for the highest score possible.")
    print(" - Davies-Bouldin: Aim for the lowest score possible (closer to 0).")
else:
    print("\nWarning: Not enough valid clusters or non-noise points in the sample to calculate evaluation metrics.")
# ----------------------------------------------------------------------------------------

In [ ]:

import pandas as pd
import numpy as np
import os

# --- CONFIGURATION ---
INPUT_DATA_FILE = "clustered_customer_data.csv"
TRAITS_OUTPUT_FILE = "cluster_persona_traits.csv"

# Reconstructed list of single-value columns for analysis (used for mode comparison)
FULL_SINGLE_VALUE_COLS = [
    'equity_balance', 'gender', 'credit_behavior', 'urbanicity', 'credit_tier', 'education_individual',
    'person_role', 'age', 'discretionary_spend', 'net_worth', 'family_income_decile', 'wealth_rating',
    'home_equity', 'religion', 'age_gender_combination', 'internet_media_quantiles',
    'morning_radio_media_quantiles', 'online_video_media_quantiles', 'primetime_radio_media_quantiles',
    'streaming_video_media_quantiles', 'tv_media_quantiles', 'air_conditioning', 'automotive_repair_spent',
    'automotive_repair_spent_quantiles', 'cable_satellite_provider', 'car_ownership_cost_quantiles',
    'car_purchase_price_quantiles', 'cash_in_bank_quantiles', 'checking_account', 'college_savings_plan',
    'credit_card_issuer', 'credit_card_rewards', 'credit_card_tier', 'credit_score_quantiles',
    'dinner_club', 'distance_to_nearest_airport', 'distance_to_nearest_college', 'estimated_home_value_quantiles',
    'home_built_year', 'home_loan_interest_rate_quantiles', 'home_loan_term', 'home_purchase_price_quantiles',
    'home_rental_value_quantiles', 'home_type', 'household_size', 'household_type', 'insurance_homeowners',
    'insurance_property_quantiles', 'insurance_vehicle_quantiles', 'internet_provider', 'investment_account_quantiles',
    'length_of_residence', 'loan_to_value_ratio_quantiles', 'loan_type', 'loan_type_quantiles',
    'mail_order_purchases_quantiles', 'mobile_phones_number_in_household', 'mortgage_amount_quantiles',
    'online_payments_quantiles', 'person_count', 'political_affiliation', 'primary_language',
    'retirement_plan_quantiles', 'social_media_platform', 'stock_market_investments', 'tax_preparation_method',
    'time_at_current_address_quantiles', 'vehicle_lease_purchase_make', 'vehicle_lease_purchase_make_model',
    'vehicle_lease_purchase_type', 'vehicle_lease_purchase_vehicle_count', 'vehicle_owned',
    'vintage_of_credit_file_quantiles', 'wireless_carrier', 'wireless_spend_quantiles'
]

def characterize_cluster(df, cluster_id):
    """
    Calculates feature modes for a specific cluster and returns a DataFrame
    of features that differ significantly from the population mode.
    """
    cluster_df = df[df['Cluster'] == cluster_id].copy()
    cols_to_analyze = [col for col in FULL_SINGLE_VALUE_COLS if col in df.columns]

    empty_df = pd.DataFrame(columns=['Cluster_ID', 'Feature', 'Cluster_Mode', 'Cluster_Mode_Pct', 'Population_Mode'])

    if not cols_to_analyze or cluster_df.empty:
        return empty_df

    # Calculate population mode and cluster mode
    pop_mode_series = df[cols_to_analyze].mode()
    cluster_mode_series = cluster_df[cols_to_analyze].mode()

    if pop_mode_series.empty or cluster_mode_series.empty:
        return empty_df

    population_mode = pop_mode_series.iloc[0]
    cluster_mode = cluster_mode_series.iloc[0]

    diff_features = []
    for col in cols_to_analyze:
        pop_val = population_mode.get(col)
        cluster_val = cluster_mode.get(col)

        # Check for mode difference
        if pop_val != cluster_val and pd.notna(pop_val) and pd.notna(cluster_val):
            # Calculate the percentage of the cluster that holds the mode value
            cluster_mode_pct = cluster_df[col].value_counts(normalize=True).get(cluster_val, 0)

            diff_features.append({
                'Cluster_ID': cluster_id,
                'Feature': col,
                'Cluster_Mode': cluster_val,
                'Cluster_Mode_Pct': f"{cluster_mode_pct:.1%}",
                'Population_Mode': pop_val
            })

    diff_df = pd.DataFrame(diff_features)

    if diff_df.empty:
        return empty_df
    else:
        # Sort by the numeric percentage value
        # FIX: Use pd.to_numeric instead of astype(..., errors='coerce') for broader compatibility
        diff_df['Cluster_Mode_Pct_Val'] = pd.to_numeric(diff_df['Cluster_Mode_Pct'].str.replace('%', ''), errors='coerce')
        diff_df = diff_df.sort_values(by='Cluster_Mode_Pct_Val', ascending=False)
        return diff_df[['Cluster_ID', 'Feature', 'Cluster_Mode', 'Cluster_Mode_Pct', 'Population_Mode']]

def run_characterization(df):
    """
    Runs characterization on the top 10 non-noise clusters and saves traits to a CSV file.
    """
    # Identify Top 10 non-noise clusters (excluding noise, which is usually -1)
    top_clusters_to_analyze = df['Cluster'].value_counts().drop(-1, errors='ignore').head(10).index.tolist()

    if not top_clusters_to_analyze:
        print("No non-noise clusters found to analyze.")
        return

    print(f"\nStarting Cluster Characterization for Top {len(top_clusters_to_analyze)} Segments: {top_clusters_to_analyze}")

    all_cluster_reports = []
    for cluster_id in top_clusters_to_analyze:
        report_df = characterize_cluster(df, cluster_id)
        all_cluster_reports.append(report_df)

    final_report_df = pd.concat(all_cluster_reports, ignore_index=True)

    # Save the final consolidated DataFrame to a CSV file
    final_report_df.to_csv(TRAITS_OUTPUT_FILE, index=False)
    print(f"✅ Detailed cluster characterization report saved to {TRAITS_OUTPUT_FILE}")

    # Display a preview
    print("\n--- Preview of the Saved Cluster Traits (First 20 differentiating features) ---")
    print(final_report_df.head(20).to_markdown(index=False, numalign="left", stralign="left"))


# --- Main Execution ---
if __name__ == "__main__":
    try:
        # Reload the full DataFrame with the 'Cluster' column attached
        df_full = pd.read_csv(INPUT_DATA_FILE, low_memory=False)
        print(f"\nSuccessfully loaded saved data from {INPUT_DATA_FILE}. Shape: {df_full.shape}")
        run_characterization(df_full)

    except FileNotFoundError:
        print(f"❌ ERROR: File {INPUT_DATA_FILE} not found. Please ensure STEP 5 ran successfully to create it.")
    except Exception as e:
        print(f"❌ An error occurred during characterization: {e}")

In [ ]:

import pandas as pd
import json
import os
import time
from google.cloud import aiplatform

# --- CONFIGURATION (Vertex AI) ---
PROJECT_NUMBER = "582235377603"
REGION = "us-central1"
ENDPOINT_ID = "mg-endpoint-fbd9c367-d5c4-4691-891e-1b682af3ab72" # Your GPT-OSS endpoint
INPUT_TRAITS_FILE = "cluster_persona_traits.csv"

# --- HELPER FUNCTIONS ---

# Function to implement exponential backoff for API calls
def call_endpoint_with_retry(endpoint, instances, parameters, max_retries=5):
    """Calls Vertex AI endpoint with exponential backoff."""
    for attempt in range(max_retries):
        try:
            #
            response = endpoint.predict(instances=instances, parameters=parameters)
            return response
        except Exception as e:
            if attempt < max_retries - 1:
                wait_time = 2 ** attempt  # 1, 2, 4, 8, ...
                print(f"⚠️ Retrying LLM call in {wait_time}s due to error: {e}")
                time.sleep(wait_time)
            else:
                raise Exception(f"Failed to call Vertex AI endpoint after {max_retries} attempts: {e}")
    return None

def generate_persona_report_llm():
    """
    Loads cluster traits, constructs the LLM prompt, calls the Vertex AI endpoint,
    and outputs the final structured persona report.
    """
    try:
        df_traits = pd.read_csv(INPUT_TRAITS_FILE)
        print(f"\nSuccessfully loaded cluster traits from {INPUT_TRAITS_FILE}. Total differentiating features found: {df_traits.shape[0]}")
    except FileNotFoundError:
        print(f"❌ ERROR: File {INPUT_TRAITS_FILE} not found. Please ensure Step 6 ran successfully.")
        return
    except Exception as e:
        print(f"❌ An error occurred while loading cluster traits: {e}")
        return

    # Convert percentage string to float for proper sorting
    df_llm = df_traits.copy()
    # FIX: Use pd.to_numeric instead of astype(..., errors='coerce') for broader compatibility
    df_llm['Cluster_Mode_Pct_Val'] = pd.to_numeric(df_llm['Cluster_Mode_Pct'].str.replace('%', ''), errors='coerce')

    # Group by Cluster_ID and select the top N differentiating features (e.g., top 10)
    TOP_N_TRAITS = 10
    grouped = (
        df_llm.sort_values(by='Cluster_Mode_Pct_Val', ascending=False)
        .groupby('Cluster_ID')
        .head(TOP_N_TRAITS)
        .reset_index(drop=True)
    )

    # Prepare data for the prompt
    cluster_reports = []
    for cluster_id in grouped['Cluster_ID'].unique():
        cluster_df = grouped[grouped['Cluster_ID'] == cluster_id]
        trait_list = []
        for _, row in cluster_df.iterrows():
            trait = (
                f"{row['Feature']}: {row['Cluster_Mode']} ({row['Cluster_Mode_Pct']}) "
                f"vs. Population Mode: {row['Population_Mode']}"
            )
            trait_list.append(trait)
        # Use a list of traits for the LLM to process
        cluster_reports.append({
            "Cluster_ID": int(cluster_id),
            "Top_Differentiating_Features": trait_list
        })

    data_for_llm = json.dumps(cluster_reports, indent=2)

    # --- Construct the LLM Prompt and Schema ---

    # Define the required output structure
    json_schema = [
        {
            "Cluster_ID": 1,
            "Defining_Persona_Nickname": "Example Nickname, e.g., 'The Digital Savvy Executive'",
            "Top_Differentiating_Features_Summary": "A concise, single-paragraph summary of the cluster's most unique traits and potential marketing hook."
        }
    ]

    system_instruction = (
        "You are a sophisticated marketing analyst specializing in customer segmentation. "
        "Your task is to review the provided JSON array of cluster data. "
        "For each segment, assign a descriptive and creative nickname and consolidate "
        "the key differentiating features into a concise, actionable, single-paragraph summary. "
        "The summary should sound like a marketing briefing. "
        "Output ONLY a JSON array of objects. Each object MUST strictly follow the specified JSON schema. "
        "Do not include any explanation, conversational text, or markdown formatting (e.g., ```json) outside the JSON."
        "The output must start with '###JSON_OUTPUT###' followed immediately by the JSON array."
    )

    user_query = (
        f"Analyze the following JSON array, where each entry shows the top differentiating traits for a customer cluster. "
        f"The traits are defined by comparing the cluster's most common value (Cluster Mode) to the overall population's most common value (Population Mode). "
        f"Generate a nickname and a summary for each cluster. "
        f"Strictly adhere to the following JSON schema:\n```json\n{json.dumps(json_schema, indent=2)}\n```\n\n"
        f"Input Data:\n{data_for_llm}"
    )

    full_prompt = f"{system_instruction}\n\n---\n\n{user_query}"

    # --- API Call Setup ---
    aiplatform.init(project=PROJECT_NUMBER, location=REGION)
    endpoint = aiplatform.Endpoint(endpoint_name=ENDPOINT_ID)
    print(f"\nEndpoint '{ENDPOINT_ID}' loaded for LLM call.")

    instance_dict = {"prompt": full_prompt, "max_tokens": 1024}
    parameters_dict = {"temperature": 0.0}

    print("Calling Vertex AI Endpoint for Persona Generation...")

    try:
        response = call_endpoint_with_retry(endpoint, [instance_dict], parameters_dict)

        generated_text = None
        if response.predictions:
            # Extract the generated text, which often comes as a string or nested in a dict
            if isinstance(response.predictions[0], str):
                generated_text = response.predictions[0]
            elif isinstance(response.predictions[0], dict) and 'generated_text' in response.predictions[0]:
                generated_text = response.predictions[0]['generated_text']

            if not generated_text:
                print("❌ ERROR: LLM prediction was successful but returned empty content.")
                return

            # --- Robust JSON Cleanup and Parsing ---
            json_start_marker = "###JSON_OUTPUT###"

            # 1. Strip everything before the required marker (if present)
            if json_start_marker in generated_text:
                generated_text = generated_text.split(json_start_marker, 1)[1].strip()

            # 2. If marker is missing, try to find the actual start of the JSON array/object
            else:
                start_index = -1
                for char in ['[', '{']:
                    idx = generated_text.find(char)
                    if idx != -1 and (start_index == -1 or idx < start_index):
                        start_index = idx

                if start_index != -1:
                    generated_text = generated_text[start_index:].strip()
                else:
                    print("❌ ERROR: Could not find required '###JSON_OUTPUT###' marker or start of JSON ('[' or '{').")
                    print("\n--- Raw LLM Output (for debugging) ---\n")
                    print(generated_text)
                    print("\n---------------------------------------\n")
                    return


            try:
                # The LLM output is expected to be a strict JSON array
                parsed_json = json.loads(generated_text)

                # Format the final report for clean console output
                output_rows = []
                for cluster in parsed_json:
                    output_rows.append({
                        "Cluster ID": cluster.get('Cluster_ID', 'N/A'),
                        "Defining Persona/Nickname": cluster.get('Defining_Persona_Nickname', 'N/A'),
                        "Persona Summary": cluster.get('Top_Differentiating_Features_Summary', 'N/A')
                    })

                output_df = pd.DataFrame(output_rows)

                print("\n" + "="*90)
                print("✅ Final Generated Cluster Persona Report (LLM Analysis)")
                print("="*90)
                # Print the final structured output as a Markdown table for readability
                print(output_df.to_markdown(index=False, numalign="left", stralign="left"))
                print("="*90)
                # [Image of Customer Segmentation Report]

            except json.JSONDecodeError as json_e:
                print(f"❌ CRITICAL PARSING ERROR: The LLM output was not valid JSON.")
                print(f"Error details: {json_e}")
                print("\n--- Raw LLM Output (for debugging) ---\n")
                print(generated_text)
                print("\n---------------------------------------\n")
                return

    except Exception as e:
        print(f"❌ Critical Error in LLM Prediction or Execution: {e}")


# --- Main Execution ---
if __name__ == "__main__":
    generate_persona_report_llm()

In [ ]:

import json
from google.cloud import aiplatform
from docx import Document
from docx.shared import Pt
import re
import time
import random

# -------------------- CONFIG --------------------
PROJECT_NUMBER = "582235377603"
REGION = "us-central1"
ENDPOINT_ID = "mg-endpoint-fbd9c367-d5c4-4691-891e-1b682af3ab72"


# -------------------- SAMPLE CLUSTERS (Note: 'Defining_Persona_Nickname' is REMOVED) --------------------
CLUSTER_PERSONAS_DATA = [
  {
    "Cluster_ID": 23,
    "Top_Differentiating_Features": [
      "urbanicity: Urban (82.2%) vs. Population Mode: Suburban",
      "gender: Male (47.2%) vs. Population Mode: Female",
      "internet_media_quantiles: Internet: Highest Quintile(Q1)|Internet: 2nd Highest Quintile(Q2) (35.2%) vs. Population Mode: Internet: Lowest Quintile(Q5)|Internet: Highest Quintile(Q1)",
      "age: 25 - 34 (33.0%) vs. Population Mode: 35 - 44",
      "credit_behavior: Up & Coming Consumers - HH that are relatively new to credit (27.6%) vs. Population Mode: Thriving Consumers - HH that have high spend & are active users of credit",
      "age_gender_combination: 25-34 Male (15.9%) vs. Population Mode: 65+ Female",
      "family_income_decile: 7TH DECILE (11.9%) vs. Population Mode: 10TH (HIGHEST) DECILE",
      "primetime_radio_media_quantiles: Primetime Radio: Lowest Quintile(Q5)|Primetime Radio: Highest Quintile(Q1) (11.6%) vs. Population Mode: Primetime Radio: Highest Quintile(Q1)|Primetime Radio: Lowest Quintile(Q5)"
    ]
  },
  {
    "Cluster_ID": 30,
    "Top_Differentiating_Features": [
      "urbanicity: Rural (81.5%) vs. Population Mode: Suburban",
      "primetime_radio_media_quantiles: Primetime Radio: Lowest Quintile(Q5)|Primetime Radio: Highest Quintile(Q1) (50.3%) vs. Population Mode: Primetime Radio: Highest Quintile(Q1)|Primetime Radio: Lowest Quintile(Q5)",
      "credit_behavior: MISSING (37.4%) vs. Population Mode: Thriving Consumers - HH that have high spend & are active users of credit",
      "credit_tier: Sub Prime - 1st Quartile (Lowest Average Credit Score) (30.1%) vs. Population Mode: Sub Prime to Near Prime - 2nd Quartile",
      "age: 65 - 74 (24.3%) vs. Population Mode: 35 - 44",
      "family_income_decile: 4TH DECILE (14.1%) vs. Population Mode: 10TH (HIGHEST) DECILE"
    ]
  },
  {
    "Cluster_ID": 5,
    "Top_Differentiating_Features": [
      "urbanicity: Rural (81.1%) vs. Population Mode: Suburban",
      "equity_balance: Equity Balance High (50.0%) vs. Population Mode: MISSING",
      "credit_tier: Prime to Super Prime - 4th Quartile (Highest Average Credit Score) (28.6%) vs. Population Mode: Sub Prime to Near Prime - 2nd Quartile",
      "age: 65 - 74 (26.4%) vs. Population Mode: 35 - 44",
      "age_gender_combination: 65+ Male (23.4%) vs. Population Mode: 65+ Female",
      "internet_media_quantiles: Internet: 2nd Lowest Quintile(Q4)|Internet: Lowest Quintile(Q5)|Internet: Middle Quintile(Q3) (17.1%) vs. Population Mode: Internet: Lowest Quintile(Q5)|Internet: Highest Quintile(Q1)",
      "primetime_radio_media_quantiles: Primetime Radio: Middle Quintile(Q3)|Primetime Radio: 2nd Highest Quintile(Q2)|Primetime Radio: Highest Quintile(Q1) (17.0%) vs. Population Mode: Primetime Radio: Highest Quintile(Q1)|Primetime Radio: Lowest Quintile(Q5)",
      "mobile_phones_number_in_household: 2|3|Any (16.9%) vs. Population Mode: 1",
      "family_income_decile: 8TH DECILE (14.6%) vs. Population Mode: 10TH (HIGHEST) DECILE"
    ]
  },
  {
    "Cluster_ID": 1,
    "Top_Differentiating_Features": [
      "equity_balance: Equity Balance High (67.9%) vs. Population Mode: MISSING",
      "age: 45 - 54 (50.1%) vs. Population Mode: 35 - 44",
      "gender: Male (47.7%) vs. Population Mode: Female",
      "air_conditioning: Central (39.2%) vs. Population Mode: MISSING",
      "credit_tier: Near Prime to Prime - 3rd Quartile (38.4%) vs. Population Mode: Sub Prime to Near Prime - 2nd Quartile",
      "person_role: Other (37.9%) vs. Population Mode: Primary Decision Maker",
      "age_gender_combination: 45-54 Female (24.8%) vs. Population Mode: 65+ Female",
      "mobile_phones_number_in_household: 4+|3|Any (16.9%) vs. Population Mode: 1",
      "primetime_radio_media_quantiles: Primetime Radio: 2nd Highest Quintile(Q2)|Primetime Radio: 2nd Lowest Quintile(Q4)|Primetime Radio: Middle Quintile(Q3) (16.7%) vs. Population Mode: Primetime Radio: Highest Quintile(Q1)|Primetime Radio: Lowest Quintile(Q5)",
      "internet_media_quantiles: Internet: 2nd Lowest Quintile(Q4)|Internet: 2nd Highest Quintile(Q2)|Internet: Middle Quintile(Q3) (11.2%) vs. Population Mode: Internet: Lowest Quintile(Q5)|Internet: Highest Quintile(Q1)"
    ]
  },
  {
    "Cluster_ID": 4,
    "Top_Differentiating_Features": [
      "equity_balance: Equity Balance High (67.6%) vs. Population Mode: MISSING",
      "urbanicity: Rural (49.6%) vs. Population Mode: Suburban",
      "person_role: Other (43.3%) vs. Population Mode: Primary Decision Maker",
      "credit_tier: Near Prime to Prime - 3rd Quartile (31.3%) vs. Population Mode: Sub Prime to Near Prime - 2nd Quartile",
      "age: 65 - 74 (28.0%) vs. Population Mode: 35 - 44",
      "education_individual: High School (25.1%) vs. Population Mode: MISSING",
      "age_gender_combination: 65+ Male (20.2%) vs. Population Mode: 65+ Female",
      "primetime_radio_media_quantiles: Primetime Radio: 2nd Lowest Quintile(Q4)|Primetime Radio: 2nd Highest Quintile(Q2)|Primetime Radio: Middle Quintile(Q3) (17.2%) vs. Population Mode: Primetime Radio: Highest Quintile(Q1)|Primetime Radio: Lowest Quintile(Q5)",
      "internet_media_quantiles: Internet: Middle Quintile(Q3)|Internet: 2nd Lowest Quintile(Q4)|Internet: Lowest Quintile(Q5) (16.9%) vs. Population Mode: Internet: Lowest Quintile(Q5)|Internet: Highest Quintile(Q1)",
      "mobile_phones_number_in_household: 3|2|Any|4+ (4.5%) vs. Population Mode: 1"
    ]
  },
  {
    "Cluster_ID": 28,
    "Top_Differentiating_Features": [
      "equity_balance: Equity Balance High (62.5%) vs. Population Mode: MISSING",
      "urbanicity: Rural (50.4%) vs. Population Mode: Suburban",
      "credit_behavior: Up & Coming Consumers - HH that are relatively new to credit (37.2%) vs. Population Mode: Thriving Consumers - HH that have high spend & are active users of credit",
      "primetime_radio_media_quantiles: Primetime Radio: Highest Quintile(Q1)|Primetime Radio: 2nd Highest Quintile(Q2) (29.0%) vs. Population Mode: Primetime Radio: Highest Quintile(Q1)|Primetime Radio: Lowest Quintile(Q5)",
      "age: 55 - 64 (27.6%) vs. Population Mode: 35 - 44",
      "internet_media_quantiles: Internet: Highest Quintile(Q1)|Internet: Lowest Quintile(Q5) (17.1%) vs. Population Mode: Internet: Lowest Quintile(Q5)|Internet: Highest Quintile(Q1)",
      "age_gender_combination: 55-64 Female (14.3%) vs. Population Mode: 65+ Female",
      "family_income_decile: 6TH DECILE (12.6%) vs. Population Mode: 10TH (HIGHEST) DECILE",
      "mobile_phones_number_in_household: Any|3|4+ (11.4%) vs. Population Mode: 1"
    ]
  },
  {
    "Cluster_ID": 3,
    "Top_Differentiating_Features": [
      "equity_balance: Equity Balance High (61.7%) vs. Population Mode: MISSING",
      "mobile_phones_number_in_household: Any|2 (50.6%) vs. Population Mode: 1",
      "age: 65 - 74 (43.8%) vs. Population Mode: 35 - 44",
      "credit_tier: Prime to Super Prime - 4th Quartile (Highest Average Credit Score) (40.9%) vs. Population Mode: Sub Prime to Near Prime - 2nd Quartile",
      "net_worth: Net Worth - Greater than $499,999 (38.2%) vs. Population Mode: MISSING",
      "education_individual: College Graduates (24.3%) vs. Population Mode: MISSING",
      "internet_media_quantiles: Internet: Middle Quintile(Q3)|Internet: 2nd Lowest Quintile(Q4)|Internet: Lowest Quintile(Q5) (16.9%) vs. Population Mode: Internet: Lowest Quintile(Q5)|Internet: Highest Quintile(Q1)",
      "primetime_radio_media_quantiles: Primetime Radio: 2nd Highest Quintile(Q2)|Primetime Radio: 2nd Lowest Quintile(Q4)|Primetime Radio: Middle Quintile(Q3) (16.9%) vs. Population Mode: Primetime Radio: Highest Quintile(Q1)|Primetime Radio: Lowest Quintile(Q5)"
    ]
  },
  {
    "Cluster_ID": 19,
    "Top_Differentiating_Features": [
      "urbanicity: Urban (55.8%) vs. Population Mode: Suburban",
      "credit_tier: Sub Prime - 1st Quartile (Lowest Average Credit Score) (40.6%) vs. Population Mode: Sub Prime to Near Prime - 2nd Quartile",
      "credit_behavior: Up & Coming Consumers - HH that are relatively new to credit (39.9%) vs. Population Mode: Thriving Consumers - HH that have high spend & are active users of credit",
      "family_income_decile: 1ST (LOWEST) DECILE (23.3%) vs. Population Mode: 10TH (HIGHEST) DECILE",
      "age_gender_combination: 35-44 Male (14.9%) vs. Population Mode: 65+ Female"
    ]
  },
  {
    "Cluster_ID": 0,
    "Top_Differentiating_Features": [
      "equity_balance: Equity Balance High (52.3%) vs. Population Mode: MISSING",
      "primetime_radio_media_quantiles: Primetime Radio: 2nd Lowest Quintile(Q4)|Primetime Radio: Middle Quintile(Q3) (50.1%) vs. Population Mode: Primetime Radio: Highest Quintile(Q1)|Primetime Radio: Lowest Quintile(Q5)",
      "gender: Male (48.9%) vs. Population Mode: Female",
      "credit_tier: Near Prime to Prime - 3rd Quartile (32.5%) vs. Population Mode: Sub Prime to Near Prime - 2nd Quartile",
      "age: 55 - 64 (28.1%) vs. Population Mode: 35 - 44",
      "internet_media_quantiles: Internet: 2nd Highest Quintile(Q2)|Internet: Highest Quintile(Q1)|Internet: Middle Quintile(Q3) (16.9%) vs. Population Mode: Internet: Lowest Quintile(Q5)|Internet: Highest Quintile(Q1)",
      "age_gender_combination: 55-64 Female (14.0%) vs. Population Mode: 65+ Female",
      "mobile_phones_number_in_household: Any|2|4+|3 (4.5%) vs. Population Mode: 1"
    ]
  },
  {
    "Cluster_ID": 21,
    "Top_Differentiating_Features": [
      "gender: Male (48.1%) vs. Population Mode: Female",
      "urbanicity: Urban (35.2%) vs. Population Mode: Suburban",
      "credit_tier: Near Prime to Prime - 3rd Quartile (26.6%) vs. Population Mode: Sub Prime to Near Prime - 2nd Quartile",
      "age: 55 - 64 (24.8%) vs. Population Mode: 35 - 44",
      "primetime_radio_media_quantiles: Primetime Radio: Highest Quintile(Q1)|Primetime Radio: 2nd Lowest Quintile(Q4)|Primetime Radio: Middle Quintile(Q3) (17.2%) vs. Population Mode: Primetime Radio: Highest Quintile(Q1)|Primetime Radio: Lowest Quintile(Q5)",
      "age_gender_combination: 55-64 Male (12.6%) vs. Population Mode: 65+ Female",
      "mobile_phones_number_in_household: Any|2|3|4+ (4.5%) vs. Population Mode: 1",
      "internet_media_quantiles: Internet: 2nd Highest Quintile(Q2)|Internet: Middle Quintile(Q3)|Internet: Highest Quintile(Q1)|Internet: 2nd Lowest Quintile(Q4) (4.4%) vs. Population Mode: Internet: Lowest Quintile(Q5)|Internet: Highest Quintile(Q1)"
    ]
  }
]
# -------------------- HELPER FUNCTIONS --------------------
def create_feature_prompt(cluster):
    cluster_id = cluster["Cluster_ID"]
    features_text = "\n".join(cluster["Top_Differentiating_Features"])

    # The Defining_Persona_Nickname is hardcoded as 'TBD' in the structure
    # since we are now relying solely on the LLM to create the Friendly_Persona_Name.
    prompt = f"""
You are a senior marketing analyst. Analyze the following customer cluster (Cluster ID: {cluster_id}).
This cluster is currently **unnamed (TBD)**.
1. Suggest a **marketing-friendly, persona-style name** for this cluster (2-4 words, appealing for business presentations). This will be the "Friendly_Persona_Name".
2. Provide a detailed explanation of each feature in 3-4 sentences, focused on marketing insights, customer behavior, and potential engagement strategies.
3. Output JSON in this required format. Ensure the data for each feature is extracted and included in the interpretation object. Note that "cluster_value" and "population_value" should extract the mode information from the raw feature text:

{{
  "Cluster_ID": {cluster_id},
  "Defining_Persona_Nickname": "TBD",
  "Friendly_Persona_Name": "<suggested_marketing_friendly_name>",
  "Top_Differentiating_Features": [
    {{
      "feature": "<feature_name>",
      "cluster_value": "<cluster_mode_and_percentage>",
      "population_value": "<population_mode_comparison>",
      "interpretation": "<detailed marketing insight>"
    }}
  ]
}}

Cluster Features:
{features_text}
"""
    return prompt.strip()


def convert_json_to_docx(report_json, filename="Cluster_Report.docx"):
    doc = Document()
    doc.add_heading("Cluster Personas Report", 0)

    for cluster in report_json:
        cluster_id = cluster.get("Cluster_ID")
        # Prioritize the LLM-generated friendly name
        nickname = cluster.get("Friendly_Persona_Name", f"Unnamed Cluster {cluster_id}")
        features = cluster.get("Top_Differentiating_Features", [])

        doc.add_heading(f"CLUSTER {cluster_id} — {nickname}", level=1)

        for feature in features:
            feature_name = feature.get("feature", "")
            interpretation = feature.get("interpretation", "")

            # Feature title in bold
            p = doc.add_paragraph()
            run = p.add_run(feature_name)
            run.bold = True
            run.font.size = Pt(12)

            # Interpretation below
            p2 = doc.add_paragraph(interpretation)
            p2.paragraph_format.space_after = Pt(8)

        doc.add_paragraph("")  # Blank line

    doc.save(filename)
    print(f"✅ Report saved to {filename}")


# -------------------- MAIN FUNCTION --------------------
def generate_cluster_report(clusters):
    aiplatform.init(project=PROJECT_NUMBER, location=REGION)
    endpoint = aiplatform.Endpoint(endpoint_name=ENDPOINT_ID)
    final_report = []

    for cluster in clusters:
        prompt = create_feature_prompt(cluster)
        instance = {
            "@requestFormat": "chatCompletions",
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": 2048
        }
        parameters = {"temperature": 0.3}

        generated_text = ""
        try:
            response = endpoint.predict(instances=[instance], parameters=parameters)
            if response.predictions:
                if isinstance(response.predictions, dict) and "choices" in response.predictions:
                    generated_text = response.predictions["choices"][0].get("message", {}).get("content", "")
                elif isinstance(response.predictions, list) and len(response.predictions) > 0:
                    prediction = response.predictions[0]
                    generated_text = prediction.get("choices", [{}])[0].get("message", {}).get("content", "") or \
                                     prediction.get("content", "")
        except Exception as e:
            print(f"❌ Error during prediction for Cluster {cluster['Cluster_ID']}: {e}")
            continue

        if generated_text:
            # Extract JSON from any text using regex
            match = re.search(r"\{(?:.|\n)*\}", generated_text)
            if match:
                json_str = match.group(0)
                try:
                    cluster_json = json.loads(json_str)
                    final_report.append(cluster_json)
                except json.JSONDecodeError as e:
                    print(f"❌ JSON parse error for Cluster {cluster['Cluster_ID']}: {e}")
                    print(json_str)
            else:
                print(f"⚠️ No JSON block found in prediction for Cluster {cluster['Cluster_ID']}")

    return final_report


# -------------------- EXECUTION --------------------
if __name__ == "__main__":
    json_report = generate_cluster_report(CLUSTER_PERSONAS_DATA)
    if json_report:
        convert_json_to_docx(json_report)
    else:
        print("⚠️ No report generated.")

# **Part-1**

In [ ]:
# ============================================================
# 📦 BUILD & SAVE PERSONAS PIPELINE (RUN ONCE)
# ============================================================

from google.cloud import bigquery
import pandas as pd, numpy as np, os
from collections import Counter
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples
from transformers import pipeline
import chromadb

# PARAMETERS
N_CLUSTERS = 10
MIN_CLUSTER_SIZE = 30
SILHOUETTE_Q = 0.30
SAMPLE_FRACTION = 0.001
MAX_PERSONA_SAMPLE = 100
PERSIST_DIR = "/content/chroma_store"

# BigQuery load
client = bigquery.Client(project="stgw-shared-ai-dev")
query = f"""
SELECT *
FROM `stgw-shared-ai-dev.parquet_data_set.clustered_profiles_imputed_only_cleaned`
WHERE RAND() < {SAMPLE_FRACTION}
LIMIT 5000
"""
df = client.query(query).to_dataframe()
print("✅ Loaded sample rows:", df.shape)

# ---------------------------
# 2. Column groups (same as before)
# ---------------------------
groups = {
    "demographics": [
        "gender_imputed","age_imputed","generations_imputed","marital_status_imputed",
        "urbanicity_imputed","education_individual_imputed","ethnicity_imputed","language_spoken_imputed"
    ],
    "financial": [
        "family_income_decile_imputed","average_household_income_imputed","net_worth_imputed",
        "credit_tier_imputed","credit_card_spend_imputed","credit_card_balance_imputed","financial_beliefs_imputed",
        "discretionary_spend_imputed","wealth_rating_imputed","family_level_of_investable_assets_imputed",
        "home_equity_imputed","loan_imputed","mortgage_loan_imputed","auto_loan_imputed",
        "investments_type_imputed","financial_institutions_imputed"
    ],
    "property_home": [
        "home_ownership_imputed","dwelling_unit_size_imputed","building_type_imputed",
        "property_size_imputed","property_realty_indicator_imputed","current_home_value_imputed",
        "number_of_bedrooms_imputed","number_of_bathrooms_imputed","heating_type_imputed","air_conditioning_imputed"
    ],
    "technology": [
        "cell_smartphones_tablets_imputed","mobile_devices_imputed","mobile_phone_brand_imputed",
        "mobile_plan_type_imputed","i_device_types_imputed","i_device_models_imputed","i_device_oss_imputed",
        "streaming_system_device_use_imputed","technology_imputed","computer_purchase_imputed",
        "tablets_imputed","i_app_taxonomies_imputed"
    ],
    "shopping": [
        "shopping_imputed","stores_imputed","commerce_retail_shopping_imputed","department_discount_warehouse_imputed",
        "apparel_imputed","shoes_imputed","coupons_used_imputed","mail_purchases_imputed"
    ],
    "food_grocery": [c for c in df.columns if c.startswith("grocery_")],
    "lifestyle": [
        "leisure_activities_hobbies_imputed","health_and_fitness_imputed","green_living_imputed",
        "auto_imputed","sports_interests_imputed","entertainment_interests_imputed",
        "travel_interests_imputed","children_family_imputed"
    ],
    "media": [
        "television_shows_imputed","entertainment_sites_imputed","sports_sites_imputed",
        "news_sites_imputed","social_sites_imputed","i_iab_categories_imputed","movie_genre_saw_in_theater_imputed"
    ],
    "political_social": [
        "voting_behavior_imputed","political_affiliations_imputed","political_outlook_imputed",
        "religion_imputed","charitable_contributions_imputed"
    ]
}

for g in groups:
    groups[g] = [c for c in groups[g] if c in df.columns]
all_signal_cols = [c for cols in groups.values() for c in cols]

# ---------------------------
# 3. Persona text build
# ---------------------------
def describe_section(df_row, cols, cap=3):
    vals = df_row[cols].dropna().astype(str).tolist()
    vals = [v for v in vals if v.lower() not in ("none","nan","__missing__")]
    uniq = []
    for v in vals:
        if v not in uniq:
            uniq.append(v)
        if len(uniq) >= cap:
            break
    return ", ".join(uniq) if uniq else ""

def build_row_persona(r):
    parts = []
    for theme, cols in groups.items():
        text = describe_section(r, cols, cap=3)
        if text:
            parts.append(f"{theme}: {text}")
    return " | ".join(parts)

df["row_persona_text"] = df.apply(build_row_persona, axis=1)

# ---------------------------
# 4. Embedding + clustering
# ---------------------------
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
row_embeddings = embed_model.encode(df["row_persona_text"].tolist(), show_progress_bar=True)

kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=42, n_init=10)
df["recluster_label"] = kmeans.fit_predict(row_embeddings)

valid_clusters = df["recluster_label"].value_counts()[lambda x: x >= MIN_CLUSTER_SIZE].index
df = df[df["recluster_label"].isin(valid_clusters)].copy()

sil = silhouette_samples(row_embeddings, df["recluster_label"])
df["silhouette"] = sil
df_core = (
    df.groupby("recluster_label", group_keys=False)
    .apply(lambda g: g[g["silhouette"] > g["silhouette"].quantile(SILHOUETTE_Q)])
    .reset_index(drop=True)
)

# ---------------------------
# 5. Cluster summaries
# ---------------------------
def mode_safe(s): s = s.dropna().astype(str); return s.mode().iloc[0] if not s.mode().empty else None

cluster_summary = []
for cid, grp in df_core.groupby("recluster_label"):
    row = {"cluster": int(cid), "size": len(grp)}
    for c in all_signal_cols:
        row[c] = mode_safe(grp[c])
    row["persona_snippet"] = " | ".join(grp["row_persona_text"].head(MAX_PERSONA_SAMPLE).tolist())[:1000]
    cluster_summary.append(row)

cluster_summary = pd.DataFrame(cluster_summary)

# ---------------------------
# 6. Summarization
# ---------------------------
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6", device=-1)

def make_prompt_and_summarize(r):
    prompt = f"""
    Gender: {r.get('gender_imputed','Unknown')}.
    Age: {r.get('age_imputed','Unknown')}.
    Urbanicity: {r.get('urbanicity_imputed','Unknown')}.
    Income: {r.get('family_income_decile_imputed','Unknown')}.
    Credit tier: {r.get('credit_tier_imputed','Unknown')}.
    Traits: {r.get('persona_snippet','')[:400]}
    """
    try:
        out = summarizer(prompt, max_length=80, min_length=20, truncation=True)
        return out[0]['summary_text']
    except:
        return prompt[:200]

cluster_summary["persona_text_clean"] = cluster_summary.apply(make_prompt_and_summarize, axis=1)

# ---------------------------
# 7. Save to Chroma (persistent)
# ---------------------------
os.makedirs(PERSIST_DIR, exist_ok=True)
chroma_client = chromadb.PersistentClient(path=PERSIST_DIR)
try: chroma_client.delete_collection("cluster_personas_clean_v2")
except: pass

collection = chroma_client.create_collection("cluster_personas_clean_v2")
cluster_embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
cluster_embeddings = cluster_embed_model.encode(cluster_summary["persona_text_clean"].tolist(), show_progress_bar=True)

collection.add(
    ids=[str(i) for i in cluster_summary["cluster"].tolist()],
    embeddings=[list(map(float, e)) for e in cluster_embeddings],
    documents=cluster_summary["persona_text_clean"].tolist(),
    metadatas=cluster_summary.to_dict(orient="records")
)

cluster_summary.to_csv("/content/cluster_summary_v2.csv", index=False)
print("✅ Saved Chroma + CSV to /content/chroma_store and /content/cluster_summary_v2.csv")


#**STEP-2**

In [ ]:
# ============================================================
# 🔍 QUERY PERSONAS PIPELINE (REUSABLE)
# ============================================================

import pandas as pd
from sentence_transformers import SentenceTransformer
import chromadb

# Load stored data
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
chroma_client = chromadb.PersistentClient(path="/content/chroma_store")
collection = chroma_client.get_collection("cluster_personas_clean_v2")
cluster_summary = pd.read_csv("/content/cluster_summary_v2.csv")

print(f"✅ Loaded {collection.count()} personas for querying")

# Query function
def query_clusters_natural(user_query, top_k=5):
    q_emb = embed_model.encode([user_query]).tolist()
    res = collection.query(query_embeddings=q_emb, n_results=top_k)

    results = []
    for i, cid in enumerate(res["ids"][0]):
        meta = res["metadatas"][0][i]
        doc = res["documents"][0][i]
        results.append({
            "cluster": meta.get("cluster"),
            "score": res["distances"][0][i],
            "size": meta.get("size"),
            "summary": doc,
            "age": meta.get("age_imputed"),
            "gender": meta.get("gender_imputed"),
            "urbanicity": meta.get("urbanicity_imputed"),
            "income": meta.get("family_income_decile_imputed"),
        })
    return pd.DataFrame(results)

# Example use
print(query_clusters_natural("Which Gen Zs are likely to buy Apple products or luxury brands?"))
#print(query_clusters_natural("Which groups are investing in real estate and property?"))


# **LOAD SAVED PERSONAS**

In [ ]:
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer

# Load persona summaries
cluster_summary = pd.read_csv("/content/cluster_summary_v2.csv")
print("✅ Loaded cluster summaries:", cluster_summary.shape)

# Connect to Chroma
PERSIST_DIR = "/content/chroma_store"
chroma_client = chromadb.PersistentClient(path=PERSIST_DIR)

# Load the same collection name as before
collection = chroma_client.get_collection("cluster_personas_clean_v2")

# Embedding model (must match what was used before)
embed_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")


# 2️⃣ Query Personas by Text Meaning

In [ ]:
def search_personas(query_text, top_k=3):
    query_embedding = embed_model.encode([query_text]).tolist()
    results = collection.query(query_embeddings=query_embedding, n_results=top_k)

    print(f"🔎 Query: {query_text}\n")
    for doc, meta, score in zip(results["documents"][0], results["metadatas"][0], results["distances"][0]):
        print(f"🎯 Cluster {meta['cluster']} (size={meta['size']}) — Distance={score:.3f}")
        print(meta.get("persona_text_clean", "")[:250])
        print("="*80)

# Example queries
search_personas("tech-savvy shoppers who love online streaming")
#search_personas("rural families with auto loans and modest income")
#search_personas("wealthy urban professionals with high spending on luxury goods")


# Visualize Persona Space

In [ ]:
import pandas as pd
import plotly.express as px

# Load your CSV
df = pd.read_csv("/content/cluster_summary_v2.csv")

# Truncate long text to keep hover tooltips readable
df["persona_text_clean_short"] = df["persona_text_clean"].astype(str).str.slice(0, 300) + "..."

# Create scatter plot (or switch to bar if you’re plotting counts)
fig = px.scatter(
    df,
    x="cluster",
    y="size",
    color="cluster",
    hover_name="cluster",
    hover_data={
        "size": True,
        "gender_imputed": True,
        "age_imputed": True,
        "urbanicity_imputed": True,
        "family_income_decile_imputed": True,
        "persona_text_clean_short": True,  # the truncated summary
    },
    title="Cluster Profiles Overview",
)

# Improve layout
fig.update_traces(marker=dict(size=12, line=dict(width=1, color="DarkSlateGrey")))
fig.update_layout(
    hoverlabel=dict(bgcolor="white", font_size=12),
    xaxis_title="Cluster Number",
    yaxis_title="Cluster Size",
)

fig.show()


# **CHAT WITH SELECTED PERSONA**

In [ ]:
# ============================================================
# 💬 CLUSTER PERSONA CHAT MODULE (Persistent Persona Name)
# ============================================================

from transformers import pipeline
import random

# ------------------------------------------------------------
# 1. Model setup
# ------------------------------------------------------------
chat_model = pipeline(
    "text-generation",
    model="mistralai/Mistral-7B-Instruct-v0.2",
    max_new_tokens=200,
    do_sample=True,
    temperature=0.7,
    pad_token_id=2
)

# ------------------------------------------------------------
# 2. Persona name generator
# ------------------------------------------------------------
def generate_persona_name(cluster_id, gender, age):
    male_names = ["Raj", "Amit", "Rohan", "Vikram", "Aditya", "Sanjay", "Arjun", "Kunal"]
    female_names = ["Priya", "Ananya", "Sneha", "Neha", "Isha", "Kavya", "Ritu", "Aparna"]
    neutral_names = ["Alex", "Taylor", "Jordan", "Riya", "Sam", "Kiran"]

    if isinstance(gender, str):
        gender_lower = gender.lower()
        if "male" in gender_lower:
            name = random.choice(male_names)
        elif "female" in gender_lower:
            name = random.choice(female_names)
        else:
            name = random.choice(neutral_names)
    else:
        name = random.choice(neutral_names)

    persona_title = f"Cluster {cluster_id} Persona ({name}, {age})"
    return name, persona_title


# ------------------------------------------------------------
# 3. Persona context builder (no re-randomization mid-chat)
# ------------------------------------------------------------
def get_persona_context(cluster_id, persona_name=None):
    row = cluster_summary[cluster_summary["cluster"] == cluster_id]
    if row.empty:
        return None, None

    persona = row.iloc[0]
    # Only generate new name if not provided (keeps it stable per chat)
    if persona_name is None:
        name, persona_title = generate_persona_name(
            cluster_id,
            persona.get("gender_imputed", "Unknown"),
            persona.get("age_imputed", "Unknown"),
        )
    else:
        name = persona_name
        persona_title = f"Cluster {cluster_id} Persona ({name}, {persona.get('age_imputed', 'Unknown')})"

    persona_context = f"""
You are {name}, representing behavioral Cluster {cluster_id}.
This persona is summarized as:
"{persona.get('persona_text_clean', '')}"

Demographics:
- Age Group: {persona.get('age_imputed', 'Unknown')}
- Gender: {persona.get('gender_imputed', 'Unknown')}
- Urbanicity: {persona.get('urbanicity_imputed', 'Unknown')}

Financial Profile:
- Income Decile: {persona.get('family_income_decile_imputed', 'Unknown')}
- Credit Tier: {persona.get('credit_tier_imputed', 'Unknown')}

Lifestyle & Behavior:
{persona.get('persona_snippet', '')[:500]}

Now, respond conversationally *as {name}*.
Use natural language, reflect realistic attitudes and habits typical of this cluster.
Avoid restating your profile directly — speak as a real person would.
"""
    return persona_context.strip(), name


# ------------------------------------------------------------
# 4. Chat function (persona name persists)
# ------------------------------------------------------------
def chat_with_persona(cluster_id, user_input, chat_history=None, persona_name=None):
    if chat_history is None:
        chat_history = []

    persona_context, name = get_persona_context(cluster_id, persona_name)
    if not persona_context:
        return "❌ Cluster not found.", chat_history, persona_name

    dialogue_context = "\n".join([f"User: {u}\n{name}: {r}" for u, r in chat_history])
    full_prompt = f"{persona_context}\n\n{dialogue_context}\nUser: {user_input}\n{name}:"

    output = chat_model(full_prompt)
    raw_reply = output[0]["generated_text"]
    reply = raw_reply.split(f"{name}:")[-1].strip()

    chat_history.append((user_input, reply))
    return reply, chat_history, name


# ------------------------------------------------------------
# 5. Example Conversation (Persistent Persona)
# ------------------------------------------------------------
cluster_id = 3
chat_history = []
persona_name = None  # Will be set after first call

print(f"🗣️ Chatting with Cluster {cluster_id} Persona...\n")

reply, chat_history, persona_name = chat_with_persona(cluster_id, "Hey! How do you usually spend your weekends?", chat_history, persona_name)
print(f"{persona_name}:", reply)

reply, chat_history, persona_name = chat_with_persona(cluster_id, "Which brands do you like to shop from?", chat_history, persona_name)
print(f"{persona_name}:", reply)

reply, chat_history, persona_name = chat_with_persona(cluster_id, "Do you invest in property or mutual funds?", chat_history, persona_name)
print(f"{persona_name}:", reply)


**STREAMLIT**

In [ ]:
%%writefile cluster_persona_app.py
# ============================================================
# 🌐 STREAMLIT CLUSTER PERSONA CHAT APP
# ============================================================

import streamlit as st
import pandas as pd
import random
from transformers import pipeline

# Assuming these are preloaded somewhere in Colab:
#   collection  (Chroma/FAISS)
#   embed_model (SentenceTransformer)
#   cluster_summary (DataFrame)

@st.cache_resource
def load_chat_model():
    return pipeline(
        "text-generation",
        model="mistralai/Mistral-7B-Instruct-v0.2",
        max_new_tokens=200,
        do_sample=True,
        temperature=0.7,
        pad_token_id=2
    )

chat_model = load_chat_model()

def generate_persona_name(gender):
    male_names = ["Raj", "Amit", "Rohan", "Vikram", "Aditya", "Sanjay", "Arjun", "Kunal"]
    female_names = ["Priya", "Ananya", "Sneha", "Neha", "Isha", "Kavya", "Ritu", "Aparna"]
    neutral_names = ["Alex", "Taylor", "Jordan", "Riya", "Sam", "Kiran"]

    if isinstance(gender, str):
        gender_lower = gender.lower()
        if "male" in gender_lower:
            return random.choice(male_names)
        elif "female" in gender_lower:
            return random.choice(female_names)
        else:
            return random.choice(neutral_names)
    return random.choice(neutral_names)


def get_persona_context(cluster_id, persona_name=None):
    row = cluster_summary[cluster_summary["cluster"] == cluster_id]
    if row.empty:
        return None, None
    persona = row.iloc[0]
    name = persona_name or generate_persona_name(persona.get("gender_imputed", "Unknown"))
    persona_context = f"""
You are {name}, representing behavioral Cluster {cluster_id}.
Summary: "{persona.get('persona_text_clean', '')}"

Demographics:
- Age Group: {persona.get('age_imputed', 'Unknown')}
- Gender: {persona.get('gender_imputed', 'Unknown')}
- Urbanicity: {persona.get('urbanicity_imputed', 'Unknown')}

Financial Profile:
- Income Decile: {persona.get('family_income_decile_imputed', 'Unknown')}
- Credit Tier: {persona.get('credit_tier_imputed', 'Unknown')}

Lifestyle & Behavior:
{persona.get('persona_snippet', '')[:400]}

Respond conversationally *as {name}*.
"""
    return persona_context.strip(), name


def chat_with_persona(cluster_id, user_input, chat_history=None, persona_name=None):
    if chat_history is None:
        chat_history = []
    persona_context, name = get_persona_context(cluster_id, persona_name)
    if not persona_context:
        return "❌ Cluster not found.", chat_history, persona_name

    dialogue_context = "\n".join([f"User: {u}\n{name}: {r}" for u, r in chat_history])
    full_prompt = f"{persona_context}\n\n{dialogue_context}\nUser: {user_input}\n{name}:"
    output = chat_model(full_prompt)
    raw_reply = output[0]["generated_text"]
    reply = raw_reply.split(f"{name}:")[-1].strip()
    chat_history.append((user_input, reply))
    return reply, chat_history, name


def search_personas(query_text, top_k=3):
    query_embedding = embed_model.encode([query_text]).tolist()
    results = collection.query(query_embeddings=query_embedding, n_results=top_k)
    matches = []
    for doc, meta, score in zip(results["documents"][0], results["metadatas"][0], results["distances"][0]):
        matches.append({
            "Cluster": meta["cluster"],
            "Distance": round(score, 3),
            "Size": meta.get("size", "N/A"),
            "Summary": meta.get("persona_text_clean", "")[:250]
        })
    return pd.DataFrame(matches)


st.title("🧠 Cluster Persona Explorer & Chat")
st.markdown("Search behavioral clusters and chat with a persona.")

query_text = st.text_input("Enter a query (e.g., 'tech-savvy shoppers who love online streaming')")
if query_text:
    df = search_personas(query_text)
    st.subheader("🔍 Top Matching Clusters")
    st.dataframe(df)

    cluster_options = df["Cluster"].tolist()
    selected_cluster = st.selectbox("Select a cluster to chat with", cluster_options)
    if selected_cluster:
        if "chat_history" not in st.session_state:
            st.session_state.chat_history = []
        if "persona_name" not in st.session_state:
            st.session_state.persona_name = None

        st.markdown("---")
        st.subheader(f"💬 Chat with Cluster {selected_cluster} Persona")

        user_input = st.chat_input("Say something to your persona...")
        if user_input:
            reply, st.session_state.chat_history, st.session_state.persona_name = chat_with_persona(
                selected_cluster,
                user_input,
                st.session_state.chat_history,
                st.session_state.persona_name
            )
        for u, r in st.session_state.chat_history:
            st.chat_message("user").write(u)
            st.chat_message("assistant").write(r)


In [ ]:
!pip install streamlit


In [ ]:
!npm install -g localtunnel --silent


In [ ]:
# Start localtunnel with your own password
!wget -q -O - ipv4.icanhazip.com

In [ ]:
!streamlit run cluster_persona_app.py --server.port 6006 & npx localtunnel --port 6006


In [ ]:
!nohup streamlit run cluster_persona_app.py --server.port 6006 >/dev/null 2>&1 &


In [ ]:
!cloudflared tunnel --url http://localhost:6006 --no-autoupdate


In [ ]:
import chromadb

CHROMA_PATH = "chroma_persona_store"
chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)

# List all collections in this persistent store
collections = chroma_client.list_collections()
print("Collections in store:", collections)


: 